In [1]:
import pandas as pd

import geopandas as gpd
import folium
from folium import plugins

In [2]:
import funciones_kwichon as f

In [3]:
from sklearn.model_selection import train_test_split    
from sklearn.metrics import classification_report
from sklearn.preprocessing import OrdinalEncoder

In [4]:
from xgboost import XGBClassifier

# PREDICCIÓN DE LA PROVINCIA DE DESTINO.

## Tablas y datos necesarios para la predicción

### Tabla de las migraciones del 2006 al 2021 (excluyendo desde o hacia el extranjero)

In [5]:
solo_migr = pd.read_csv('./csv_files/solo_migr_2006_2021.csv', index_col=0)
solo_migr

,SEXO,PROVNAC,EDAD,PROVALTA,MUNIALTA,ANOVAR,PROVBAJA,MUNIBAJA,TAMUALTA,TAMUBAJA,CODMUNIALTA,CODMUNIBAJA
0,1,1,2,48,20,2006,1,0,6,1,48020,1000
1,1,1,53,1,0,2006,1,59,1,6,1000,1059
2,1,1,0,25,0,2006,25,203,1,2,25000,25203
4,1,1,65,28,0,2006,28,79,1,6,28000,28079
5,6,1,3,1,0,2006,1,59,1,6,1000,1059
...,...,...,...,...,...,...,...,...,...,...,...,...
39922030,1,66,26,50,297,2021,52,1,6,4,50297,52001
39922031,1,66,25,50,297,2021,52,1,6,4,50297,52001
39922032,1,66,45,50,297,2021,52,1,6,4,50297,52001
39922033,1,66,31,51,1,2021,52,1,4,4,51001,52001


### Se hará la predicción con los datos de los años 2020 y 2021.

In [6]:
year_filter = [y for y in range(2020,2021+1)]
solo_migr = solo_migr[(solo_migr['ANOVAR'].isin(year_filter))]

In [7]:
solo_migr

,SEXO,PROVNAC,EDAD,PROVALTA,MUNIALTA,ANOVAR,PROVBAJA,MUNIBAJA,TAMUALTA,TAMUBAJA,CODMUNIALTA,CODMUNIBAJA
35888342,1,1,79,1,59,2020,1,0,6,1,1059,1000
35888343,1,1,66,1,59,2020,1,0,6,1,1059,1000
35888344,1,1,18,1,59,2020,1,0,6,1,1059,1000
35888345,6,1,15,1,59,2020,1,0,6,1,1059,1000
35888346,6,1,10,1,0,2020,1,59,1,6,1000,1059
...,...,...,...,...,...,...,...,...,...,...,...,...
39922030,1,66,26,50,297,2021,52,1,6,4,50297,52001
39922031,1,66,25,50,297,2021,52,1,6,4,50297,52001
39922032,1,66,45,50,297,2021,52,1,6,4,50297,52001
39922033,1,66,31,51,1,2021,52,1,4,4,51001,52001


### Se prepara una tabla con los datos necesarios para la predicción

In [8]:
# Se escogen columnas necesarias para la predicción
df_pred = solo_migr[['SEXO','PROVNAC','EDAD','PROVBAJA','PROVALTA']]
df_pred

,SEXO,PROVNAC,EDAD,PROVBAJA,PROVALTA
35888342,1,1,79,1,1
35888343,1,1,66,1,1
35888344,1,1,18,1,1
35888345,6,1,15,1,1
35888346,6,1,10,1,1
...,...,...,...,...,...
39922030,1,66,26,52,50
39922031,1,66,25,52,50
39922032,1,66,45,52,50
39922033,1,66,31,52,51


In [9]:
# Se prepara una lista ordenada con las provincias de la columna PROVALTA para aplicar un OrdinalEncoder 
# (necesario para la predicción con XGBOOST que necesita que los valores a predecir sean número consecutivos que 
# vayan del 0 a numero de elementos-1)

cat = df_pred['PROVALTA'].unique().tolist()
cat.sort()

# Se crea nueva columna ORD para indicar los valores generados por el OrdinalEncoder.
df_pred['ORD'] = OrdinalEncoder(categories=[cat]).fit_transform(df_pred[['PROVALTA']])
df_pred

,SEXO,PROVNAC,EDAD,PROVBAJA,PROVALTA,ORD
35888342,1,1,79,1,1,0.0
35888343,1,1,66,1,1,0.0
35888344,1,1,18,1,1,0.0
35888345,6,1,15,1,1,0.0
35888346,6,1,10,1,1,0.0
...,...,...,...,...,...,...
39922030,1,66,26,52,50,49.0
39922031,1,66,25,52,50,49.0
39922032,1,66,45,52,50,49.0
39922033,1,66,31,52,51,50.0


## Predicción con XGBOOST

In [10]:
#Se prepara la variable X excluyendo los valores de la predicción

X=df_pred.drop(columns=['PROVALTA','ORD'])
X

,SEXO,PROVNAC,EDAD,PROVBAJA
35888342,1,1,79,1
35888343,1,1,66,1
35888344,1,1,18,1
35888345,6,1,15,1
35888346,6,1,10,1
...,...,...,...,...
39922030,1,66,26,52
39922031,1,66,25,52
39922032,1,66,45,52
39922033,1,66,31,52


In [11]:
# Variable y con la columna a predecir
y=df_pred['ORD']

In [12]:
# Se preparan las variables para entrenar el modelo y para testear
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=3)

## Modelo XBClassifier con máxima profundidad = 3

In [13]:
# Se instancia el modelo, se escoge una profundidad máxima de 3 para ver resultados ya que son 3 millones de filas
# Se dejan el resto de parámetros por defecto.
model_3d = XGBClassifier(max_depth=3)

In [14]:
# Se entrena el modelo
model_3d.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=3, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, objective='multi:softprob',
              predictor='auto', ...)

In [15]:
# Se predice con las variables de test
y_pred_3d = model_3d.predict(X_test)

In [16]:
# Resultados aceptables para tratarse de tantas clases (son 52 posibles provincias de destino)
print(classification_report(y_pred_3d, y_test))

              precision    recall  f1-score   support

           0       0.34      0.44      0.38      2539
           1       0.41      0.41      0.41      3595
           2       0.61      0.63      0.62     25546
           3       0.57      0.63      0.60      9190
           4       0.17      0.44      0.24       997
           5       0.52      0.52      0.52      5557
           6       0.58      0.66      0.62     17377
           7       0.78      0.71      0.74     95552
           8       0.37      0.50      0.43      3278
           9       0.42      0.48      0.45      3545
          10       0.53      0.48      0.51     12274
          11       0.51      0.60      0.55      7180
          12       0.40      0.43      0.42      4586
          13       0.46      0.41      0.43      5301
          14       0.70      0.65      0.67     15058
          15       0.33      0.40      0.36      2359
          16       0.57      0.64      0.60     13155
          17       0.65    

In [17]:
# Se guarda el modelo
pd.to_pickle(model_3d, "./xgb_files/xgboost_prov_3d.pkl")  

### Lectura del modelo guardado

In [18]:
# Lectura del modelo para no tener que volver a entrenar
model_3d = pd.read_pickle('./xgb_files/xgboost_prov_3d.pkl')

In [19]:
# Se predice el total de la tabla
y_pred_total_3d = model_3d.predict(X)

In [20]:
# Resultados parecidos al test.
print(classification_report(y_pred_total_3d, y))

              precision    recall  f1-score   support

           0       0.34      0.44      0.38     13038
           1       0.42      0.42      0.42     17720
           2       0.62      0.63      0.62    126809
           3       0.57      0.62      0.60     45733
           4       0.16      0.43      0.24      4984
           5       0.52      0.52      0.52     27847
           6       0.58      0.66      0.62     86882
           7       0.78      0.71      0.74    476363
           8       0.38      0.50      0.43     16535
           9       0.43      0.49      0.46     17705
          10       0.54      0.49      0.52     61983
          11       0.51      0.61      0.56     35598
          12       0.42      0.44      0.43     22985
          13       0.45      0.40      0.43     26087
          14       0.70      0.66      0.68     76080
          15       0.34      0.40      0.36     11893
          16       0.58      0.65      0.61     65998
          17       0.65    

## Modelo XGBClassifier con máxima profundidad = 6

In [21]:
model_6d = XGBClassifier(max_depth=6)

In [22]:
model_6d.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, objective='multi:softprob',
              predictor='auto', ...)

In [23]:
y_pred_6d = model_6d.predict(X_test)

In [24]:
print(classification_report(y_pred_6d, y_test))

              precision    recall  f1-score   support

           0       0.35      0.44      0.39      2707
           1       0.43      0.41      0.42      3769
           2       0.62      0.63      0.62     25455
           3       0.60      0.62      0.61      9758
           4       0.19      0.44      0.26      1114
           5       0.54      0.51      0.53      5849
           6       0.57      0.68      0.62     16650
           7       0.77      0.72      0.75     93227
           8       0.41      0.49      0.45      3638
           9       0.46      0.47      0.47      4011
          10       0.54      0.48      0.51     12361
          11       0.51      0.61      0.56      7134
          12       0.44      0.43      0.44      4990
          13       0.47      0.41      0.44      5429
          14       0.70      0.66      0.68     14963
          15       0.34      0.39      0.37      2466
          16       0.56      0.65      0.60     12888
          17       0.66    

In [25]:
# Se guarda el modelo
pd.to_pickle(model_6d, "./xgb_files/xgboost_prov_6d.pkl")  
pd.to_pickle(model_6d, "./kwichon_streamlit/xgboost_prov_6d.pkl")   # para utilizar con streamlit

### Lectura del modelo guardado

In [26]:
# Lectura del modelo para no tener que volver a entrenar.
model_pred= pd.read_pickle("./xgb_files/xgboost_prov_6d.pkl")

In [27]:
y_pred_total_6d = model_pred.predict(X)

In [28]:
print(classification_report(y_pred_total_6d, y))

              precision    recall  f1-score   support

           0       0.37      0.45      0.40     13744
           1       0.45      0.43      0.44     18675
           2       0.62      0.64      0.63    126324
           3       0.60      0.62      0.61     48457
           4       0.19      0.45      0.27      5501
           5       0.55      0.52      0.53     29352
           6       0.57      0.68      0.62     83371
           7       0.78      0.72      0.75    464637
           8       0.42      0.49      0.45     18388
           9       0.48      0.49      0.48     19982
          10       0.55      0.50      0.52     62415
          11       0.52      0.62      0.56     35546
          12       0.46      0.44      0.45     24835
          13       0.47      0.41      0.44     26609
          14       0.70      0.67      0.68     75419
          15       0.36      0.40      0.38     12378
          16       0.57      0.66      0.61     64451
          17       0.67    

### Se escoge este último modelo. No se hacen más pruebas ya que la diferencia de margen de error es muy poca respecto al anterior

# Recomendador de pueblos
### Se preguntará al usuario la edad, sexo, provincia de nacimiento y provincia de vivienda actual. Se predicen las 3 provincias de destino más probables teniendo en cuenta los valores de los años 2020 y 2021 y se presentará en un mapa los pueblos de esas provincias que aparecen en las webs www.venteaviviraunpueblo.com y www.volveralpueblo.org para que pueda explorar cual podría ser su destino

### Preparación de tablas necesarias

Tabla de códigos de provincia

In [29]:
cod_prov = pd.read_csv('./csv_files/cod_prov.csv',index_col=0)

In [30]:
cod_prov.set_index('CPRO', inplace=True)
cod_prov

,NOMBRE
CPRO,
1,Araba/Álava
2,Albacete
3,Alicante/Alacant
4,Almería
5,Ávila
6,Badajoz
7,"Balears, Illes"
8,Barcelona
9,Burgos


In [31]:
# Se examinan las probabilidades con un predict_proba para escoger las 3 provincias más probables de destino
# dando los valores de: SEXO, EDAD, PROVINCIA DE NACIMIENTO Y PROVINCIA EN LA QUE VIVE

In [32]:
# Se coge al azar, como ejemplo, la fila 139 de la variable X_test
individuo = X_test.iloc[[139]]
individuo

,SEXO,PROVNAC,EDAD,PROVBAJA
36576433,1,29,37,29


In [33]:
# Destino real
destino_indiv=y_test.iloc[[139]]
destino_indiv

36576433    28.0
Name: ORD, dtype: float64

In [34]:
# Probabilidades para cada provincia.
resultado = model_pred.predict_proba(individuo).ravel() # ravel convierte a array de 1 dimensión (está de 2 dimensiones)
resultado

array([3.2511662e-04, 3.8890197e-04, 2.5343567e-03, 4.4735973e-03,
       6.2244019e-04, 8.5347652e-04, 6.5783053e-03, 9.1788312e-03,
       4.5544107e-04, 5.3788139e-04, 3.0174853e-02, 9.0751704e-04,
       1.1295993e-03, 5.6745675e-03, 1.1069915e-03, 2.2322766e-04,
       9.8906155e-04, 1.8794984e-02, 9.6466858e-04, 8.0454588e-04,
       1.4477876e-03, 8.0062851e-04, 3.0947123e-03, 8.9608826e-04,
       5.3214777e-04, 4.4850007e-04, 3.0282780e-04, 2.8851779e-02,
       8.1963295e-01, 3.6767006e-03, 1.0902877e-03, 4.5114430e-04,
       1.4872132e-03, 1.5333509e-04, 4.9463739e-03, 7.6670578e-04,
       4.1234840e-04, 4.9087913e-03, 1.1547977e-03, 2.8105415e-04,
       1.7366489e-02, 2.1535507e-04, 1.2956614e-03, 1.6321408e-04,
       2.0280422e-03, 4.3897149e-03, 6.6741701e-04, 1.1062259e-03,
       2.2223844e-04, 1.7755700e-03, 4.1250889e-03, 4.5904508e-03],
      dtype=float32)

In [35]:
# Se ordenan de mayor a menor las probabilidades del resultado.
# El índice es el mismo que la tabla cod_prov eliminando el valor 66(extranjero).
# Se seleccionan las 3 provincias con más probabilidad.
# El índice de la serie se corresponderá con el índice de la tabla cod_prov para saber el nombre.

sel =  pd.Series(resultado, index=cod_prov.index[0:52]).sort_values(ascending=False).head(3).index
sel

Int64Index([29, 11, 28], dtype='int64', name='CPRO')

In [36]:
# Nombre de las tres provincias más probables para este individuo
prov_destino = cod_prov.loc[sel,'NOMBRE']
prov_destino

CPRO
29    Málaga
11     Cádiz
28    Madrid
Name: NOMBRE, dtype: object

### Se prepara una lista de las provincias de destino posibles para mostrar más adelante en el mapa los pueblos.

In [37]:
lista_destino= prov_destino.index.tolist()
lista_destino

[29, 11, 28]

## Resultados en un mapa

### Se extrae de la web www.venteaviviraunpueblo.com el mapa de los pueblos promocionados

In [38]:
VenteGmap = {"mapID":"map","lat":"39.3262345","lng":"-4.8380649","markers":[{"title":"A Estrada","content":"La fortaleza de este pueblo de la provincia de Pontevedra se puede ver en la ...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/a-estrada\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"42.689068 ","lng":"-8.482822 "}},{"title":"A Mari\u00f1a Lucense","content":"Pocas tierras combinan el mar y la monta\u00f1a de manera tan perfecta como esta ...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/a-marina-lucense\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"43.535297","lng":"-7.045202"}},{"title":"Aceituna","content":"En la provincia de C\u00e1ceres encontramos este tranquilo y hermoso municipio en...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/aceituna\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"40.149099","lng":" -6.330698"}},{"title":"Aguilar de Campoo","content":"A los pies de la monta\u00f1a palentina encontramos este precioso pueblo lleno de...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/aguilar-de-campoo\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"42.791388","lng":"-4.260055"}},{"title":"Albares","content":"En la provincia de Guadalajara encontramos este acogedor pueblo castellano si...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/albares\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"40.308621","lng":"-3.011926"}},{"title":"Alcal\u00e1 del J\u00facar","content":"Son muchas las publicaciones y expertos los que consideran a este encantador ...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/alcala-del-jucar\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"39.192955","lng":"-1.429362"}},{"title":"Alc\u00e1ntara","content":"Un pueblo de pel\u00edcula para una vida de pel\u00edcula. As\u00ed es este precioso pueb...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/alcantara\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"39.716033","lng":"-6.885933"}},{"title":"Alcauc\u00edn","content":"Este ser\u00e1 posiblemente uno de los pueblos m\u00e1s visitados de la comarca malag...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/alcaucin\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"36.90298352412578","lng":"-4.114043326034021"}},{"title":"Alcublas","content":"Sus vecino dicen que \u00abes un gusto vivir aqu\u00ed\u00bb e invitan a todo el que quie...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/alcublas\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"39.797821","lng":"-0.704420"}},{"title":"Alcudia de Veo","content":"En pleno coraz\u00f3n del Parque Natural de la Sierra de Espad\u00e1n, en la provinci...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/alcudia-de-veo\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"39.916283","lng":"-0.355047"}},{"title":"Alfarnate","content":"En la comarca malague\u00f1a de la Axarquia encontramos este peque\u00f1o pueblo llen...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/alfarnate\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"36.993524","lng":"-4.255433"}},{"title":"Alfarnatejo","content":"Tan peque\u00f1o como agradable es este municipio malague\u00f1o perteneciente a la c...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/alfarnatejo\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"36.979203","lng":" -4.271991"}},{"title":"Algatoc\u00edn","content":"En medio de la naturaleza m\u00e1s hermosa de la Serran\u00eda de Ronda se encuentra ...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/algatocin\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"36.573557","lng":"-5.275038"}},{"title":"Alm\u00e1char","content":"En la comarca malague\u00f1a de la Axarqu\u00eda encontramos este hermoso y acogedor ...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/almachar\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"36.809020","lng":"-4.217358"}},{"title":"Almaraz","content":"En un entorno de gran belleza natural encontramos este pueblo de la provincia...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/almaraz\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"39.815057","lng":"-5.677116"}},{"title":"Almargen","content":"En la comarca malague\u00f1a de Guadalteba encontramos este acogedor pueblo con u...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/almargen\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"37.001850","lng":"-5.021827"}},{"title":"Alozaina","content":"Esta joya se encuentra casi escondida en el hermoso paraje del Parque Natural...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/alozaina\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"36.727903","lng":"-4.858497"}},{"title":"Alpandeire","content":"Por su privilegiada situaci\u00f3n en la Serran\u00eda de Ronda, no resulta extra\u00f1o ...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/alpandeire\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"36.634042","lng":"-5.203519"}},{"title":"Alpedrete","content":"En la Comunidad de Madrid se encuentra este hermoso pueblo, conocido por todo...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/alpedrete\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"40.661151","lng":"-4.031000"}},{"title":"Arenas","content":"En la comarca malague\u00f1a de la Axarqu\u00eda encontramos este precioso pueblo med...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/arenas\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"36.816212","lng":"-4.044055"}},{"title":"As Neves","content":"Al sur de la provincia de Pontevedra encontramos este hermoso pueblo gallego ...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/as-neves\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"42.088129","lng":"-8.414637"}},{"title":"Ayora","content":"En este pueblo de la provincia de Valencia, afirman que son amables y acogedo...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/ayora\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"39.059807","lng":"-1.057471"}},{"title":"Belorado","content":"Al este de la provincia de Burgos, entre hayedos y r\u00edos, encontramos a los p...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/belorado\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"42.420929","lng":" -3.190924"}},{"title":"Benadalid","content":"En la Serran\u00eda de Ronda podemos encontrar este peque\u00f1o pueblo lleno de enca...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/benadalid\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"36.606089","lng":"-5.269379"}},{"title":"Benahav\u00eds","content":"En M\u00e1laga, a tan solo 7 kil\u00f3metros de la costa encontramos este precioso pu...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/benahavis\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"36.521466","lng":"-5.045214"}},{"title":"Benalaur\u00eda","content":"Con los brazos abiertos est\u00e1n esperando a nuevos vecinos los habitantes de B...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/benalauria\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"36.593994","lng":"-5.260850"}},{"title":"Benamargosa","content":"En la comarca malague\u00f1a de la Axarqu\u00eda, en un entorno de cerros, lomas, hue...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/benamargosa\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"36.832779","lng":"-4.192723"}},{"title":"Benarrab\u00e1","content":"Un pueblo para vivir, para disfrutar y sobretodo \u00a1para teletrabajar! En la S...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/benarraba\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"36.550681","lng":"-5.274076"}},{"title":"Berlanga","content":"Al sur de la provincia de Badajoz encontramos este hermoso y cuidado pueblo e...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/berlanga\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"38.283339","lng":" -5.831091"}},{"title":"Blanca","content":"Un oasis, un vergel en el coraz\u00f3n del valle de Ricote. Este acogedor pueblo ...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/blanca\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"38.178963","lng":"-1.371303"}},{"title":"Botija","content":"En la provincia de C\u00e1ceres, en medio de la dehesa extreme\u00f1a, encontramos es...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/botija\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"39.344400","lng":"-6.073732"}},{"title":"Burgohondo","content":"En la provincia de \u00c1vila encontramos este precioso pueblo inmerso en un atra...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/burgohondo\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"40.413435","lng":"-4.786072"}},{"title":"Cadalso de los Vidrios","content":"En\u00a0 el sur oeste de Comunidad de Madrid encontramos este hist\u00f3rico municipi...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/cadalso-de-los-vidrios\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"40.301447","lng":"-4.443751"}},{"title":"Cam\u00edn Real de la Mesa","content":"Una impresionante y espectacular comarca compuesta por los pueblos de Belmont...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/camin-real-de-la-mesa\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"43.387414","lng":"-6.077684"}},{"title":"Campaspero","content":"En el sureste de la provincia de Valladolid, en la zona de m\u00e1s altitud de la...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/campaspero\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"41.491698","lng":"-4.197962 "}},{"title":"Campillo de Altobuey","content":"Este encantador pueblo de la provincia de Cuenca es un claro ejemplo de como ...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/campillo-de-altobuey\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"39.611134","lng":"-1.794452"}},{"title":"Camu\u00f1as","content":"Este acogedor pueblo toledano ofrece una amplia variedad de servicios y comer...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/camunas\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"39.379082","lng":"-3.481841"}},{"title":"Ca\u00f1ete de las Torres","content":"En medio de los mares de olivos de C\u00f3rdoba, encontramos este precioso pueblo...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/canete-de-las-torres\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"37.867630","lng":"-4.320160"}},{"title":"Ca\u00f1ete La Real","content":"En la comarca de Guadalteba encontramos uno de los pueblos con m\u00e1s magia de ...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/canete-la-real\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"36.952170","lng":"-5.024531"}},{"title":"Cangas de On\u00eds","content":"Cangas de On\u00eds ser\u00e1 el pueblo con m\u00e1s historia de Asturias. No solo porque...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/cangas-de-onis\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"43.352402","lng":"-5.131738"}},{"title":"Cangas del Narcea","content":"Ser una de las puertas de entrada al Principado de Asturias convierte a este ...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/cangas-del-narcea\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"43.178293","lng":" -6.549675"}},{"title":"Ca\u00f1izares","content":"En el coraz\u00f3n de la Sierra Norte de Cuenca encontramos este apacible municip...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/canizares\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"40.519173","lng":"-2.191847"}},{"title":"Cantalejo","content":"Sin duda es uno de los pueblos m\u00e1s potentes por su actividad econ\u00f3mica y su...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/cantalejo\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"41.257610","lng":"-3.928172"}},{"title":"Casarabonela","content":"A las faldas de la Sierra de las Nieves se encuentra este hermoso pueblo mala...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/casarabonela\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"36.785089","lng":"-4.842338"}},{"title":"Casas de L\u00e1zaro","content":"En un precioso enclave natural de la provincia de Albacete, encontramos este ...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/casas-de-lazaro\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"38.771135","lng":"-2.239816"}},{"title":"Castilblanco de los Arroyos","content":"En plena Sierra Morena y a tan solo 30 kil\u00f3metros de Sevilla encontramos est...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/castilblanco-de-los-arroyos\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"37.674649","lng":"-5.990180"}},{"title":"Cavia","content":"Tener una vida de pueblo pero con todos los servicios de una gran ciudad a un...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/cavia\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"42.284197","lng":"-3.844837"}},{"title":"Cedillo","content":"La magia y las ventajas de las zonas fronterizas, eso es lo que tiene este ac...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/cedillo\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"39.650491","lng":"-7.497399"}},{"title":"Celanova","content":"En la provincia gallega de Ourense encontramos este precioso pueblos llenos d...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/celanova\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"42.153732","lng":"-7.958035"}},{"title":"Cofrentes","content":"Historia y naturaleza conviven en perfecta armon\u00eda en este bonito pueblo val...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/cofrentes\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"39.229185","lng":"-1.062882"}},{"title":"Comarca de Somontano de Barbastro","content":"La Comarca de Somontano de Barbastro ofrece en su capital todo tipo de servic...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/somontano-sureste\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"42.028099","lng":" 0.131142 "}},{"title":"Comarca de Somontano de Barbastro ( zona Norte )","content":"En la provincia de Huesca se encuentra la comarca de Somontano de Barbastro. ...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/somontano-norte\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"42.042697","lng":" 0.123846 "}},{"title":"Comares","content":"Su altura y sus vistas hacen que a este municipio malague\u00f1o se le conozca co...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/comares\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"36.849011","lng":"-4.247550"}},{"title":"Cortes de la Frontera","content":"Uno de los pueblos m\u00e1s importantes y hermosos de la Serran\u00eda de Ronda. La i...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/cortes-de-la-frontera\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"36.618379","lng":" -5.340727"}},{"title":"Cortes de Pall\u00e1s","content":"Dicen los que viven en este precioso pueblo de Valencia que es un lujo al alc...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/cortes-de-pallas\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"39.241805","lng":"-0.941545"}},{"title":"Covarrubias","content":"Venteaviviraunpueblo.com presenta uno de los pueblos castellanos m\u00e1s bonitos...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/covarrubias\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"42.0592944","lng":"-3.5201803"}},{"title":"Cuevas Bajas","content":"Al norte de la provincia de M\u00e1laga encontramos este bonito pueblo a orillas ...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/cuevas-bajas\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"37.235007","lng":"-4.486507"}},{"title":"Cuevas de San Marcos","content":"En la frontera de M\u00e1laga con C\u00f3rdoba se encuentra este hermoso pueblo consi...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/cuevas-de-san-marcos\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"37.2696","lng":"-4.4138"}},{"title":"Cuevas del Becerro","content":"Al norte de la provincia de M\u00e1laga encontramos este bonito pueblo que da la ...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/cuevas-del-becerro\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"36.875011","lng":"-5.046313"}},{"title":"Curtis","content":"Curtis, en el interior de la provincia de La Coru\u00f1a, es desde hace cientos d...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/curtis\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"43.125608","lng":" -8.146269"}},{"title":"El Borge","content":"Este hermoso pueblo malague\u00f1o se encuentra enmarcado la comarca oriental de ...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/el-borge\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"36.814775","lng":"-4.235022"}},{"title":"El Burgo","content":"Reserva de la Biosfera, Reserva Natural y desde 2021, Parque Nacional. Estos ...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/el-burgo\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"36.789581","lng":"-4.948014"}},{"title":"El Pedernoso","content":"Este peque\u00f1o y acogedor pueblo de la provincia de Cuenca, esta situado en un...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/el-pedernoso\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"39.485024","lng":"-2.745479"}},{"title":"El Pinar de El Hierro","content":"Paradisiaca, \u00fanica, fascinante y cautivadora. As\u00ed de espectacular es la isl...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/el-pinar-del-hierro\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"27.730734","lng":"-18.032923"}},{"title":"El Saucejo","content":"Este precioso pueblo sevillano ubicado en la llamada Sierra Sur, esta repleto...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/el-saucejo\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"37.073239","lng":"-5.097895"}},{"title":"Entrerr\u00edos","content":"Un pueblo tranquilo, un pueblo para vivirlo. As\u00ed se define este acogedor mun...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/entrerrios\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"38.995496","lng":"-5.722010"}},{"title":"Faraj\u00e1n","content":"Todo lo que tiene de peque\u00f1o lo tiene de atractivo. Faraj\u00e1n es uno de los p...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/farajan\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"36.616348","lng":"-5.188615"}},{"title":"Feria","content":"En la provincia de Badajoz, a las faldas de Sierra Vieja encontramos este pre...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/feria\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"38.505177","lng":"-6.564446"}},{"title":"Fontiveros","content":"En el noroeste de la provincia de \u00c1vila encontramos este agradable pueblo ca...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/fontiveros\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"40.930114","lng":"-4.965551"}},{"title":"Fregenal de la Sierra","content":"En la dehesa extreme\u00f1a encontramos este peque\u00f1o gran pueblo pacense. Peque?...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/fregenal-de-la-sierra\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"38.169843","lng":" -6.654614"}},{"title":"Friol","content":"En el norte de Espa\u00f1a, en la provincia de Lugo encontramos este precioso pue...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/friol\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"43.031420","lng":"-7.795487"}},{"title":"Fuente Obejuna","content":"Se trata de un pueblo que muchos no saben si existe o solo es uno de los nomb...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/fuente-obejuna\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"38.267579","lng":"-5.420034"}},{"title":"Fuenterrebollo","content":"En las puertas del Parque Nacional de Las Hoces del Durat\u00f3n, encontramos est...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/fuenterrebollo\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"41.297028","lng":"-3.932652"}},{"title":"Gauc\u00edn","content":"Entre los\u00a0 alcornoques, encinas y pinos aparece uno de los pueblos m\u00e1s cono...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/gaucin\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"36.519662","lng":"-5.319929"}},{"title":"Genalguacil","content":"Sin duda, este pueblo de la Serran\u00eda de Ronda es un pueblo con Arte, con muc...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/genalguacil\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"36.545966","lng":" -5.235430"}},{"title":"Graus","content":"Naturaleza, historia y una amplia variedad de servicios e infraestructuras se...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/graus\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"42.189992","lng":"-0.335204"}},{"title":"Higueruela","content":"Un pueblo acogedor y peque\u00f1o pero con los mismos servicio de un pueblo grand...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/higueruela\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"38.963818","lng":"-1.444044"}},{"title":"Hontalbilla","content":"En el coraz\u00f3n de Segovia encontramos este milenario pueblo. Un entorno tranq...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/hontalbilla\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"41.345495","lng":"-4.121928"}},{"title":"Hontanas","content":"En el camino de Santiago, en la provincia de Burgos, encontramos este peque\u00f1...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/hontanas\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"42.312570","lng":"-4.045317"}},{"title":"Hu\u00e9rcanos","content":"En el coraz\u00f3n de La Rioja encontramos este acogedor pueblo, de gente amable ...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/huercanos\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"42.429294","lng":"-2.695704"}},{"title":"Iglesias","content":"Este encantador y acogedor pueblo de la provincia de Burgos es un remanso de ...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/iglesias\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"42.297437","lng":"-3.989267"}},{"title":"Ituero y Lama","content":"En la provincia de Segovia, a menos de una hora de Madrid, encontramos este a...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/ituero-y-lama\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"40.800218","lng":"-4.378335"}},{"title":"Jimena de la Frontera","content":"Entre la Sierra de Ronda y el mar Mediterr\u00e1neo encontramos este precioso pue...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/jimena-de-la-frontera\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"36.431941","lng":"-5.450261"}},{"title":"Jimera de L\u00edbar","content":"En un entorno natural privilegiado se encuentra este pueblo de la Serran\u00eda d...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/jimera-de-libar\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"36.651727","lng":"-5.274188"}},{"title":"Jubrique","content":"Uno de los pueblos m\u00e1s cuidados y m\u00e1s bonitos de la Serran\u00eda de Ronda. Est...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/jubrique\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"36.564994","lng":"-5.215828"}},{"title":"L\u00b4\u00c9nova","content":"Envuelto en ese manto de aromas fascinante que dejan los naranjos y las huert...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/lenova\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"39.045945","lng":"-0.481812"}},{"title":"La Adrada","content":"En el m\u00e1gico Valle del Tietar, custodiado por la Sierra de Gredos, encontram...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/la-adrada\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"40.298975","lng":"-4.639776"}},{"title":"La Bobadilla","content":"En la provincia de Ja\u00e9n encontramos este acogedor pueblo en medio de un mar ...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/la-bobadilla\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"37.660560","lng":"-4.105382"}},{"title":"La Muela","content":"Este pr\u00f3spero municipio zaragozano ofrece una amplio abanico de servicios, c...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/la-muela\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"41.581329","lng":"-1.120683"}},{"title":"Lerma","content":"Desde este hermoso pueblo castellano se dirigi\u00f3 un imperio! El Duque de Lerm...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/lerma\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"42.025281","lng":"-3.758606 "}},{"title":"Llanes","content":"Probablemente esta sea uno de los pueblos m\u00e1s deseados en Espa\u00f1a. Deseado p...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/llanes\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"43.421353","lng":"-4.754636"}},{"title":"Loja","content":"En la provincia de Granada encontramos uno de los pueblos mejor comunicados y...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/loja\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"37.168151","lng":"-4.149438"}},{"title":"Los Corrales","content":"Este pueblo sevillano se define ante todo como \u00abacogedor\u00bb. Un lugar lleno d...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/los-corrales\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"37.097751","lng":"-4.983709"}},{"title":"Los Molinos","content":"Situado en el coraz\u00f3n de la Sierra de Guadarrama, es uno de los pueblos pref...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/los-molinos\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"40.710935","lng":"-4.075583"}},{"title":"Madrigal de las Altas Torres","content":"Tradici\u00f3n, historia y adaptaci\u00f3n a los nuevos tiempos conjugan a la perfecc...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/madrigal-de-las-altas-torres\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"41.090509","lng":"-4.998192"}},{"title":"Madrigueras","content":"Un pueblo con trabajo e industria. Este municipio de Albacete ofrece los que ...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/madrigueras\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"39.237892","lng":"-1.798793"}},{"title":"Mart\u00edn de la Jara","content":"Este acogedor pueblo sevillano dispone de una nutrida oferta de servicios y c...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/martin-de-la-jara\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"37.106394","lng":"-4.962915"}},{"title":"Mayorga","content":"Un entorno agradable, tranquilo, lleno de oportunidades laborales y con una g...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/mayorga\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"42.1666301","lng":"-5.2626995"}},{"title":"Millares","content":"Ba\u00f1ado por las aguas del rio J\u00facar y rodeado de naturaleza encontramos este...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/millares\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"39.238535","lng":"-0.773645"}},{"title":"Moclinejo","content":"En la comarca malague\u00f1a de la Axarqu\u00eda encontramos este pintoresco pueblo q...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/moclinejo\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"36.771497","lng":"-4.255156"}},{"title":"Molinaseca","content":"En la comarca leonesa del Bierzo podemos encontrar este precioso pueblo en me...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/molinaseca\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"42.537896","lng":"-6.520601"}},{"title":"Montecorto","content":"Los amantes de los entornos naturales serranos encontrar\u00e1n en este pueblo, u...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/montecorto\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"36.815912","lng":"-5.296687"}},{"title":"Montemayor de Pililla","content":"Al sur de la provincia de Valladolid encontramos este tranquilo y hermoso pue...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/montemayor-de-pililla\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"41.508711","lng":"-4.458270"}},{"title":"Navarrevisca","content":"Este peque\u00f1o pueblo abulense de monta\u00f1a es perfecto para aquellos que est\u00e1...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/navarrevisca\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"40.364483","lng":"-4.894524"}},{"title":"O Pereiro de Aguiar","content":"En la provincia de Ourense encontramos este hermoso pueblo gallego enclavado ...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/o-pereiro-de-aguiar\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"42.346658","lng":"-7.799794"}},{"title":"Oj\u00e9n","content":"Este pueblo hace posible lo imposible: poder desarrollar una vida en un encla...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/ojen\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"36.564045","lng":"-4.857352"}},{"title":"Olmedilla de Alarc\u00f3n","content":"Este bonito pueblo de la provincia de Cuenca se encuentra ubicado en un entor...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/olmedilla-de-alarcon\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"39.614255","lng":"-2.104168"}},{"title":"Or\u00e9s","content":"La belleza de este peque\u00f1o pueblo medieval junto a los Pirineos, rodeado de ...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/ores\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"42.279234","lng":"-1.001120"}},{"title":"Panes Pe\u00f1amellera Baja","content":"A la entrada del impresionante Parque Nacional de Los Picos de Europa encontr...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/panes-penamellera-baja\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"43.324841","lng":"-4.584798"}},{"title":"Paredes de Nava","content":"Se trata de uno de los pueblos m\u00e1s importantes hist\u00f3ricamente de la provinc...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/paredes-de-nava\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"42.152611","lng":"-4.693969"}},{"title":"Piedralaves","content":"Uno de los pueblos m\u00e1s bonitos que se pueden encontrar en el Valle del Ti\u00e9t...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/piedralaves\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"40.317953","lng":"-4.699530"}},{"title":"Pozo Alc\u00f3n","content":"En uno de los entornos naturales m\u00e1s valorado de Europa, el Parque Natural S...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/pozo-alcon\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"37.704040","lng":"-2.934713"}},{"title":"Pozohondo","content":"Un pueblo tranquilo, perfecto para familias y bien conectado. As\u00ed nos asegur...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/pozohondo\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"38.720947","lng":"-1.909719"}},{"title":"Quincoces de Yuso","content":"Al norte de Burgos en el hermoso Valle de Losa, encontramos este pueblo de s?...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/quincoces-de-yuso\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"42.989385","lng":"-3.240024"}},{"title":"Rab\u00e9 de las Calzadas","content":"A pocos kil\u00f3metros de la ciudad Burgos podemos llevar una aut\u00e9ntica vida de...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/rabe-de-las-calzadas\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"42.340326","lng":"-3.834407"}},{"title":"Ribadesella","content":"No es casualidad que muchos arist\u00f3cratas eligieran esta villa asturiana para...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/ribadesella\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"43.4617129","lng":"-5.0587479"}},{"title":"Rois","content":"A suroeste de la provincia de La Coru\u00f1a se encuentra este tranquilo municipi...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/rois\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"42.762889","lng":"-8.710479"}},{"title":"Sajazarra","content":"En La Rioja alta encontramos uno de los pueblos m\u00e1s bonitos, con m\u00e1s histor...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/sajazarra\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"42.589022","lng":"-2.960843"}},{"title":"Salas","content":"En el interior de Asturias encontramos este acogedor pueblo entre bosques de ...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/salas\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"43.409497","lng":" -6.261509"}},{"title":"Salmeroncillos","content":"Para aquellos que busquen la tranquilidad absoluta sin alejarse demasiado de ...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/salmeroncillos\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"40.507619","lng":"-2.520026"}},{"title":"San Adri\u00e1n","content":"En la Comunidad Foral de Navarra encontramos este pueblo con una gran potenci...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/san-adrian\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"42.335455","lng":"-1.936545"}},{"title":"San Esteban de la Sierra","content":"Uno de los pueblos m\u00e1s bonitos de Castilla y Le\u00f3n. Ubicado en la Sierra de ...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/san-esteban-de-la-sierra\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"40.506761","lng":"-5.906275"}},{"title":"San Pedro","content":"\u00abUn pueblo, pueblo, con todo lo necesario para llevar una vida tranquila\u00bb, ...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/san-pedro\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"38.825439","lng":"-2.187938"}},{"title":"San Vicente de Alc\u00e1ntara","content":"Buscando el embrujo y fascinaci\u00f3n que tienen las zonas fronterizas, encontra...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/san-vicente-de-alcantara\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"39.359185","lng":"-7.137332"}},{"title":"Sep\u00falveda","content":"Uno de los pueblos castellanos m\u00e1s hermosos, con m\u00e1s historia, con un ampli...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/sepulveda\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"41.298320","lng":"-3.748989"}},{"title":"Simancas","content":"Uno de los pueblos medievales m\u00e1s bonitos y mejor conservados de Castilla, p...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/simancas\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"41.593893","lng":"-4.828654"}},{"title":"Sotillo de la Adrada","content":"Pocos municipios pueden presumir en Espa\u00f1a tanto de tener un entorno natural...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/sotillo-de-la-adrada\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"40.293846","lng":"-4.583938"}},{"title":"Teba","content":"Como acunado por el tiempo, as\u00ed parece que descansa este milenario pueblo en...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/teba\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"36.983429","lng":"-4.919170"}},{"title":"Teixeiro","content":"En el interior de la provincia de La Coru\u00f1a se encuentra este peque\u00f1o puebl...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/teixeiro\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"43.139419","lng":" -8.036540"}},{"title":"Tolox","content":"En el coraz\u00f3n de la Sierra de las nieves encontramos este bonito pueblo mala...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/tolox\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"36.687048","lng":"-4.904778"}},{"title":"Toreno","content":"En la comarca leonesa de El Bierzo encontramos este pueblo ancestral enclavad...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/toreno\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"42.696853","lng":"-6.515129"}},{"title":"Torrecillas de la Tiesa","content":"Un peque\u00f1o y acogedor pueblo que tiene de todo. As\u00ed podr\u00edamos definir a es...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/torrecillas-de-la-tiesa\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"39.567404","lng":"-5.742596"}},{"title":"Ubrique","content":"En la Sierra de C\u00e1diz encontramos este espectacular pueblo blanco en medio d...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/ubrique\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"36.673881","lng":"-5.446626"}},{"title":"Valdeprados","content":"En la provincia castellana de Segovia encontramos este pueblo de cuento, tan ...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/valdeprados\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"40.818516","lng":"-4.257402"}},{"title":"Valle de Abdalaj\u00eds","content":"Entre las comarcas malague\u00f1as del Valle de Guadalhorce y de Antequera, encon...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/valle-de-abdalajis\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"36.931827","lng":"-4.682723"}},{"title":"Valoria la Buena","content":"Los habitantes de este peque\u00f1o pueblo vallisoletano, aseguran que su nombre?...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/valoria-la-buena\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"41.800521 ","lng":"-4.529110 "}},{"title":"Valverdejo","content":"\u00a1Con los brazos abiertos! As\u00ed est\u00e1n deseando recibir a nuevos vecinos en e...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/valverdejo\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"39.616099","lng":"-2.023161"}},{"title":"Vara de Rey","content":"Un pueblo peque\u00f1o y acogedor en el que no se echar\u00e1 nada en falta. As\u00ed es ...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/vara-de-rey\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"39.424881","lng":"-2.292964"}},{"title":"Vedra","content":"Al sur de la provincia de La Coru\u00f1a se encuentra este precioso municipio ba?...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/vedra\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"42.775726","lng":" -8.480720"}},{"title":"Vera del Moncayo","content":"En la provincia de Zaragoza encontramos este precioso pueblo a los pies del m...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/vera-del-moncayo\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"41.824811","lng":"-1.687750"}},{"title":"Vilanova de Arousa","content":"El Atl\u00e1ntico marca la vida de este pueblo pontevedr\u00e9s que vive de cara al o...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/vilanova-de-arousa\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"42.563404","lng":"-8.827257 "}},{"title":"Villablino","content":"Se trata de uno de uno de los municipios m\u00e1s destacados del norte de la prov...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/villablino\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"42.939932","lng":"-6.322193"}},{"title":"Villadiego","content":"En este pueblo de Burgos aseguran que tiene de todo, que no se echa de menos ...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/villadiego\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"42.515502","lng":"-4.011075"}},{"title":"Villal\u00f3n Campos","content":"En plena Tierra de Campos se encuentra este municipio vallisoletano lleno de ...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/villalon-campos\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"42.0984494","lng":"-5.0347132"}},{"title":"Villanueva de Tapia","content":"Entre campos de cereal y mares de olivos encontramos este hermoso pueblo anda...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/villanueva-de-tapia\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"37.182201","lng":"-4.334967"}},{"title":"Villanueva del Duque","content":"Uno de los enclaves con m\u00e1s solera de la provincia de C\u00f3rdoba. Este municip...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/villanueva-del-duque\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"38.394229","lng":"-4.998225"}},{"title":"Villarcayo de Merindad de Castilla la Vieja","content":"Al norte de la provincia de Burgos encontramos este municipio en un valle rep...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/villarcayo-de-merindad-de-castilla-la-vieja\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"42.938744","lng":" -3.571298"}},{"title":"Villarrodrigo","content":"En la provincia andaluza de Ja\u00e9n encontramos este encantador pueblo en un en...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/villarrodrigo\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"38.487345","lng":"-2.636110"}},{"title":"Yunquera","content":"En la Sierra de las Nieves de M\u00e1laga encontramos este hermoso municipio llen...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/yunquera\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"36.733989","lng":"-4.918423"}},{"title":"Zarra","content":"En Valencia encontramos este encantador pueblo que enamora a todo el que lo c...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/zarra\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"39.091614","lng":"-1.074911"}},{"title":"Zarzuela del Monte","content":"Este peque\u00f1o pueblo segoviano con mucho encanto, est\u00e1 a poco m\u00e1s de una ho...","paramlink":"https:\/\/venteaviviraunpueblo.com\/pueblo\/zarzuela-del-monte\/","icon":"https:\/\/venteaviviraunpueblo.com\/app\/themes\/silence\/images\/map-pin.png","pin":{"lat":"40.809249","lng":"-4.336507"}}]};


In [39]:
# Se crea una tabla a partir de los datos indicados en el campo 'markers'
df_vente= pd.json_normalize(VenteGmap['markers'])
df_vente

,title,content,paramlink,icon,pin.lat,pin.lng
0,A Estrada,La fortaleza de este pueblo de la provincia de...,https:\/\/venteaviviraunpueblo.com\/pueblo\/a-...,https:\/\/venteaviviraunpueblo.com\/app\/theme...,42.689068,-8.482822
1,A Mariña Lucense,Pocas tierras combinan el mar y la montaña de ...,https:\/\/venteaviviraunpueblo.com\/pueblo\/a-...,https:\/\/venteaviviraunpueblo.com\/app\/theme...,43.535297,-7.045202
2,Aceituna,En la provincia de Cáceres encontramos este tr...,https:\/\/venteaviviraunpueblo.com\/pueblo\/ac...,https:\/\/venteaviviraunpueblo.com\/app\/theme...,40.149099,-6.330698
3,Aguilar de Campoo,A los pies de la montaña palentina encontramos...,https:\/\/venteaviviraunpueblo.com\/pueblo\/ag...,https:\/\/venteaviviraunpueblo.com\/app\/theme...,42.791388,-4.260055
4,Albares,En la provincia de Guadalajara encontramos est...,https:\/\/venteaviviraunpueblo.com\/pueblo\/al...,https:\/\/venteaviviraunpueblo.com\/app\/theme...,40.308621,-3.011926
...,...,...,...,...,...,...
146,Villarcayo de Merindad de Castilla la Vieja,Al norte de la provincia de Burgos encontramos...,https:\/\/venteaviviraunpueblo.com\/pueblo\/vi...,https:\/\/venteaviviraunpueblo.com\/app\/theme...,42.938744,-3.571298
147,Villarrodrigo,En la provincia andaluza de Jaén encontramos e...,https:\/\/venteaviviraunpueblo.com\/pueblo\/vi...,https:\/\/venteaviviraunpueblo.com\/app\/theme...,38.487345,-2.636110
148,Yunquera,En la Sierra de las Nieves de Málaga encontram...,https:\/\/venteaviviraunpueblo.com\/pueblo\/yu...,https:\/\/venteaviviraunpueblo.com\/app\/theme...,36.733989,-4.918423
149,Zarra,En Valencia encontramos este encantador pueblo...,https:\/\/venteaviviraunpueblo.com\/pueblo\/za...,https:\/\/venteaviviraunpueblo.com\/app\/theme...,39.091614,-1.074911


In [40]:
# Se eliminan las columnas no necesarias
df_vente.drop(columns=['content','icon',], inplace=True)

# Se renombran las columnas.
df_vente.columns=['Municipio', 'Link_web','lat','lng']
df_vente

,Municipio,Link_web,lat,lng
0,A Estrada,https:\/\/venteaviviraunpueblo.com\/pueblo\/a-...,42.689068,-8.482822
1,A Mariña Lucense,https:\/\/venteaviviraunpueblo.com\/pueblo\/a-...,43.535297,-7.045202
2,Aceituna,https:\/\/venteaviviraunpueblo.com\/pueblo\/ac...,40.149099,-6.330698
3,Aguilar de Campoo,https:\/\/venteaviviraunpueblo.com\/pueblo\/ag...,42.791388,-4.260055
4,Albares,https:\/\/venteaviviraunpueblo.com\/pueblo\/al...,40.308621,-3.011926
...,...,...,...,...
146,Villarcayo de Merindad de Castilla la Vieja,https:\/\/venteaviviraunpueblo.com\/pueblo\/vi...,42.938744,-3.571298
147,Villarrodrigo,https:\/\/venteaviviraunpueblo.com\/pueblo\/vi...,38.487345,-2.636110
148,Yunquera,https:\/\/venteaviviraunpueblo.com\/pueblo\/yu...,36.733989,-4.918423
149,Zarra,https:\/\/venteaviviraunpueblo.com\/pueblo\/za...,39.091614,-1.074911


In [41]:
# Se modifica la web para que sea más legible
df_vente['Link_web']=df_vente['Link_web'].apply(lambda x: x.replace('\\/\\/', '//'))
df_vente['Link_web']=df_vente['Link_web'].apply(lambda x: x.replace('\\/', '/'))

In [42]:
df_vente

,Municipio,Link_web,lat,lng
0,A Estrada,https://venteaviviraunpueblo.com/pueblo/a-estr...,42.689068,-8.482822
1,A Mariña Lucense,https://venteaviviraunpueblo.com/pueblo/a-mari...,43.535297,-7.045202
2,Aceituna,https://venteaviviraunpueblo.com/pueblo/aceituna/,40.149099,-6.330698
3,Aguilar de Campoo,https://venteaviviraunpueblo.com/pueblo/aguila...,42.791388,-4.260055
4,Albares,https://venteaviviraunpueblo.com/pueblo/albares/,40.308621,-3.011926
...,...,...,...,...
146,Villarcayo de Merindad de Castilla la Vieja,https://venteaviviraunpueblo.com/pueblo/villar...,42.938744,-3.571298
147,Villarrodrigo,https://venteaviviraunpueblo.com/pueblo/villar...,38.487345,-2.636110
148,Yunquera,https://venteaviviraunpueblo.com/pueblo/yunquera/,36.733989,-4.918423
149,Zarra,https://venteaviviraunpueblo.com/pueblo/zarra/,39.091614,-1.074911


### Se extrae de la web www.volveralpueblo.org el mapa de los pueblos promocionados

In [43]:
markers = [{"type":"Business","id":14,"reference":"NCM2022001","community":7,"ccaa":"Castilla La Mancha","province":"Guadalajara","town":"Peralejos de las Truchas","postcode":"19313","form":1,"jobform":'null',"sector":2,"lat":"40.5929386","lng":"-1.9093132"},{"type":"Business","id":16,"reference":"NCL2022003","community":8,"ccaa":"Castilla y Leon","province":"Burgos","town":"Villadiego","postcode":"09120","form":1,"jobform":'null',"sector":2,"lat":"42.5146862","lng":"-4.0101705"},{"type":"Business","id":17,"reference":"NCL2022004","community":8,"ccaa":"Castilla y Leon","province":"Burgos","town":"Villahoz","postcode":"09343","form":2,"jobform":'null',"sector":2,"lat":"42.0771757","lng":"-3.9126371"},{"type":"Business","id":19,"reference":"NCL2022005","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Carri\u00f3n de los condes","postcode":"34120","form":1,"jobform":'null',"sector":2,"lat":"42.3307302","lng":"-4.6140481"},{"type":"Business","id":20,"reference":"NCL2022006","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Fr\u00f3mista","postcode":"34440","form":1,"jobform":'null',"sector":2,"lat":"42.2676329","lng":"-4.4053497"},{"type":"Business","id":22,"reference":"NCL2022008","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"Villavante","postcode":"24393","form":1,"jobform":'null',"sector":2,"lat":"42.4626305","lng":"-5.8367024"},{"type":"Business","id":24,"reference":"NCL2022010","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"San Cebri\u00e1n de campos","postcode":"34429","form":1,"jobform":'null',"sector":2,"lat":"42.2018325","lng":"-4.5308585"},{"type":"Business","id":25,"reference":"NCL2022011","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Carri\u00f3n de los condes","postcode":"34120","form":1,"jobform":'null',"sector":6,"lat":"42.3307302","lng":"-4.6140481"},{"type":"Business","id":27,"reference":"NCL2022013","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Carri\u00f3n de los condes","postcode":"34120","form":2,"jobform":'null',"sector":2,"lat":"42.3307302","lng":"-4.6140481"},{"type":"Business","id":34,"reference":"NCL2022016","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Castrillo de Villavega","postcode":"34478","form":7,"jobform":'null',"sector":'null',"lat":"42.4543635","lng":"-4.4819074"},{"type":"Business","id":36,"reference":"NVC2022002","community":17,"ccaa":"Comunidad Valenciana","province":"Castellon","town":"Torreblanca","postcode":"12596","form":1,"jobform":'null',"sector":2,"lat":"40.2201804","lng":"0.1951319"},{"type":"Business","id":38,"reference":"NAS2022002","community":3,"ccaa":"Asturias","province":"Asturias","town":"Infiesto","postcode":"33530","form":6,"jobform":'null',"sector":2,"lat":"43.3471387","lng":"-5.3630233"},{"type":"Business","id":43,"reference":"NCL2022021","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Santoyo","postcode":"34490","form":1,"jobform":'null',"sector":'null',"lat":"42.2148747","lng":"-4.3425481"},{"type":"Business","id":45,"reference":"NCB2022002","community":6,"ccaa":"Cantabria","province":"Cantabria","town":"San Pedro del Romeral","postcode":"39686","form":2,"jobform":'null',"sector":2,"lat":"43.1203153","lng":"-3.8349976"},{"type":"Business","id":47,"reference":"NCL2022024","community":8,"ccaa":"Castilla y Leon","province":"Avila","town":"El Barco de \u00c1vila","postcode":"05600","form":2,"jobform":'null',"sector":2,"lat":"40.3569063","lng":"-5.5236746"},{"type":"Business","id":48,"reference":"NCL2022025","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"Secarejo","postcode":"24273","form":2,"jobform":'null',"sector":2,"lat":"42.6503426","lng":"-5.8146181"},{"type":"Business","id":49,"reference":"NCL2022026","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Calzada de los Molinos","postcode":"34129","form":6,"jobform":'null',"sector":'null',"lat":"42.327351","lng":"-4.6521386"},{"type":"Business","id":50,"reference":"NCL2022027","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Carri\u00f3n de los Condes","postcode":"34120","form":6,"jobform":'null',"sector":'null',"lat":"42.3307302","lng":"-4.6140481"},{"type":"Business","id":51,"reference":"NCL2022028","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Carri\u00f3n de los Condes","postcode":"34120","form":1,"jobform":'null',"sector":'null',"lat":"42.3307302","lng":"-4.6140481"},{"type":"Business","id":54,"reference":"NCL2022031","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Cervatos de la Cueza","postcode":"34309","form":6,"jobform":'null',"sector":'null',"lat":"42.2905776","lng":"-4.7698058"},{"type":"Business","id":55,"reference":"NCL2022032","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Cervatos de la Cueza","postcode":"34309","form":7,"jobform":'null',"sector":'null',"lat":"42.2905776","lng":"-4.7698058"},{"type":"Business","id":56,"reference":"NCL2022033","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"Piedrasecha","postcode":"24123","form":1,"jobform":'null',"sector":2,"lat":"42.8242674","lng":"-5.7785426"},{"type":"Business","id":58,"reference":"NAN2022001","community":1,"ccaa":"Andalucia","province":"Huelva","town":"Arroyomolinos de Le\u00f3n","postcode":"21280","form":1,"jobform":'null',"sector":2,"lat":"38.0240295","lng":"-6.4216722"},{"type":"Business","id":59,"reference":"NAN2022002","community":1,"ccaa":"Andalucia","province":"Huelva","town":"Arroyomolinos de Le\u00f3n","postcode":"21280","form":6,"jobform":'null',"sector":2,"lat":"38.0240295","lng":"-6.4216722"},{"type":"Business","id":60,"reference":"NCL2022034","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Carri\u00f3n de los Condes","postcode":"34120","form":2,"jobform":'null',"sector":'null',"lat":"42.3307302","lng":"-4.6140481"},{"type":"Business","id":61,"reference":"NCL2022035","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Calzada de los Molinos","postcode":"34129","form":2,"jobform":'null',"sector":'null',"lat":"42.327351","lng":"-4.6521386"},{"type":"Business","id":62,"reference":"NCL2022036","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Carri\u00f3n de los Condes","postcode":"34120","form":2,"jobform":'null',"sector":'null',"lat":"42.3307302","lng":"-4.6140481"},{"type":"Business","id":63,"reference":"NCL2022037","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"Sope\u00f1a de Carneros","postcode":"24719","form":1,"jobform":'null',"sector":9,"lat":"42.4882225","lng":"-6.0450888"},{"type":"Business","id":64,"reference":"NAN2022003","community":1,"ccaa":"Andalucia","province":"Granada","town":"La Taha de Pitres","postcode":"18414","form":1,"jobform":'null',"sector":2,"lat":"36.9357387","lng":"-3.32608"},{"type":"Business","id":65,"reference":"NAR2022002","community":2,"ccaa":"Aragon","province":"Zaragoza","town":"Uncastillo","postcode":"50678","form":1,"jobform":'null',"sector":2,"lat":"42.3599895","lng":"-1.1317041"},{"type":"Business","id":67,"reference":"NVC2022005","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"Castell\u00f3 de Rugat","postcode":"46841","form":6,"jobform":'null',"sector":10,"lat":"38.8735003","lng":"-0.3830754"},{"type":"Business","id":69,"reference":"NCL2022038","community":8,"ccaa":"Castilla y Leon","province":"Valladolid","town":"Villanueva de los Caballeros","postcode":"47850","form":6,"jobform":'null',"sector":'null',"lat":"41.7590608","lng":"-5.2488573"},{"type":"Business","id":70,"reference":"NCL2022039","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"Toral de los Vados","postcode":"24560","form":1,"jobform":'null',"sector":'null',"lat":"42.5439605","lng":"-6.7788461"},{"type":"Business","id":71,"reference":"NCL2022040","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"Sahag\u00fan","postcode":"24320","form":7,"jobform":'null',"sector":'null',"lat":"42.3709599","lng":"-5.0299493"},{"type":"Business","id":72,"reference":"NCB2022003","community":6,"ccaa":"Cantabria","province":"Cantabria","town":"Selaya","postcode":"39696","form":6,"jobform":'null',"sector":13,"lat":"43.1997485","lng":"-3.7739869"},{"type":"Business","id":73,"reference":"NCL2022041","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Lantadilla","postcode":"34468","form":1,"jobform":'null',"sector":4,"lat":"42.3406834","lng":"-4.2775694"},{"type":"Business","id":75,"reference":"NGA2022001","community":11,"ccaa":"Galicia","province":"Ourense","town":"Sandi\u00e1s","postcode":"32692","form":6,"jobform":'null',"sector":2,"lat":"42.1380059","lng":"-7.7362384"},{"type":"Business","id":76,"reference":"NAS2022003","community":3,"ccaa":"Asturias","province":"Asturias","town":"Vegarrionda, L\u00b4Arteosa","postcode":"33536","form":6,"jobform":'null',"sector":14,"lat":"43.3084084","lng":"-5.410683"},{"type":"Business","id":78,"reference":"NEX2022001","community":10,"ccaa":"Extremadura","province":"Badajoz","town":"El Torviscal","postcode":"06717","form":1,"jobform":'null',"sector":'null',"lat":"39.0729758","lng":"-5.7569735"},{"type":"Business","id":79,"reference":"NEX2022002","community":10,"ccaa":"Extremadura","province":"Caceres","town":"Serrej\u00f3n","postcode":"10528","form":2,"jobform":'null',"sector":'null',"lat":"39.8178348","lng":"-5.8017668"},{"type":"Business","id":80,"reference":"NEX2022003","community":10,"ccaa":"Extremadura","province":"Caceres","town":"Ca\u00f1amero","postcode":"10136","form":6,"jobform":'null',"sector":2,"lat":"39.3788085","lng":"-5.3874113"},{"type":"Business","id":81,"reference":"NEX2022004","community":10,"ccaa":"Extremadura","province":"Badajoz","town":"Barba\u00f1o","postcode":"06499","form":1,"jobform":'null',"sector":2,"lat":"38.8744591","lng":"-6.5870143"},{"type":"Business","id":82,"reference":"NAS2022004","community":3,"ccaa":"Asturias","province":"Asturias","town":"Pesoz","postcode":"33735","form":8,"jobform":'null',"sector":2,"lat":"43.257105","lng":"-6.8763166"},{"type":"Business","id":85,"reference":"NGA2022002","community":11,"ccaa":"Galicia","province":"Ourense","town":"Xinzo de Limia","postcode":"32630","form":7,"jobform":'null',"sector":'null',"lat":"42.0631102","lng":"-7.7259471"},{"type":"Business","id":86,"reference":"NCL2022043","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Carri\u00f3n de los Condes","postcode":"34120","form":2,"jobform":'null',"sector":'null',"lat":"42.3307302","lng":"-4.6140481"},{"type":"Business","id":88,"reference":"NCL2022044","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Carri\u00f3n de los Condes","postcode":"34120","form":2,"jobform":'null',"sector":'null',"lat":"42.3307302","lng":"-4.6140481"},{"type":"Business","id":89,"reference":"NAS2022005","community":3,"ccaa":"Asturias","province":"Asturias","town":"Grandas de Salime","postcode":"33730","form":8,"jobform":'null',"sector":2,"lat":"43.2091032","lng":"-6.8887202"},{"type":"Business","id":90,"reference":"NAS2022006","community":3,"ccaa":"Asturias","province":"Asturias","town":"Fondodevilla","postcode":"33810","form":2,"jobform":'null',"sector":2,"lat":"42.9378045","lng":"-6.7544177"},{"type":"Business","id":91,"reference":"NAS2022007","community":3,"ccaa":"Asturias","province":"Asturias","town":"Ventosa","postcode":"33777","form":8,"jobform":'null',"sector":2,"lat":"43.2822307","lng":"-6.9531225"},{"type":"Business","id":93,"reference":"NEX2022005","community":10,"ccaa":"Extremadura","province":"Caceres","town":"Castejada","postcode":"10520","form":7,"jobform":'null',"sector":10,"lat":"39.9046082","lng":"-5.6668306"},{"type":"Business","id":97,"reference":"NEX2022006","community":10,"ccaa":"Extremadura","province":"Caceres","town":"Casas de Miravete","postcode":"10360","form":6,"jobform":'null',"sector":2,"lat":"39.7270186","lng":"-5.7438055"},{"type":"Business","id":98,"reference":"NCL2022045","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"CACABELOS","postcode":"24540","form":2,"jobform":'null',"sector":2,"lat":"42.598666","lng":"-6.7259138"},{"type":"Business","id":100,"reference":"NEX2022007","community":10,"ccaa":"Extremadura","province":"Caceres","town":"Herv\u00e1s","postcode":"10700","form":1,"jobform":'null',"sector":10,"lat":"40.273599611494205","lng":"-5.860954532777526"},{"type":"Business","id":102,"reference":"NGA2022004","community":11,"ccaa":"Galicia","province":"Ourense","town":"Loureses","postcode":"32634","form":1,"jobform":'null',"sector":2,"lat":"41.99262104096378","lng":"-7.775464883646797"},{"type":"Business","id":103,"reference":"NGA2022005","community":11,"ccaa":"Galicia","province":"Ourense","town":"XINZO DE LIMIA","postcode":"32630","form":7,"jobform":'null',"sector":9,"lat":"42.07458649728702","lng":"-7.725732377571947"},{"type":"Business","id":104,"reference":"NGA2022006","community":11,"ccaa":"Galicia","province":"Ourense","town":"San Pedro de Laro\u00e1","postcode":"32632","form":1,"jobform":'null',"sector":4,"lat":"42,036701","lng":"7,683513"},{"type":"Business","id":105,"reference":"NGA2022007","community":11,"ccaa":"Galicia","province":"Ourense","town":"XINZO DE LIMIA","postcode":"32630","form":6,"jobform":'null',"sector":2,"lat":"42.072563676910164","lng":"-7.726440483918016"},{"type":"Business","id":106,"reference":"NAS2022008","community":3,"ccaa":"Asturias","province":"Asturias","town":"BOAL","postcode":"33720","form":2,"jobform":'null',"sector":2,"lat":"43.42514749692097","lng":"-6.81800024362082"},{"type":"Business","id":107,"reference":"NCM2022003","community":7,"ccaa":"Castilla La Mancha","province":"Ciudad Real","town":"Navalpino","postcode":"13193","form":1,"jobform":'null',"sector":2,"lat":"39.22825028003555","lng":"-4.59039206327472"},{"type":"Business","id":109,"reference":"NCL2022047","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"FABERO","postcode":"24420","form":7,"jobform":'null',"sector":2,"lat":"42.77975530588271","lng":"-6.625182977270176"},{"type":"Business","id":110,"reference":"NCL2022048","community":8,"ccaa":"Castilla y Leon","province":"Avila","town":"Robledillo","postcode":"05130","form":2,"jobform":'null',"sector":2,"lat":"40.54065593815947","lng":"-4.929407153830194"},{"type":"Business","id":115,"reference":"NGA2022008","community":11,"ccaa":"Galicia","province":"Lugo","town":"Campa da Bra\u00f1a","postcode":"27664","form":1,"jobform":'null',"sector":2,"lat":"42.826996981991776,","lng":"-6.933049836779477"},{"type":"Business","id":116,"reference":"NVC2022006","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"RAFELCOFER","postcode":"46716","form":1,"jobform":'null',"sector":2,"lat":"38.93705065557202","lng":"-0.16798909623695926"},{"type":"Business","id":117,"reference":"NGA2022009","community":11,"ccaa":"Galicia","province":"A Coru\u00f1a","town":"Aldea Bustelo- Doso","postcode":"15569","form":1,"jobform":'null',"sector":2,"lat":"43.51191258086928","lng":"-8.094962227863618"},{"type":"Business","id":118,"reference":"NCL2022047","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"LILLO DEL BIERZO","postcode":"24428","form":6,"jobform":'null',"sector":2,"lat":"42.86723969277626","lng":"-6.601326144899172"},{"type":"Business","id":120,"reference":"NCL2022048","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"Santa Marina del Rey","postcode":"24393","form":2,"jobform":'null',"sector":2,"lat":"42,512000","lng":"-5,858050"},{"type":"Business","id":121,"reference":"NVC2022007","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"Navarr\u00e9s","postcode":"46823","form":1,"jobform":'null',"sector":2,"lat":"39.10615778378527","lng":"-0.6952708648606356"},{"type":"Business","id":122,"reference":"NCT2022003","community":9,"ccaa":"Catalu\u00f1a","province":"Girona","town":"Sant Juli\u00e1 de Ramis","postcode":"17481","form":1,"jobform":'null',"sector":2,"lat":"42.03288028752827","lng":"- 2.8381287263650847"},{"type":"Business","id":126,"reference":"NCL2022049","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"Hospital de \u00d3rbigo","postcode":"24286","form":2,"jobform":'null',"sector":2,"lat":"42.476450463663056","lng":"-5.880759453902438"},{"type":"Business","id":127,"reference":"NCL2022050","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"VEGUELLINA DE \u00d3RBIGO","postcode":"24350","form":2,"jobform":'null',"sector":6,"lat":"42.4434780326517","lng":"-5.883102248817355"},{"type":"Business","id":128,"reference":"NCM2022003","community":7,"ccaa":"Castilla La Mancha","province":"Ciudad Real","town":"Villanueva de  los Infantes","postcode":"13320","form":1,"jobform":'null',"sector":2,"lat":"38.74048717450804","lng":"-3.0096137657909745"},{"type":"Business","id":129,"reference":"NCM2022004","community":7,"ccaa":"Castilla La Mancha","province":"Cuenca","town":"El Provencio","postcode":"16670","form":4,"jobform":'null',"sector":13,"lat":"39.426930856158975","lng":"-2.5718998643376594"},{"type":"Business","id":130,"reference":"NCL2022051","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"Carrizo de la Ribera","postcode":"24270","form":2,"jobform":'null',"sector":13,"lat":"42.61325192237683","lng":"-5.8388558136843685"},{"type":"Business","id":132,"reference":"NCL2022053","community":8,"ccaa":"Castilla y Leon","province":"Avila","town":"GILBUENA","postcode":"05619","form":2,"jobform":'null',"sector":2,"lat":"40.41975904264449","lng":"-5.605730662536861"},{"type":"Business","id":133,"reference":"NCL2022054","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"Benavides","postcode":"24280","form":2,"jobform":'null',"sector":6,"lat":"42.50518406134272","lng":"-5.899047724979384"},{"type":"Business","id":136,"reference":"NAR2022010","community":2,"ccaa":"Aragon","province":"Teruel","town":"Lanzuela","postcode":"44491","form":4,"jobform":'null',"sector":2,"lat":"41.11785807782834","lng":"-1.2022948066428452"},{"type":"Business","id":137,"reference":"NVC2022009","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"R\u00f2tova","postcode":"46725","form":1,"jobform":'null',"sector":10,"lat":"38.9397947","lng":"-0.2701692"},{"type":"Business","id":139,"reference":"NCL2022056","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"VEGUELLINA DE \u00d3RBIGO","postcode":"24350","form":2,"jobform":'null',"sector":13,"lat":"42.44260083902818","lng":"-5.8855815596253125"},{"type":"Business","id":142,"reference":"NCL2022057","community":8,"ccaa":"Castilla y Leon","province":"Avila","town":"Mengamu\u00f1oz","postcode":"05131","form":2,"jobform":'null',"sector":2,"lat":"40.57141335510937","lng":"-4.987827553333086"},{"type":"Business","id":143,"reference":"NCL2022058","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"RIEGO DE AMBROS","postcode":"24413","form":1,"jobform":'null',"sector":2,"lat":"42.52448698385704","lng":"-6.47939744015 Latitud 6119"},{"type":"Business","id":144,"reference":"NEX2022009","community":10,"ccaa":"Extremadura","province":"Caceres","town":"Gargantilla","postcode":"10749","form":2,"jobform":'null',"sector":2,"lat":"40.249602788936045","lng":"-5.920285497066451"},{"type":"Business","id":145,"reference":"NEX2022010","community":10,"ccaa":"Extremadura","province":"Caceres","town":"Gargantilla","postcode":"10749","form":2,"jobform":'null',"sector":10,"lat":"40.25011620980373","lng":"-5.920397612268799"},{"type":"Business","id":146,"reference":"NEX2022011","community":10,"ccaa":"Extremadura","province":"Caceres","town":"Herv\u00e1s","postcode":"10700","form":3,"jobform":'null',"sector":13,"lat":"40.27363831395706","lng":"-5.861291496850734"},{"type":"Business","id":147,"reference":"NCL2022059","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"San Mart\u00edn del Camino","postcode":"24393","form":2,"jobform":'null',"sector":2,"lat":"42.49758404440626","lng":"-5.809453332389006"},{"type":"Business","id":148,"reference":"NEX2022012","community":10,"ccaa":"Extremadura","province":"Badajoz","town":"Zalamea de la Serena","postcode":"06430","form":7,"jobform":'null',"sector":6,"lat":"38.65757125866637","lng":"-5.659983680920507"},{"type":"Business","id":149,"reference":"NEX2022013","community":10,"ccaa":"Extremadura","province":"Caceres","town":"Aldeanueva del Camino","postcode":"10740","form":2,"jobform":'null',"sector":13,"lat":"40.26006553726059","lng":"-5.933158101354831"},{"type":"Business","id":150,"reference":"NCL2022060","community":8,"ccaa":"Castilla y Leon","province":"Avila","town":"Barco de \u00c1vila","postcode":"05600","form":7,"jobform":'null',"sector":9,"lat":"40.3592333","lng":"-5.5236"},{"type":"Business","id":151,"reference":"NEX2022014","community":10,"ccaa":"Extremadura","province":"Caceres","town":"Herv\u00e1s","postcode":"10700","form":2,"jobform":'null',"sector":13,"lat":"40.2734661","lng":"-5.8628467"},{"type":"Business","id":152,"reference":"NAS2022011","community":3,"ccaa":"Asturias","province":"Asturias","town":"Infiesto \/ L\u0027Infiestu","postcode":"33530","form":9,"jobform":'null',"sector":2,"lat":"43.3471387","lng":"-5.3630233"},{"type":"Business","id":153,"reference":"NCL2022061","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"Cogorderos","postcode":"24712","form":2,"jobform":'null',"sector":2,"lat":"42.5437329","lng":"-6.0283638"},{"type":"Business","id":155,"reference":"NVC2022010","community":17,"ccaa":"Comunidad Valenciana","province":"Castellon","town":"Zucaina","postcode":"12125","form":2,"jobform":'null',"sector":10,"lat":"40.133517","lng":"-0.419434"},{"type":"Business","id":156,"reference":"NAR2022011","community":2,"ccaa":"Aragon","province":"Zaragoza","town":"Belchite","postcode":"50130","form":1,"jobform":'null',"sector":10,"lat":"41.302676","lng":"-0.755154"},{"type":"Business","id":157,"reference":"NCL2023063","community":8,"ccaa":"Castilla y Leon","province":"Avila","town":"EL BARCO DE \u00c1VILA","postcode":"05600","form":2,"jobform":'null',"sector":2,"lat":"40.3569063","lng":"-5.5236746"},{"type":"Business","id":160,"reference":"NAR2023013","community":2,"ccaa":"Aragon","province":"Zaragoza","town":"Orera","postcode":"50331","form":9,"jobform":'null',"sector":2,"lat":"41.2983632","lng":"-1.4791251"},{"type":"Business","id":161,"reference":"NVC2023011","community":17,"ccaa":"Comunidad Valenciana","province":"Castellon","town":"Barona","postcode":"12194","form":2,"jobform":'null',"sector":2,"lat":"40.1660489","lng":"-0.0638746"},{"type":"Business","id":162,"reference":"NCL2023065","community":8,"ccaa":"Castilla y Leon","province":"Avila","town":"LA LASTRA DEL CANO","postcode":"05630","form":2,"jobform":'null',"sector":2,"lat":"40.3538449","lng":"-5.4067177"},{"type":"Business","id":163,"reference":"NCL2023066","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"Barrios de Luna","postcode":"24148","form":2,"jobform":'null',"sector":2,"lat":"42.8497549","lng":"-5.8637585"},{"type":"Business","id":164,"reference":"NCL2023067","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"Carrizo","postcode":"24270","form":1,"jobform":'null',"sector":2,"lat":"42.5839551","lng":"-5.8292753"},{"type":"Business","id":165,"reference":"NCL2023068","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"BENAVIDES","postcode":"24280","form":3,"jobform":'null',"sector":14,"lat":"42.5332128","lng":"-5.9621106"},{"type":"Business","id":166,"reference":"NCL2023069","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"BERLANGA DEL BIERZO","postcode":"24438","form":2,"jobform":'null',"sector":2,"lat":"42.7304458","lng":"-6.6003638"},{"type":"Business","id":167,"reference":"NAS2023012","community":3,"ccaa":"Asturias","province":"Asturias","town":"Rozadas","postcode":"33729","form":2,"jobform":'null',"sector":2,"lat":"43.4407325","lng":"-6.8588885"},{"type":"Business","id":168,"reference":"NAS2023013","community":3,"ccaa":"Asturias","province":"Asturias","town":"Rozadas","postcode":"33729","form":1,"jobform":'null',"sector":7,"lat":"43.4407325","lng":"-6.8588885"},{"type":"Business","id":169,"reference":"NAN2023004","community":1,"ccaa":"Andalucia","province":"Malaga","town":"Gauc\u00edn","postcode":"29480","form":7,"jobform":'null',"sector":6,"lat":"36.5190987","lng":"-5.3184194"},{"type":"Business","id":170,"reference":"NCL2023070","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Osorno La Mayor","postcode":"34460","form":3,"jobform":'null',"sector":2,"lat":"42.369478","lng":"-4.3770704"},{"type":"Business","id":171,"reference":"NAS2023014","community":3,"ccaa":"Asturias","province":"Asturias","town":"La Arquera","postcode":"33860","form":1,"jobform":'null',"sector":4,"lat":"43.463931","lng":"-6.2452278"},{"type":"Business","id":172,"reference":"NCL2023071","community":8,"ccaa":"Castilla y Leon","province":"Burgos","town":"Tubilla del Agua","postcode":"09143","form":1,"jobform":'null',"sector":2,"lat":"42.709919","lng":"-3.8051187"},{"type":"Business","id":173,"reference":"NCL2023072","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"Milla del Paramo","postcode":"24393","form":2,"jobform":'null',"sector":2,"lat":"42.4675964","lng":"-5.7991309"},{"type":"Business","id":174,"reference":"NCL2023073","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"Puente de \u00d3rbigo","postcode":"24286","form":3,"jobform":'null',"sector":2,"lat":"42.4636342","lng":"-5.8769366"},{"type":"Business","id":175,"reference":"NCB2023005","community":6,"ccaa":"Cantabria","province":"Cantabria","town":'null',"postcode":'null',"form":2,"jobform":'null',"sector":3,"lat":"43.1577629","lng":"-3.7822445"},{"type":"Business","id":176,"reference":"NCL2023074","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Casc\u00f3n de la Nava","postcode":"34170","form":2,"jobform":'null',"sector":10,"lat":"42.0567375","lng":"-4.6443893"},{"type":"Business","id":177,"reference":"NAS2023015","community":3,"ccaa":"Asturias","province":"Asturias","town":"Salass","postcode":"33860","form":1,"jobform":'null',"sector":2,"lat":"43.4093046","lng":"-6.2621567"},{"type":"Business","id":178,"reference":"NCL2023075","community":8,"ccaa":"Castilla y Leon","province":"Avila","town":"Padiernos","postcode":"05520","form":2,"jobform":'null',"sector":2,"lat":"40.6224505","lng":"-4.8462294"},{"type":"Business","id":179,"reference":"NCL2023076","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Carri\u00f3n de los Condes","postcode":"34120","form":2,"jobform":'null',"sector":2,"lat":"42.3307302","lng":"-4.6140481"},{"type":"Business","id":180,"reference":"NCL2023077","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"PERANZANES","postcode":"24429","form":2,"jobform":'null',"sector":2,"lat":"42.8770147","lng":"-6.6334158"},{"type":"Business","id":181,"reference":"NGA2023010","community":11,"ccaa":"Galicia","province":"Ourense","town":'null',"postcode":'null',"form":4,"jobform":'null',"sector":2,"lat":"41.9896162","lng":"-7.4225881"},{"type":"Business","id":182,"reference":"NCL2023078","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Carri\u00f3n de los Condes","postcode":"34120","form":7,"jobform":'null',"sector":6,"lat":"42.3307302","lng":"-4.6140481"},{"type":"Business","id":183,"reference":"NGA2023011","community":11,"ccaa":"Galicia","province":"Ourense","town":"Vilardev\u00f3s","postcode":"32616","form":2,"jobform":'null',"sector":2,"lat":"41.907314","lng":"-7.3125592"},{"type":"Business","id":184,"reference":"NCL2023079","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"Santib\u00e1\u00f1ez de la Isla","postcode":"24795","form":2,"jobform":'null',"sector":2,"lat":"42.3749176","lng":"-5.9350268"},{"type":"Business","id":185,"reference":"NCL2023080","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"San Crist\u00f3bal de la Polantera","postcode":"24359","form":2,"jobform":'null',"sector":2,"lat":"42.3910551","lng":"-5.907295"},{"type":"Business","id":186,"reference":"NAR2023014","community":2,"ccaa":"Aragon","province":"Zaragoza","town":"S\u00e1daba","postcode":"50670","form":2,"jobform":'null',"sector":2,"lat":"41.6542253","lng":"-0.8974682"},{"type":"Business","id":187,"reference":"NVC2023012","community":17,"ccaa":"Comunidad Valenciana","province":"Alicante","town":"L\u0027ORXA","postcode":"03860","form":3,"jobform":'null',"sector":2,"lat":"38.8443159","lng":"-0.3107403"},{"type":"House","id":4,"reference":"118","community":3,"ccaa":"Asturias","province":"Asturias","town":"Llandepereira","postcode":"33739","form":5,"jobform":'null',"sector":'null',"lat":"43.1666437","lng":"-6.9531225"},{"type":"House","id":5,"reference":"188","community":1,"ccaa":"Andalucia","province":"Malaga","town":"Benalaur\u00eda","postcode":"29491","form":1,"jobform":'null',"sector":'null',"lat":"36.5943747","lng":"-5.2609348"},{"type":"House","id":8,"reference":"119","community":3,"ccaa":"Asturias","province":"Asturias","town":"Pelou","postcode":"33730","form":5,"jobform":'null',"sector":'null',"lat":"43.1603124","lng":"-6.9009585"},{"type":"House","id":11,"reference":"189","community":1,"ccaa":"Andalucia","province":"Malaga","town":"Benalaur\u00eda","postcode":"29491","form":1,"jobform":'null',"sector":'null',"lat":"36.5943747","lng":"-5.2609348"},{"type":"House","id":12,"reference":"120","community":3,"ccaa":"Asturias","province":"Asturias","town":"Villarmayor","postcode":"33730","form":5,"jobform":'null',"sector":'null',"lat":"41.0145073","lng":"-5.9740049"},{"type":"House","id":13,"reference":"190","community":1,"ccaa":"Andalucia","province":"Malaga","town":"Benalaur\u00eda","postcode":"29491","form":2,"jobform":'null',"sector":'null',"lat":"36.5943747","lng":"-5.2609348"},{"type":"House","id":16,"reference":"121","community":3,"ccaa":"Asturias","province":"Asturias","town":"Santamarina","postcode":"33158","form":5,"jobform":'null',"sector":'null',"lat":"43.5547369","lng":"-6.3107788"},{"type":"House","id":17,"reference":"191","community":1,"ccaa":"Andalucia","province":"Malaga","town":"Benalaur\u00eda","postcode":"29491","form":1,"jobform":'null',"sector":'null',"lat":"36.5943747","lng":"-5.2609348"},{"type":"House","id":24,"reference":"192","community":1,"ccaa":"Andalucia","province":"Malaga","town":"Benalaur\u00eda","postcode":"29491","form":1,"jobform":'null',"sector":'null',"lat":"36.5943747","lng":"-5.2609348"},{"type":"House","id":25,"reference":"124","community":3,"ccaa":"Asturias","province":"Asturias","town":"Penadecabras","postcode":"33757","form":5,"jobform":'null',"sector":'null',"lat":"43.4650721","lng":"-6.8428051"},{"type":"House","id":27,"reference":"193","community":1,"ccaa":"Andalucia","province":"Malaga","town":"Benalaur\u00eda","postcode":"29491","form":1,"jobform":'null',"sector":'null',"lat":"36.5943747","lng":"-5.2609348"},{"type":"House","id":29,"reference":"125","community":3,"ccaa":"Asturias","province":"Asturias","town":"Romelle","postcode":"33757","form":5,"jobform":'null',"sector":'null',"lat":"43.4750103","lng":"-6.8619045"},{"type":"House","id":31,"reference":"194","community":1,"ccaa":"Andalucia","province":"Malaga","town":"Benalaur\u00eda","postcode":"29491","form":1,"jobform":'null',"sector":'null',"lat":"36.5943747","lng":"-5.2609348"},{"type":"House","id":36,"reference":"195","community":1,"ccaa":"Andalucia","province":"Malaga","town":"Benalaur\u00eda","postcode":"29491","form":1,"jobform":'null',"sector":'null',"lat":"36.5943747","lng":"-5.2609348"},{"type":"House","id":38,"reference":"196","community":1,"ccaa":"Andalucia","province":"Malaga","town":"Benalaur\u00eda","postcode":"29491","form":1,"jobform":'null',"sector":'null',"lat":"36.5943747","lng":"-5.2609348"},{"type":"House","id":39,"reference":"128","community":3,"ccaa":"Asturias","province":"Asturias","town":"And\u00eda","postcode":"33756","form":5,"jobform":'null',"sector":'null',"lat":"43.487748906149335","lng":"-6.831956811731843"},{"type":"House","id":41,"reference":"197","community":1,"ccaa":"Andalucia","province":"Malaga","town":"Benalaur\u00eda","postcode":"29491","form":1,"jobform":'null',"sector":'null',"lat":"36.5943747","lng":"-5.2609348"},{"type":"House","id":42,"reference":"129","community":3,"ccaa":"Asturias","province":"Asturias","town":"Villalmarzo","postcode":"33759","form":5,"jobform":'null',"sector":'null',"lat":"43.5085132","lng":"-6.8500086"},{"type":"House","id":44,"reference":"130","community":3,"ccaa":"Asturias","province":"Asturias","town":"Miudes","postcode":"33758","form":5,"jobform":'null',"sector":'null',"lat":"43.529508","lng":"-6.8105873"},{"type":"House","id":47,"reference":"131","community":3,"ccaa":"Asturias","province":"Asturias","town":"San Xuan","postcode":"33417","form":5,"jobform":'null',"sector":'null',"lat":"43.5877915","lng":"-5.9356583"},{"type":"House","id":48,"reference":"198","community":1,"ccaa":"Andalucia","province":"Malaga","town":"Benalaur\u00eda","postcode":"29491","form":1,"jobform":'null',"sector":'null',"lat":"36.5943747","lng":"-5.2609348"},{"type":"House","id":51,"reference":"132","community":3,"ccaa":"Asturias","province":"Asturias","town":"Sueiro","postcode":"33759","form":5,"jobform":'null',"sector":'null',"lat":"43.5288353","lng":"-6.8709885"},{"type":"House","id":52,"reference":"199","community":1,"ccaa":"Andalucia","province":"Malaga","town":"Benalaur\u00eda","postcode":"29491","form":1,"jobform":'null',"sector":'null',"lat":"36.5943747","lng":"-5.2609348"},{"type":"House","id":53,"reference":"52","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Poblaci\u00f3n de Arroyo","postcode":"34347","form":1,"jobform":'null',"sector":'null',"lat":"42.3368583","lng":"-4.8744304"},{"type":"House","id":55,"reference":"133","community":3,"ccaa":"Asturias","province":"Asturias","town":"A Bra\u00f1a","postcode":"33757","form":5,"jobform":'null',"sector":'null',"lat":"43.4763472","lng":"-6.8591222"},{"type":"House","id":56,"reference":"200","community":1,"ccaa":"Andalucia","province":"Malaga","town":"Benalaur\u00eda","postcode":"29491","form":2,"jobform":'null',"sector":'null',"lat":"36.5943747","lng":"-5.2609348"},{"type":"House","id":60,"reference":"54","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Villalc\u00f3n","postcode":"34347","form":1,"jobform":'null',"sector":'null',"lat":"42.2931395","lng":"-4.8553046"},{"type":"House","id":62,"reference":"201","community":1,"ccaa":"Andalucia","province":"Malaga","town":"Benalaur\u00eda","postcode":"29491","form":2,"jobform":'null',"sector":'null',"lat":"36.5943747","lng":"-5.2609348"},{"type":"House","id":74,"reference":"137","community":3,"ccaa":"Asturias","province":"Asturias","town":"Lijou","postcode":"33735","form":5,"jobform":'null',"sector":'null',"lat":"43.29803532782073","lng":"-6.876113920590566"},{"type":"House","id":76,"reference":"205","community":1,"ccaa":"Andalucia","province":"Malaga","town":"Benalaur\u00eda","postcode":"29491","form":1,"jobform":'null',"sector":'null',"lat":"36.5943747","lng":"-5.2609348"},{"type":"House","id":79,"reference":"206","community":1,"ccaa":"Andalucia","province":"Malaga","town":"Benalaur\u00eda","postcode":"29491","form":1,"jobform":'null',"sector":'null',"lat":"36.5943747","lng":"-5.2609348"},{"type":"House","id":86,"reference":"140","community":3,"ccaa":"Asturias","province":"Asturias","town":"Pesoz","postcode":"33735","form":5,"jobform":'null',"sector":'null',"lat":"43.257105","lng":"-6.8763166"},{"type":"House","id":87,"reference":"208","community":11,"ccaa":"Galicia","province":"Ourense","town":"PADROSO (SARREAUS)","postcode":"32631","form":5,"jobform":'null',"sector":'null',"lat":"42.1042981","lng":"-7.6066777"},{"type":"House","id":89,"reference":"141","community":3,"ccaa":"Asturias","province":"Asturias","town":"Pesoz","postcode":"33735","form":5,"jobform":'null',"sector":'null',"lat":"43.257105","lng":"-6.8763166"},{"type":"House","id":91,"reference":"22","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Villarmentero de Campos","postcode":"34447","form":1,"jobform":'null',"sector":'null',"lat":"42.2977807","lng":"-4.4999368"},{"type":"House","id":99,"reference":"211","community":11,"ccaa":"Galicia","province":"Ourense","town":"LODOSELO (SARREAUS)","postcode":"32696","form":1,"jobform":'null',"sector":'null',"lat":"42.0536672","lng":"-7.6005786"},{"type":"House","id":103,"reference":"25","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Carri\u00f3n de los Condes","postcode":"34120","form":1,"jobform":'null',"sector":'null',"lat":"42.3307302","lng":"-4.6140481"},{"type":"House","id":110,"reference":"214","community":11,"ccaa":"Galicia","province":"Ourense","town":"LODOSELO (SARREAUS)","postcode":"32696","form":1,"jobform":'null',"sector":'null',"lat":"42.0536672","lng":"-7.6005786"},{"type":"House","id":112,"reference":"147","community":3,"ccaa":"Asturias","province":"Asturias","town":"Santa Eufemia","postcode":"33776","form":5,"jobform":'null',"sector":'null',"lat":"43.3040607","lng":"-6.9919069"},{"type":"House","id":121,"reference":"149","community":3,"ccaa":"Asturias","province":"Asturias","town":"Espinareu","postcode":"33537","form":1,"jobform":'null',"sector":'null',"lat":"43.2908509","lng":"-5.3625516"},{"type":"House","id":124,"reference":"150","community":3,"ccaa":"Asturias","province":"Asturias","town":"El Otero","postcode":"33818","form":1,"jobform":'null',"sector":'null',"lat":"43.0589615","lng":"-6.4850111"},{"type":"House","id":126,"reference":"218","community":6,"ccaa":"Cantabria","province":"Cantabria","town":"VEGA DE PAS","postcode":"39685","form":1,"jobform":'null',"sector":'null',"lat":"43.1577629","lng":"-3.7822445"},{"type":"House","id":127,"reference":"31","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Fr\u00f3mista","postcode":"34440","form":1,"jobform":'null',"sector":'null',"lat":"42.2676329","lng":"-4.4053497"},{"type":"House","id":131,"reference":"499","community":6,"ccaa":"Cantabria","province":"Cantabria","town":"Collado de Cieza","postcode":"39407","form":2,"jobform":'null',"sector":'null',"lat":"43.2397221","lng":"-4.0966671"},{"type":"House","id":134,"reference":"152","community":3,"ccaa":"Asturias","province":"Asturias","town":"Rozapanera","postcode":"33537","form":1,"jobform":'null',"sector":'null',"lat":"43.290909738986045","lng":"-5.353515140434169"},{"type":"House","id":136,"reference":"221","community":6,"ccaa":"Cantabria","province":"Cantabria","town":"VEGA DE PAS","postcode":"39685","form":1,"jobform":'null',"sector":'null',"lat":"43.1577629","lng":"-3.7822445"},{"type":"House","id":141,"reference":"222","community":6,"ccaa":"Cantabria","province":"Cantabria","town":"VEGA DE PAS","postcode":"39685","form":1,"jobform":'null',"sector":'null',"lat":"43.1577629","lng":"-3.7822445"},{"type":"House","id":157,"reference":"38","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Fr\u00f3mista","postcode":"34440","form":1,"jobform":'null',"sector":'null',"lat":"42.2676329","lng":"-4.4053497"},{"type":"House","id":161,"reference":"159","community":8,"ccaa":"Castilla y Leon","province":"Valladolid","town":"Tordehumos","postcode":"47830","form":1,"jobform":'null',"sector":'null',"lat":"41.8152783","lng":"-5.1575429"},{"type":"House","id":164,"reference":"160","community":8,"ccaa":"Castilla y Leon","province":"Valladolid","town":"Tordehumos","postcode":"47830","form":1,"jobform":'null',"sector":'null',"lat":"41.8152783","lng":"-5.1575429"},{"type":"House","id":165,"reference":"275","community":8,"ccaa":"Castilla y Leon","province":"Valladolid","town":"Mayorga","postcode":"47680","form":2,"jobform":'null',"sector":'null',"lat":"42.1662402","lng":"-5.2645259"},{"type":"House","id":166,"reference":"229","community":6,"ccaa":"Cantabria","province":"Cantabria","town":"LUENA","postcode":"39681","form":1,"jobform":'null',"sector":'null',"lat":"43.0608122","lng":"-3.8757119"},{"type":"House","id":168,"reference":"230","community":6,"ccaa":"Cantabria","province":"Cantabria","town":"LUENA","postcode":"39681","form":1,"jobform":'null',"sector":'null',"lat":"43.0608122","lng":"-3.8757119"},{"type":"House","id":169,"reference":"276","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"Albires","postcode":"24293","form":1,"jobform":'null',"sector":'null',"lat":"42.2528305","lng":"-5.2677908"},{"type":"House","id":170,"reference":"81","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Fr\u00f3mista","postcode":"34440","form":6,"jobform":'null',"sector":'null',"lat":"42.2676329","lng":"-4.4053497"},{"type":"House","id":171,"reference":"161","community":8,"ccaa":"Castilla y Leon","province":"Valladolid","town":"Tordehumos","postcode":"47830","form":1,"jobform":'null',"sector":'null',"lat":"41.8152783","lng":"-5.1575429"},{"type":"House","id":172,"reference":"277","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"Albires","postcode":"24293","form":1,"jobform":'null',"sector":'null',"lat":"42.2528305","lng":"-5.2677908"},{"type":"House","id":175,"reference":"278","community":8,"ccaa":"Castilla y Leon","province":"Valladolid","town":"Melgar de Abajo","postcode":"47687","form":2,"jobform":'null',"sector":'null',"lat":"42.2437542","lng":"-5.1425553"},{"type":"House","id":176,"reference":"82","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Lomas de Campos","postcode":"34449","form":1,"jobform":'null',"sector":'null',"lat":"42.2736197","lng":"-4.5514827"},{"type":"House","id":179,"reference":"279","community":8,"ccaa":"Castilla y Leon","province":"Valladolid","town":"Melgar de Abajo","postcode":"47687","form":2,"jobform":'null',"sector":'null',"lat":"42.2437542","lng":"-5.1425553"},{"type":"House","id":181,"reference":"280","community":8,"ccaa":"Castilla y Leon","province":"Valladolid","town":"Melgar de Abajo","postcode":"47687","form":2,"jobform":'null',"sector":'null',"lat":"42.2437542","lng":"-5.1425553"},{"type":"House","id":183,"reference":"84","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Poblaci\u00f3n de Arroyo","postcode":"34347","form":1,"jobform":'null',"sector":'null',"lat":"42.3368583","lng":"-4.8744304"},{"type":"House","id":184,"reference":"281","community":8,"ccaa":"Castilla y Leon","province":"Valladolid","town":"Melgar de Abajo","postcode":"47687","form":1,"jobform":'null',"sector":'null',"lat":"42.2437542","lng":"-5.1425553"},{"type":"House","id":185,"reference":"164","community":8,"ccaa":"Castilla y Leon","province":"Valladolid","town":"Tordehumos","postcode":"47830","form":1,"jobform":'null',"sector":'null',"lat":"41.8152783","lng":"-5.1575429"},{"type":"House","id":187,"reference":"165","community":8,"ccaa":"Castilla y Leon","province":"Valladolid","town":"Barcial de la Loma","postcode":"47674","form":1,"jobform":'null',"sector":'null',"lat":"41.951302","lng":"-5.2834645"},{"type":"House","id":211,"reference":"240","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"HUSILLOS","postcode":"34419","form":1,"jobform":'null',"sector":'null',"lat":"42.08981","lng":"-4.5269923"},{"type":"House","id":212,"reference":"91","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Villalc\u00e1zar de Sirga","postcode":"34449","form":1,"jobform":'null',"sector":'null',"lat":"42.3169992","lng":"-4.5424587"},{"type":"House","id":214,"reference":"170","community":8,"ccaa":"Castilla y Leon","province":"Valladolid","town":"Villabr\u00e1gima","postcode":"47820","form":1,"jobform":'null',"sector":'null',"lat":"41.8220989","lng":"-5.115848"},{"type":"House","id":221,"reference":"292","community":11,"ccaa":"Galicia","province":"Ourense","town":"LODOSELO","postcode":"32696","form":1,"jobform":'null',"sector":'null',"lat":"42.0536672","lng":"-7.6005786"},{"type":"House","id":222,"reference":"172","community":8,"ccaa":"Castilla y Leon","province":"Valladolid","town":"Palazuelo de Vedija","postcode":"47812","form":1,"jobform":'null',"sector":'null',"lat":"41.9297819","lng":"-5.14451"},{"type":"House","id":224,"reference":"243","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"HUSILLOS","postcode":"34419","form":1,"jobform":'null',"sector":'null',"lat":"42.08981","lng":"-4.5269923"},{"type":"House","id":226,"reference":"294","community":6,"ccaa":"Cantabria","province":"Cantabria","town":"SAN PEDRO DEL ROMERAL","postcode":"39686","form":1,"jobform":'null',"sector":'null',"lat":"43.1147857","lng":"-3.8180412"},{"type":"House","id":230,"reference":"295","community":6,"ccaa":"Cantabria","province":"Cantabria","town":"SAN PEDRO DEL ROMERAL","postcode":"39686","form":1,"jobform":'null',"sector":'null',"lat":"43.1147857","lng":"-3.8180412"},{"type":"House","id":231,"reference":"174","community":8,"ccaa":"Castilla y Leon","province":"Valladolid","town":"Palazuelo de Vedija","postcode":"47812","form":5,"jobform":'null',"sector":'null',"lat":"41.9297819","lng":"-5.14451"},{"type":"House","id":234,"reference":"175","community":8,"ccaa":"Castilla y Leon","province":"Valladolid","town":"Pozuelo de la Orden","postcode":"47831","form":1,"jobform":'null',"sector":'null',"lat":"41.8212339","lng":"-5.2586497"},{"type":"House","id":236,"reference":"296","community":6,"ccaa":"Cantabria","province":"Cantabria","town":"VEGA DE PAS","postcode":"39685","form":1,"jobform":'null',"sector":'null',"lat":"43.1577629","lng":"-3.7822445"},{"type":"House","id":240,"reference":"297","community":6,"ccaa":"Cantabria","province":"Cantabria","town":"VEGA DE PAS","postcode":"39685","form":1,"jobform":'null',"sector":'null',"lat":"43.1577629","lng":"-3.7822445"},{"type":"House","id":242,"reference":"248","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"T\u00e1mara de Campos","postcode":"34439","form":1,"jobform":'null',"sector":'null',"lat":"42.2034861","lng":"-4.3931363"},{"type":"House","id":243,"reference":"177","community":8,"ccaa":"Castilla y Leon","province":"Valladolid","town":"San Pedro de Latarce","postcode":"47851","form":1,"jobform":'null',"sector":'null',"lat":"41.7357226","lng":"-5.3246285"},{"type":"House","id":244,"reference":"298","community":6,"ccaa":"Cantabria","province":"Cantabria","town":"VEGA DE PAS","postcode":"39685","form":1,"jobform":'null',"sector":'null',"lat":"43.1577629","lng":"-3.7822445"},{"type":"House","id":247,"reference":"249","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Valdespina","postcode":"34420","form":1,"jobform":'null',"sector":'null',"lat":"42.1695222","lng":"-4.4593791"},{"type":"House","id":250,"reference":"250","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Valdespina","postcode":"34419","form":1,"jobform":'null',"sector":'null',"lat":"42.0803287","lng":"-4.4696765"},{"type":"House","id":254,"reference":"251","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Amusco","postcode":"34419","form":1,"jobform":'null',"sector":'null',"lat":"42.0803287","lng":"-4.4696765"},{"type":"House","id":261,"reference":"103","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Carri\u00f3n de los Condes","postcode":"34120","form":1,"jobform":'null',"sector":'null',"lat":"42.3307302","lng":"-4.6140481"},{"type":"House","id":262,"reference":"253","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"SANTOYO","postcode":"34490","form":6,"jobform":'null',"sector":'null',"lat":"42.21439644457152","lng":"-4.341334460072892"},{"type":"House","id":265,"reference":"254","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"SANTOYO","postcode":"34490","form":1,"jobform":'null',"sector":'null',"lat":"42.214452363200536","lng":"-4.341258959189276"},{"type":"House","id":267,"reference":"304","community":6,"ccaa":"Cantabria","province":"Cantabria","town":"SELAYA","postcode":"39696","form":1,"jobform":'null',"sector":'null',"lat":"43.1997485","lng":"-3.7739869"},{"type":"House","id":268,"reference":"104","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Villaturde","postcode":"34129","form":1,"jobform":'null',"sector":'null',"lat":"42.3784494","lng":"-4.6711939"},{"type":"House","id":269,"reference":"255","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"MONZON DE CAMPOS","postcode":"34410","form":1,"jobform":'null',"sector":'null',"lat":"42.1159093","lng":"-4.4939766"},{"type":"House","id":271,"reference":"105","community":3,"ccaa":"Asturias","province":"Asturias","town":"Trev\u00e9","postcode":"33729","form":5,"jobform":'null',"sector":'null',"lat":"43.4506947","lng":"-6.8751434"},{"type":"House","id":272,"reference":"256","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"MONZON DE CAMPOS","postcode":"34410","form":1,"jobform":'null',"sector":'null',"lat":"42.1159093","lng":"-4.4939766"},{"type":"House","id":273,"reference":"305","community":6,"ccaa":"Cantabria","province":"Cantabria","town":"SAN PEDRO DEL ROMERAL","postcode":"39686","form":1,"jobform":'null',"sector":'null',"lat":"43.1147857","lng":"-3.8180412"},{"type":"House","id":275,"reference":"185","community":1,"ccaa":"Andalucia","province":"Malaga","town":"Benalaur\u00eda","postcode":"29491","form":1,"jobform":'null',"sector":'null',"lat":"36.5943747","lng":"-5.2609348"},{"type":"House","id":277,"reference":"106","community":3,"ccaa":"Asturias","province":"Asturias","town":"Villanueva","postcode":"33725","form":5,"jobform":'null',"sector":'null',"lat":"43.4507289","lng":"-6.7868794"},{"type":"House","id":279,"reference":"307","community":11,"ccaa":"Galicia","province":"Ourense","town":"BARBADAS","postcode":"32890","form":1,"jobform":'null',"sector":'null',"lat":"42.2981451","lng":"-7.8871273"},{"type":"House","id":280,"reference":"107","community":3,"ccaa":"Asturias","province":"Asturias","town":"Pi\u00f1eira","postcode":"33392","form":5,"jobform":'null',"sector":'null',"lat":"43.4800012","lng":"-5.7384442"},{"type":"House","id":281,"reference":"258","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"VILLAJIMENA","postcode":"34419","form":1,"jobform":'null',"sector":'null',"lat":"42.0851306","lng":"-4.4333224"},{"type":"House","id":282,"reference":"108","community":3,"ccaa":"Asturias","province":"Asturias","town":"Lendiglesia","postcode":"33727","form":5,"jobform":'null',"sector":'null',"lat":"43.3886671","lng":"-6.7815237"},{"type":"House","id":284,"reference":"308","community":11,"ccaa":"Galicia","province":"Ourense","town":"REBOREDA - PORQUEIRA","postcode":"32643","form":1,"jobform":'null',"sector":'null',"lat":"42.0100444","lng":"-7.8728403"},{"type":"House","id":285,"reference":"260","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"FUENTES DE VALDEPERO","postcode":"34419","form":1,"jobform":'null',"sector":'null',"lat":"42.0747776","lng":"-4.5012249"},{"type":"House","id":287,"reference":"109","community":3,"ccaa":"Asturias","province":"Asturias","town":"Sarceda","postcode":"33727","form":5,"jobform":'null',"sector":'null',"lat":"43.3736275","lng":"-6.8047905"},{"type":"House","id":288,"reference":"309","community":1,"ccaa":"Andalucia","province":"Granada","town":"Galera","postcode":"18840","form":1,"jobform":'null',"sector":'null',"lat":"37.7420931","lng":"-2.5513692"},{"type":"House","id":289,"reference":"110","community":3,"ccaa":"Asturias","province":"Asturias","town":"Merou","postcode":"33727","form":5,"jobform":'null',"sector":'null',"lat":"43.4018747","lng":"-6.7990986"},{"type":"House","id":290,"reference":"261","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"FUENTES DE VALDEPERO","postcode":"34419","form":1,"jobform":'null',"sector":'null',"lat":"42.0747776","lng":"-4.5012249"},{"type":"House","id":296,"reference":"322","community":6,"ccaa":"Cantabria","province":"Cantabria","town":"VEGA DE PAS","postcode":"39685","form":2,"jobform":'null',"sector":'null',"lat":"43.1577629","lng":"-3.7822445"},{"type":"House","id":297,"reference":"313","community":11,"ccaa":"Galicia","province":"Lugo","town":"Moreiras, San Miguel do Monte - Chantada","postcode":"27511","form":1,"jobform":'null',"sector":'null',"lat":"42.6109991","lng":"-7.7706788"},{"type":"House","id":298,"reference":"264","community":8,"ccaa":"Castilla y Leon","province":"Valladolid","town":"Mayorga","postcode":"47680","form":1,"jobform":'null',"sector":'null',"lat":"42.1662402","lng":"-5.2645259"},{"type":"House","id":301,"reference":"265","community":8,"ccaa":"Castilla y Leon","province":"Valladolid","town":"Mayorga","postcode":"47680","form":1,"jobform":'null',"sector":'null',"lat":"42.1662402","lng":"-5.2645259"},{"type":"House","id":302,"reference":"266","community":8,"ccaa":"Castilla y Leon","province":"Valladolid","town":"Mayorga","postcode":"47680","form":5,"jobform":'null',"sector":'null',"lat":"42.1662402","lng":"-5.2645259"},{"type":"House","id":303,"reference":"111","community":3,"ccaa":"Asturias","province":"Asturias","town":"Villar de San Pedro","postcode":"33728","form":6,"jobform":'null',"sector":'null',"lat":"42.0942643","lng":"-7.8014482"},{"type":"House","id":304,"reference":"267","community":8,"ccaa":"Castilla y Leon","province":"Valladolid","town":"Mayorga","postcode":"47680","form":1,"jobform":'null',"sector":'null',"lat":"42.1662402","lng":"-5.2645259"},{"type":"House","id":305,"reference":"268","community":8,"ccaa":"Castilla y Leon","province":"Valladolid","town":"Mayorga","postcode":"47680","form":1,"jobform":'null',"sector":'null',"lat":"42.1662402","lng":"-5.2645259"},{"type":"House","id":306,"reference":"112","community":3,"ccaa":"Asturias","province":"Asturias","town":"Obanza","postcode":"33778","form":5,"jobform":'null',"sector":'null',"lat":"43.453803","lng":"-6.974604"},{"type":"House","id":307,"reference":"269","community":8,"ccaa":"Castilla y Leon","province":"Valladolid","town":"Mayorga","postcode":"47680","form":1,"jobform":'null',"sector":'null',"lat":"42.1662402","lng":"-5.2645259"},{"type":"House","id":308,"reference":"113","community":3,"ccaa":"Asturias","province":"Asturias","town":"Vilar\u00edn","postcode":"27659","form":5,"jobform":'null',"sector":'null',"lat":"42.930735","lng":"-7.0175879"},{"type":"House","id":309,"reference":"114","community":3,"ccaa":"Asturias","province":"Asturias","town":"Xonte","postcode":"33519","form":5,"jobform":'null',"sector":'null',"lat":"43.4124892","lng":"-5.6352801"},{"type":"House","id":310,"reference":"270","community":8,"ccaa":"Castilla y Leon","province":"Valladolid","town":"Mayorga","postcode":"47680","form":1,"jobform":'null',"sector":'null',"lat":"42.1662402","lng":"-5.2645259"},{"type":"House","id":312,"reference":"271","community":8,"ccaa":"Castilla y Leon","province":"Valladolid","town":"Mayorga","postcode":"47680","form":1,"jobform":'null',"sector":'null',"lat":"42.1662402","lng":"-5.2645259"},{"type":"House","id":315,"reference":"272","community":8,"ccaa":"Castilla y Leon","province":"Valladolid","town":"Mayorga","postcode":"47680","form":1,"jobform":'null',"sector":'null',"lat":"42.1662402","lng":"-5.2645259"},{"type":"House","id":318,"reference":"273","community":8,"ccaa":"Castilla y Leon","province":"Valladolid","town":"Mayorga","postcode":"47680","form":1,"jobform":'null',"sector":'null',"lat":"42.1662402","lng":"-5.2645259"},{"type":"House","id":319,"reference":"117","community":3,"ccaa":"Asturias","province":"Asturias","town":"Folgosa","postcode":"33739","form":5,"jobform":'null',"sector":'null',"lat":"43.1734791","lng":"-6.9670511"},{"type":"House","id":321,"reference":"274","community":8,"ccaa":"Castilla y Leon","province":"Valladolid","town":"Mayorga","postcode":"47680","form":1,"jobform":'null',"sector":'null',"lat":"42.1662402","lng":"-5.2645259"},{"type":"House","id":325,"reference":"317","community":8,"ccaa":"Castilla y Leon","province":"Burgos","town":"Torresandino","postcode":"09310","form":1,"jobform":'null',"sector":'null',"lat":"41.8295399","lng":"-3.9151847"},{"type":"House","id":327,"reference":"318","community":11,"ccaa":"Galicia","province":"A Coru\u00f1a","town":"Esteiro","postcode":"15240","form":1,"jobform":'null',"sector":'null',"lat":"42.7875914","lng":"-8.9842315"},{"type":"House","id":328,"reference":"355","community":10,"ccaa":"Extremadura","province":"Badajoz","town":"Vivares","postcode":"06412","form":1,"jobform":'null',"sector":'null',"lat":"39.0867606","lng":"-5.8967338"},{"type":"House","id":332,"reference":"388","community":10,"ccaa":"Extremadura","province":"Caceres","town":"El Torviscol","postcode":"06719","form":1,"jobform":'null',"sector":'null',"lat":"39.0708005","lng":"-5.7352478"},{"type":"House","id":333,"reference":"331","community":2,"ccaa":"Aragon","province":"Zaragoza","town":"Maella","postcode":"50710","form":1,"jobform":'null',"sector":'null',"lat":"41.1230647","lng":"0.1395532"},{"type":"House","id":336,"reference":"332","community":10,"ccaa":"Extremadura","province":"Badajoz","town":"Puebla del Maestre","postcode":"06906","form":1,"jobform":'null',"sector":'null',"lat":"38.0851179","lng":"-6.0791724"},{"type":"House","id":339,"reference":"333","community":6,"ccaa":"Cantabria","province":"Cantabria","town":"Bolmir","postcode":"39213","form":1,"jobform":'null',"sector":'null',"lat":"42.9761191","lng":"-4.0797673"},{"type":"House","id":343,"reference":"334","community":6,"ccaa":"Cantabria","province":"Cantabria","town":"Bolmir","postcode":"39213","form":1,"jobform":'null',"sector":'null',"lat":"42.9761191","lng":"-4.0797673"},{"type":"House","id":346,"reference":"392","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"Castielfabib","postcode":"46141","form":1,"jobform":'null',"sector":'null',"lat":"40.1309055","lng":"-1.3053578"},{"type":"House","id":349,"reference":"360","community":10,"ccaa":"Extremadura","province":"Badajoz","town":"Calera de Le\u00f3n","postcode":"06292","form":1,"jobform":'null',"sector":'null',"lat":"38.1070754","lng":"-6.3365657"},{"type":"House","id":353,"reference":"361","community":10,"ccaa":"Extremadura","province":"Badajoz","town":"Calera de Le\u00f3n","postcode":"06292","form":1,"jobform":'null',"sector":'null',"lat":"38.1070754","lng":"-6.3365657"},{"type":"House","id":354,"reference":"394","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"Casas Altas","postcode":"46147","form":1,"jobform":'null',"sector":'null',"lat":"40.0393236","lng":"-1.2629691"},{"type":"House","id":358,"reference":"395","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"Casas Bajas","postcode":"46147","form":1,"jobform":'null',"sector":'null',"lat":"40.02988421301763","lng":"-1.2622699939173192"},{"type":"House","id":359,"reference":"338","community":6,"ccaa":"Cantabria","province":"Cantabria","town":"Tezanos de Villacarriedo","postcode":"39649","form":1,"jobform":'null',"sector":'null',"lat":"43.2216045","lng":"-3.8254599"},{"type":"House","id":360,"reference":"438","community":11,"ccaa":"Galicia","province":"Ourense","town":"Figueiredo, Coles","postcode":"32152","form":1,"jobform":'null',"sector":'null',"lat":"42.4148161","lng":"-7.8390267"},{"type":"House","id":361,"reference":"363","community":8,"ccaa":"Castilla y Leon","province":"Valladolid","town":"C\u00f3geces del Monte","postcode":"47313","form":1,"jobform":'null',"sector":'null',"lat":"41.5091581","lng":"-4.3167056"},{"type":"House","id":362,"reference":"396","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"Arroyo Cerezo (Castielfabib)","postcode":"46140","form":1,"jobform":'null',"sector":'null',"lat":"40.1215665","lng":"-1.4196142"},{"type":"House","id":364,"reference":"397","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"Los Santos (Castielfabib)","postcode":"46142","form":1,"jobform":'null',"sector":'null',"lat":"40.1087511","lng":"-1.2830851"},{"type":"House","id":365,"reference":"439","community":11,"ccaa":"Galicia","province":"Ourense","town":"Valdoasno, Nogueira de Ramu\u00edn","postcode":"32161","form":1,"jobform":'null',"sector":'null',"lat":"42.3808662","lng":"-7.7688358"},{"type":"House","id":369,"reference":"440","community":11,"ccaa":"Galicia","province":"Lugo","town":"Arcos de Frades. Pol. Lugo","postcode":"27279","form":1,"jobform":'null',"sector":'null',"lat":"43.1343914","lng":"-7.378006"},{"type":"House","id":371,"reference":"341","community":10,"ccaa":"Extremadura","province":"Caceres","town":"Madro\u00f1era","postcode":"10210","form":1,"jobform":'null',"sector":'null',"lat":"39.4256677","lng":"-5.757632"},{"type":"House","id":372,"reference":"399","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"M\u00e1s de Jacinto (Castielfabib)","postcode":"46144","form":1,"jobform":'null',"sector":'null',"lat":"40.1289541","lng":"-1.2499989"},{"type":"House","id":375,"reference":"400","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"Ademuz","postcode":"46140","form":1,"jobform":'null',"sector":'null',"lat":"40.0613379","lng":"-1.2870052"},{"type":"House","id":376,"reference":"445","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"CASAS ALTAS","postcode":"46147","form":2,"jobform":'null',"sector":'null',"lat":"40.0393236","lng":"-1.2629691"},{"type":"House","id":377,"reference":"442","community":7,"ccaa":"Castilla La Mancha","province":"Ciudad Real","town":"Alcazar de San Juan","postcode":"13600","form":1,"jobform":'null',"sector":'null',"lat":"39.3900748","lng":"-3.2100984"},{"type":"House","id":378,"reference":"367","community":2,"ccaa":"Aragon","province":"Zaragoza","town":"TOBED","postcode":"50325","form":1,"jobform":'null',"sector":'null',"lat":"41.3426616","lng":"-1.3951694"},{"type":"House","id":379,"reference":"401","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"Ademuz","postcode":"46140","form":1,"jobform":'null',"sector":'null',"lat":"40.0613379","lng":"-1.2870052"},{"type":"House","id":380,"reference":"448","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"Casas Altas","postcode":"46147","form":2,"jobform":'null',"sector":'null',"lat":"40.0393236","lng":"-1.2629691"},{"type":"House","id":382,"reference":"368","community":2,"ccaa":"Aragon","province":"Zaragoza","town":"TOBED","postcode":"50325","form":1,"jobform":'null',"sector":'null',"lat":"41.3426616","lng":"-1.3951694"},{"type":"House","id":383,"reference":"386","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"CORULL\u00d3N","postcode":"24514","form":2,"jobform":'null',"sector":'null',"lat":"42.5787434","lng":"-6.8196922"},{"type":"House","id":385,"reference":"444","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"ADEMUZ","postcode":"46140","form":5,"jobform":'null',"sector":'null',"lat":"40.0613379","lng":"-1.2870052"},{"type":"House","id":387,"reference":"387","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"CORULL\u00d3N","postcode":"24514","form":2,"jobform":'null',"sector":'null',"lat":"42.5787434","lng":"-6.8196922"},{"type":"House","id":390,"reference":"354","community":10,"ccaa":"Extremadura","province":"Badajoz","town":"Monterrubio de la serena","postcode":"06427","form":5,"jobform":'null',"sector":'null',"lat":"38.588169","lng":"-5.4450222"},{"type":"House","id":391,"reference":"404","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"Vallanca","postcode":"46145","form":1,"jobform":'null',"sector":'null',"lat":"40.0451459","lng":"-1.3565692"},{"type":"House","id":394,"reference":"405","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"Vallanca","postcode":"46145","form":1,"jobform":'null',"sector":'null',"lat":"40.0451459","lng":"-1.3565692"},{"type":"House","id":395,"reference":"371","community":2,"ccaa":"Aragon","province":"Zaragoza","town":"CODOS","postcode":"50326","form":1,"jobform":'null',"sector":'null',"lat":"41.29922385875746","lng":"-1.374476840092847"},{"type":"House","id":397,"reference":"393","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"Casas Altas","postcode":"46147","form":2,"jobform":'null',"sector":'null',"lat":"40.0393236","lng":"-1.2629691"},{"type":"House","id":398,"reference":"447","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"Ademuz","postcode":"46140","form":5,"jobform":'null',"sector":'null',"lat":"40.0613379","lng":"-1.2870052"},{"type":"House","id":401,"reference":"398","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"Castielfabib","postcode":"46141","form":1,"jobform":'null',"sector":'null',"lat":"40.1309055","lng":"-1.3053578"},{"type":"House","id":403,"reference":"373","community":2,"ccaa":"Aragon","province":"Zaragoza","town":"CODOS","postcode":"50326","form":1,"jobform":'null',"sector":'null',"lat":"41.29282544636516","lng":"-1.3740672336651232"},{"type":"House","id":405,"reference":"402","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"M\u00e1s el Olmo (Ademuz)","postcode":"46140","form":1,"jobform":'null',"sector":'null',"lat":"40.0613379","lng":"-1.2870052"},{"type":"House","id":406,"reference":"449","community":11,"ccaa":"Galicia","province":"Lugo","town":"A FONSAGRADA","postcode":"27113","form":1,"jobform":'null',"sector":'null',"lat":"43.1306604","lng":"-6.9101806"},{"type":"House","id":409,"reference":"409","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"Castielfabib","postcode":"46141","form":1,"jobform":'null',"sector":'null',"lat":"40.1309055","lng":"-1.3053578"},{"type":"House","id":410,"reference":"450","community":11,"ccaa":"Galicia","province":"Lugo","town":"A FONSAGRADA","postcode":"27116","form":1,"jobform":'null',"sector":'null',"lat":"43.0994925","lng":"-7.1036396"},{"type":"House","id":412,"reference":"410","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"M\u00e1s de Jacinto (Castielfabib)","postcode":"46144","form":1,"jobform":'null',"sector":'null',"lat":"40.1289541","lng":"-1.2499989"},{"type":"House","id":413,"reference":"451","community":11,"ccaa":"Galicia","province":"Lugo","town":"A FONSAGRADA","postcode":"27100","form":1,"jobform":'null',"sector":'null',"lat":"43.12471","lng":"-7.0688574"},{"type":"House","id":414,"reference":"411","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"Los Santos (Castielfabib)","postcode":"46142","form":1,"jobform":'null',"sector":'null',"lat":"40.1087511","lng":"-1.2830851"},{"type":"House","id":417,"reference":"452","community":11,"ccaa":"Galicia","province":"Lugo","town":"A FONSAGRADA","postcode":"27113","form":1,"jobform":'null',"sector":'null',"lat":"43.1306604","lng":"-6.9101806"},{"type":"House","id":418,"reference":"412","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"Arroyo Cerezon (Castielfabib)","postcode":"46140","form":1,"jobform":'null',"sector":'null',"lat":"40.1215665","lng":"-1.4196142"},{"type":"House","id":419,"reference":"377","community":3,"ccaa":"Asturias","province":"Asturias","town":"Cardes, Infiesto","postcode":"33539","form":5,"jobform":'null',"sector":'null',"lat":"43.3471387","lng":"-5.3630233"},{"type":"House","id":420,"reference":"352","community":8,"ccaa":"Castilla y Leon","province":"Soria","town":"QUINTANA REDONDA","postcode":"42291","form":1,"jobform":'null',"sector":'null',"lat":"41.6401096","lng":"-2.6152059"},{"type":"House","id":422,"reference":"413","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"M\u00e1s del Olmo (Ademuz)","postcode":"46140","form":1,"jobform":'null',"sector":'null',"lat":"40.0682738","lng":"-1.1742997"},{"type":"House","id":424,"reference":"414","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"Cuesta del Rato (Castielfabib)","postcode":"46141","form":1,"jobform":'null',"sector":'null',"lat":"40.147474","lng":"-1.3113794"},{"type":"House","id":425,"reference":"353","community":10,"ccaa":"Extremadura","province":"Badajoz","town":"Monterrubio de la serena","postcode":"06427","form":1,"jobform":'null',"sector":'null',"lat":"38.588169","lng":"-5.4450222"},{"type":"House","id":428,"reference":"415","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"M\u00e1s de Jacinto (Castielfabib)","postcode":"46144","form":1,"jobform":'null',"sector":'null',"lat":"40.1289541","lng":"-1.2499989"},{"type":"House","id":430,"reference":"380","community":17,"ccaa":"Comunidad Valenciana","province":"Castellon","town":"J\u00e9rica","postcode":"12450","form":1,"jobform":'null',"sector":'null',"lat":"39.919063116182755","lng":"-0.5720122913002158"},{"type":"House","id":431,"reference":"416","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"Ademuz","postcode":"46140","form":1,"jobform":'null',"sector":'null',"lat":"40.0613379","lng":"-1.2870052"},{"type":"House","id":437,"reference":"403","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"M\u00e1s el Olmo (Ademuz)","postcode":"46140","form":1,"jobform":'null',"sector":'null',"lat":"40.0613379","lng":"-1.2870052"},{"type":"House","id":439,"reference":"406","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"Vallanca","postcode":"46145","form":2,"jobform":'null',"sector":'null',"lat":"40.0451459","lng":"-1.3565692"},{"type":"House","id":441,"reference":"463","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"Torrealta","postcode":"30509","form":5,"jobform":'null',"sector":'null',"lat":"38.03584260588592","lng":"-1.2107866187306298"},{"type":"House","id":444,"reference":"384","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"San Mart\u00edn del Camino","postcode":"24393","form":2,"jobform":'null',"sector":'null',"lat":"42.493976","lng":"-5.8096749"},{"type":"House","id":446,"reference":"421","community":2,"ccaa":"Aragon","province":"Zaragoza","town":"Codos","postcode":"50326","form":1,"jobform":'null',"sector":'null',"lat":"41.296887616019305","lng":"-1.374913918735477"},{"type":"House","id":447,"reference":"385","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"San Mart\u00edn del Camino","postcode":"24393","form":2,"jobform":'null',"sector":'null',"lat":"42.493976","lng":"-5.8096749"},{"type":"House","id":448,"reference":"465","community":2,"ccaa":"Aragon","province":"Teruel","town":"EL POBO","postcode":"44155","form":1,"jobform":'null',"sector":'null',"lat":"40.5076116","lng":"-0.8609602"},{"type":"House","id":449,"reference":"459","community":6,"ccaa":"Cantabria","province":"Cantabria","town":"SAN PEDRO DEL ROMERAL","postcode":"39686","form":1,"jobform":'null',"sector":'null',"lat":"43.1147857","lng":"-3.8180412"},{"type":"House","id":450,"reference":"422","community":8,"ccaa":"Castilla y Leon","province":"Avila","town":"Poyales del Hoyo","postcode":"05492","form":1,"jobform":'null',"sector":'null',"lat":"40.1728074","lng":"-5.166016"},{"type":"House","id":451,"reference":"359","community":10,"ccaa":"Extremadura","province":"Caceres","town":"Casas de Miravete","postcode":"10360","form":5,"jobform":'null',"sector":'null',"lat":"39.7270186","lng":"-5.7438055"},{"type":"House","id":455,"reference":"467","community":7,"ccaa":"Castilla La Mancha","province":"Albacete","town":"EL HORCAJO","postcode":"02314","form":1,"jobform":'null',"sector":'null',"lat":"38.720796","lng":"-2.4704602"},{"type":"House","id":456,"reference":"461","community":6,"ccaa":"Cantabria","province":"Cantabria","town":"CARRASCAL DE LUENA","postcode":"39687","form":1,"jobform":'null',"sector":'null',"lat":"43.0895895","lng":"-3.8568095"},{"type":"House","id":458,"reference":"468","community":7,"ccaa":"Castilla La Mancha","province":"Toledo","town":"LOS ALARES DE LOS MONTES","postcode":"45138","form":1,"jobform":'null',"sector":'null',"lat":"39.4853763","lng":"-4.7270069"},{"type":"House","id":460,"reference":"462","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"Ademuz","postcode":"46140","form":1,"jobform":'null',"sector":'null',"lat":"40.0613379","lng":"-1.2870052"},{"type":"House","id":462,"reference":"500","community":3,"ccaa":"Asturias","province":"Asturias","town":"Dega\u00f1a","postcode":"33812","form":1,"jobform":'null',"sector":'null',"lat":"42.9356216","lng":"-6.6103778"},{"type":"House","id":463,"reference":"469","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"San Justo de Cabanillas","postcode":"24319","form":1,"jobform":'null',"sector":'null',"lat":"42.6938356","lng":"-6.3658221"},{"type":"House","id":464,"reference":"426","community":8,"ccaa":"Castilla y Leon","province":"Avila","town":"Casas de la vega","postcode":"05692","form":1,"jobform":'null',"sector":'null',"lat":"40.3896679","lng":"-5.5495875"},{"type":"House","id":467,"reference":"427","community":2,"ccaa":"Aragon","province":"Teruel","town":"Tornos","postcode":"44230","form":1,"jobform":'null',"sector":'null',"lat":"40.96137850206979","lng":"-1.434418304714122"},{"type":"House","id":470,"reference":"516","community":11,"ccaa":"Galicia","province":"Lugo","town":"Samos","postcode":"27620","form":6,"jobform":'null',"sector":'null',"lat":"42.72977334527744","lng":"-7.328778073900665"},{"type":"House","id":471,"reference":"407","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"Vallanca","postcode":"46145","form":2,"jobform":'null',"sector":'null',"lat":"40.0451459","lng":"-1.3565692"},{"type":"House","id":472,"reference":"472","community":10,"ccaa":"Extremadura","province":"Badajoz","town":"Campanario","postcode":"06460","form":1,"jobform":'null',"sector":'null',"lat":"38.8753618","lng":"-5.5827334"},{"type":"House","id":473,"reference":"503","community":7,"ccaa":"Castilla La Mancha","province":"Guadalajara","town":"Pastrana","postcode":"19100","form":1,"jobform":'null',"sector":'null',"lat":"40.4265360189115","lng":"-2.9204882739289504"},{"type":"House","id":474,"reference":"517","community":8,"ccaa":"Castilla y Leon","province":"Soria","town":"Calata\u00f1azor","postcode":"42193","form":2,"jobform":'null',"sector":'null',"lat":"41.6991588","lng":"-2.8175177"},{"type":"House","id":475,"reference":"473","community":11,"ccaa":"Galicia","province":"Pontevedra","town":"Gondomar","postcode":"36316","form":1,"jobform":'null',"sector":'null',"lat":"42.1252599","lng":"-8.7315981"},{"type":"House","id":476,"reference":"518","community":6,"ccaa":"Cantabria","province":"Cantabria","town":"San Pedro del Romeral","postcode":"39686","form":1,"jobform":'null',"sector":'null',"lat":"43.1147857","lng":"-3.8180412"},{"type":"House","id":481,"reference":"430","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"Ademuz","postcode":"46140","form":1,"jobform":'null',"sector":'null',"lat":"40.0613379","lng":"-1.2870052"},{"type":"House","id":482,"reference":"520","community":11,"ccaa":"Galicia","province":"Lugo","town":"O Valadouro","postcode":"27770","form":1,"jobform":'null',"sector":'null',"lat":"43.5504705","lng":"-7.4411232"},{"type":"House","id":483,"reference":"476","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"MONZON DE CAMPOS","postcode":"34410","form":1,"jobform":'null',"sector":'null',"lat":"42.1159093","lng":"-4.4939766"},{"type":"House","id":484,"reference":"431","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"Ademuz","postcode":"46140","form":1,"jobform":'null',"sector":'null',"lat":"40.0613379","lng":"-1.2870052"},{"type":"House","id":485,"reference":"521","community":7,"ccaa":"Castilla La Mancha","province":"Cuenca","town":"La Alberca de Z\u00e1ncara","postcode":"16620","form":1,"jobform":'null',"sector":'null',"lat":"39.5141295","lng":"-2.4925685"},{"type":"House","id":486,"reference":"477","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"MONZON DE CAMPOS","postcode":"34410","form":1,"jobform":'null',"sector":'null',"lat":"42.1159093","lng":"-4.4939766"},{"type":"House","id":488,"reference":"478","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"MONZON DE CAMPOS","postcode":"34410","form":1,"jobform":'null',"sector":'null',"lat":"42.1159093","lng":"-4.4939766"},{"type":"House","id":489,"reference":"432","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"Sesga","postcode":"46140","form":5,"jobform":'null',"sector":'null',"lat":"40.0235467","lng":"-1.1926111"},{"type":"House","id":490,"reference":"479","community":11,"ccaa":"Galicia","province":"Ourense","town":"Vilarello","postcode":"32616","form":1,"jobform":'null',"sector":'null',"lat":"41.8577125","lng":"-7.348032"},{"type":"House","id":491,"reference":"523","community":7,"ccaa":"Castilla La Mancha","province":"Albacete","town":"Elche de la Sierra","postcode":"02430","form":1,"jobform":'null',"sector":'null',"lat":"38.4500743","lng":"-2.0481401"},{"type":"House","id":494,"reference":"481","community":11,"ccaa":"Galicia","province":"Ourense","town":"Viana do bolo","postcode":"32550","form":1,"jobform":'null',"sector":'null',"lat":"42.180337","lng":"-7.1130103"},{"type":"House","id":496,"reference":"526","community":13,"ccaa":"Murcia","province":"Murcia","town":"El Moralejo","postcode":"30412","form":2,"jobform":'null',"sector":'null',"lat":"37.9519535","lng":"-2.1309081"},{"type":"House","id":502,"reference":"505","community":1,"ccaa":"Andalucia","province":"Cordoba","town":"Priego de C\u00f3rdoba","postcode":"14800","form":1,"jobform":'null',"sector":'null',"lat":"37.437791","lng":"-4.1987434"},{"type":"House","id":503,"reference":"545","community":11,"ccaa":"Galicia","province":"Ourense","town":"A Rua de Valdeorras","postcode":"32350","form":1,"jobform":'null',"sector":'null',"lat":"42.3937437","lng":"-7.1197867"},{"type":"House","id":506,"reference":"546","community":11,"ccaa":"Galicia","province":"A Coru\u00f1a","town":"Nar\u00f3n","postcode":"15570","form":1,"jobform":'null',"sector":'null',"lat":"43.5012892","lng":"-8.1923743"},{"type":"House","id":507,"reference":"507","community":7,"ccaa":"Castilla La Mancha","province":"Ciudad Real","town":"Castellar de Santiago","postcode":"13750","form":1,"jobform":'null',"sector":'null',"lat":"38.537768","lng":"-3.2772247"},{"type":"House","id":509,"reference":"547","community":8,"ccaa":"Castilla y Leon","province":"Salamanca","town":"Villavede de Guare\u00f1a","postcode":"37428","form":2,"jobform":'null',"sector":'null',"lat":"41.0638739","lng":"-5.5256422"},{"type":"House","id":510,"reference":"508","community":7,"ccaa":"Castilla La Mancha","province":"Ciudad Real","town":"Castellar de Santiago","postcode":"13750","form":1,"jobform":'null',"sector":'null',"lat":"38.537768","lng":"-3.2772247"},{"type":"House","id":511,"reference":"487","community":11,"ccaa":"Galicia","province":"Ourense","town":"Vilardev\u00f3s","postcode":"32617","form":1,"jobform":'null',"sector":'null',"lat":"41.907314","lng":"-7.3125592"},{"type":"House","id":513,"reference":"509","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"Hospital de \u00d3rbigo","postcode":"24286","form":1,"jobform":'null',"sector":'null',"lat":"42.4627248","lng":"-5.8815485"},{"type":"House","id":515,"reference":"510","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"Laguna de Negrillos","postcode":"24234","form":1,"jobform":'null',"sector":'null',"lat":"42.2384488","lng":"-5.6602562"},{"type":"House","id":516,"reference":"488","community":11,"ccaa":"Galicia","province":"Ourense","town":"Vilardev\u00f3s","postcode":"32617","form":1,"jobform":'null',"sector":'null',"lat":"41.907314","lng":"-7.3125592"},{"type":"House","id":517,"reference":"550","community":6,"ccaa":"Cantabria","province":"Cantabria","town":"VEGA DE PAS","postcode":"39685","form":1,"jobform":'null',"sector":'null',"lat":"43.1577629","lng":"-3.7822445"},{"type":"House","id":518,"reference":"511","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"Villarejo de \u00d3rbigo","postcode":"24358","form":1,"jobform":'null',"sector":'null',"lat":"42.4450763","lng":"-5.9046545"},{"type":"House","id":519,"reference":"489","community":11,"ccaa":"Galicia","province":"Ourense","town":"Vilardev\u00f3s","postcode":"32617","form":1,"jobform":'null',"sector":'null',"lat":"41.907314","lng":"-7.3125592"},{"type":"House","id":520,"reference":"551","community":6,"ccaa":"Cantabria","province":"Cantabria","town":"VEGA DE PAS","postcode":"39685","form":4,"jobform":'null',"sector":'null',"lat":"43.1577629","lng":"-3.7822445"},{"type":"House","id":521,"reference":"490","community":11,"ccaa":"Galicia","province":"Ourense","town":"Vilardev\u00f3s","postcode":"32617","form":1,"jobform":'null',"sector":'null',"lat":"41.907314","lng":"-7.3125592"},{"type":"House","id":522,"reference":"552","community":6,"ccaa":"Cantabria","province":"Cantabria","town":"VEGA DE PAS","postcode":"39685","form":1,"jobform":'null',"sector":'null',"lat":"43.1577629","lng":"-3.7822445"},{"type":"House","id":523,"reference":"553","community":2,"ccaa":"Aragon","province":"Teruel","town":"Cabra de Mora","postcode":"44409","form":1,"jobform":'null',"sector":'null',"lat":"40.3172204","lng":"-0.8076798"},{"type":"House","id":524,"reference":"554","community":2,"ccaa":"Aragon","province":"Teruel","town":"Cabra de Mora","postcode":"44409","form":2,"jobform":'null',"sector":'null',"lat":"40.3172204","lng":"-0.8076798"},{"type":"House","id":525,"reference":"555","community":2,"ccaa":"Aragon","province":"Teruel","town":"Cabra de Mora","postcode":"44409","form":2,"jobform":'null',"sector":'null',"lat":"40.3172204","lng":"-0.8076798"},{"type":"House","id":527,"reference":"491","community":11,"ccaa":"Galicia","province":"Ourense","town":"Vilardev\u00f3s","postcode":"32617","form":1,"jobform":'null',"sector":'null',"lat":"41.907314","lng":"-7.3125592"},{"type":"House","id":528,"reference":"492","community":11,"ccaa":"Galicia","province":"Ourense","town":"Vilardev\u00f3s","postcode":"32617","form":1,"jobform":'null',"sector":'null',"lat":"41.907314","lng":"-7.3125592"},{"type":"House","id":529,"reference":"493","community":11,"ccaa":"Galicia","province":"Ourense","town":"Cualedro","postcode":"32689","form":1,"jobform":'null',"sector":'null',"lat":"41.9777343","lng":"-7.5572231"},{"type":"House","id":530,"reference":"557","community":8,"ccaa":"Castilla y Leon","province":"Burgos","town":"Castrojer\u00edz","postcode":"09110","form":1,"jobform":'null',"sector":'null',"lat":"42.2884823","lng":"-4.1424199"},{"type":"House","id":532,"reference":"559","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"Castielfabib","postcode":"46141","form":1,"jobform":'null',"sector":'null',"lat":"40.1309055","lng":"-1.3053578"},{"type":"House","id":533,"reference":"560","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"Castielfabib","postcode":"46141","form":1,"jobform":'null',"sector":'null',"lat":"40.1309055","lng":"-1.3053578"},{"type":"House","id":534,"reference":"494","community":11,"ccaa":"Galicia","province":"Ourense","town":"Laza","postcode":"32620","form":1,"jobform":'null',"sector":'null',"lat":"42.105515715386055","lng":"-7.492089673093952"},{"type":"House","id":535,"reference":"495","community":11,"ccaa":"Galicia","province":"Ourense","town":"Laza","postcode":"32620","form":1,"jobform":'null',"sector":'null',"lat":"42.09616293271941","lng":"-7.492089673093952"},{"type":"House","id":536,"reference":"561","community":3,"ccaa":"Asturias","province":"Asturias","town":"Caldevilla, localidad Sevares . Municipio Pilo\u00f1a","postcode":"33584","form":1,"jobform":'null',"sector":'null',"lat":"43.3538742","lng":"-5.2398825"},{"type":"House","id":538,"reference":"562","community":3,"ccaa":"Asturias","province":"Asturias","town":"La Marea. Pilo\u00f1a","postcode":"33536","form":1,"jobform":'null',"sector":'null',"lat":"43.2746671","lng":"-5.4203005"},{"type":"House","id":539,"reference":"563","community":3,"ccaa":"Asturias","province":"Asturias","town":"Villamayor. Pilo\u00f1a","postcode":"33583","form":1,"jobform":'null',"sector":'null',"lat":"43.3603187","lng":"-5.3056764"},{"type":"House","id":541,"reference":"564","community":8,"ccaa":"Castilla y Leon","province":"Avila","town":"El Barco de \u00c1vila","postcode":"05600","form":1,"jobform":'null',"sector":'null',"lat":"40.3569063","lng":"-5.5236746"},{"type":"House","id":545,"reference":"512","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"Puente de \u00d3rbigo","postcode":"24286","form":5,"jobform":'null',"sector":'null',"lat":"42.4636342","lng":"-5.8769366"},{"type":"House","id":546,"reference":"513","community":11,"ccaa":"Galicia","province":"Ourense","town":"Bande","postcode":"32840","form":2,"jobform":'null',"sector":'null',"lat":"42.0420367356602","lng":"-7.971534457303168"},{"type":"House","id":547,"reference":"514","community":11,"ccaa":"Galicia","province":"Lugo","town":"Samos","postcode":"27620","form":5,"jobform":'null',"sector":'null',"lat":"42.72975569386634","lng":"-7.329426879578075"},{"type":"House","id":548,"reference":"515","community":11,"ccaa":"Galicia","province":"Lugo","town":"Samos","postcode":"27620","form":6,"jobform":'null',"sector":'null',"lat":"42.72981149619656","lng":"-7.328459127835643"},{"type":"House","id":549,"reference":"527","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"HOSPITAL DE \u00d3rbigo","postcode":"24286","form":1,"jobform":'null',"sector":'null',"lat":"42.4627248","lng":"-5.8815485"},{"type":"House","id":552,"reference":"530","community":6,"ccaa":"Cantabria","province":"Cantabria","town":"B\u00ba VIA\u00d1A","postcode":"39685","form":1,"jobform":'null',"sector":'null',"lat":"43.144444","lng":"-3.795833"},{"type":"House","id":559,"reference":"537","community":8,"ccaa":"Castilla y Leon","province":"Valladolid","town":"Villafrechos","postcode":"47810","form":1,"jobform":'null',"sector":'null',"lat":"41.8918449","lng":"-5.2175331"},{"type":"House","id":560,"reference":"538","community":8,"ccaa":"Castilla y Leon","province":"Valladolid","town":"Aguilar de Campos","postcode":"47814","form":1,"jobform":'null',"sector":'null',"lat":"41.985012","lng":"-5.1810098"},{"type":"House","id":561,"reference":"539","community":8,"ccaa":"Castilla y Leon","province":"Valladolid","town":"Aguilar de Campos","postcode":"47814","form":1,"jobform":'null',"sector":'null',"lat":"41.985012","lng":"-5.1810098"},{"type":"House","id":562,"reference":"540","community":8,"ccaa":"Castilla y Leon","province":"Valladolid","town":"Aguilar de Campos","postcode":"47814","form":1,"jobform":'null',"sector":'null',"lat":"41.985012","lng":"-5.1810098"},{"type":"House","id":564,"reference":"542","community":8,"ccaa":"Castilla y Leon","province":"Valladolid","town":"Aguilar de Campos","postcode":"47814","form":1,"jobform":'null',"sector":'null',"lat":"41.985012","lng":"-5.1810098"},{"type":"House","id":565,"reference":"543","community":8,"ccaa":"Castilla y Leon","province":"Valladolid","town":"Villanueva de los Caballeros","postcode":"47850","form":1,"jobform":'null',"sector":'null',"lat":"41.7590608","lng":"-5.2488573"},{"type":"House","id":569,"reference":"VCL20220001","community":8,"ccaa":"Castilla y Leon","province":"Avila","town":"El Barco de Avila","postcode":"05600","form":1,"jobform":'null',"sector":'null',"lat":"40.3569063","lng":"-5.5236746"},{"type":"House","id":571,"reference":"VCL20220003","community":8,"ccaa":"Castilla y Leon","province":"Avila","town":"El Barco de Avila","postcode":"05600","form":1,"jobform":'null',"sector":'null',"lat":"40.3569063","lng":"-5.5236746"},{"type":"House","id":573,"reference":"VGA20220001","community":11,"ccaa":"Galicia","province":"Ourense","town":"Ver\u00edn","postcode":"32600","form":1,"jobform":'null',"sector":'null',"lat":"41.9316334","lng":"-7.4327273"},{"type":"House","id":574,"reference":"VVC20220002","community":17,"ccaa":"Comunidad Valenciana","province":"Castellon","town":"Cat\u00ed","postcode":"12513","form":1,"jobform":'null',"sector":'null',"lat":"40.47739242163002","lng":"0.023256312038585143"},{"type":"House","id":575,"reference":"VCL20220004","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Itero de la Vega","postcode":"34468","form":1,"jobform":'null',"sector":'null',"lat":"42.2873193","lng":"-4.2583327"},{"type":"House","id":576,"reference":"VCL20220005","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Fr\u00f3mista","postcode":"34440","form":1,"jobform":'null',"sector":'null',"lat":"42.2601538","lng":"-4.4079213"},{"type":"House","id":578,"reference":"VCL20220007","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Cervatos de la Cueza","postcode":"34309","form":1,"jobform":'null',"sector":'null',"lat":"42.2905776","lng":"-4.7698058"},{"type":"House","id":580,"reference":"VCL20220009","community":8,"ccaa":"Castilla y Leon","province":"Avila","town":"Gil Garc\u00eda","postcode":"05693","form":1,"jobform":'null',"sector":'null',"lat":"40.2978672","lng":"-5.594224"},{"type":"House","id":582,"reference":"VCL20220011","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Carri\u00f3n de los Condes","postcode":"34120","form":1,"jobform":'null',"sector":'null',"lat":"42.3307302","lng":"-4.6140481"},{"type":"House","id":583,"reference":"VCL20220012","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Villasarracino","postcode":"34132","form":1,"jobform":'null',"sector":'null',"lat":"42.412712","lng":"-4.4967168"},{"type":"House","id":584,"reference":"VCL20220013","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Fr\u00f3mista","postcode":"34440","form":1,"jobform":'null',"sector":'null',"lat":"42.2676329","lng":"-4.4053497"},{"type":"House","id":585,"reference":"VCL20220014","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Carri\u00f3n de los Condes","postcode":"34120","form":1,"jobform":'null',"sector":'null',"lat":"42.3307302","lng":"-4.6140481"},{"type":"House","id":586,"reference":"VCL20220015","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Perales","postcode":"34133","form":1,"jobform":'null',"sector":'null',"lat":"41.7848774","lng":"-4.4284985"},{"type":"House","id":591,"reference":"VCL20220018","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"Posadilla de la Vega","postcode":"24795","form":1,"jobform":'null',"sector":'null',"lat":"42.4013954","lng":"-5.9442736"},{"type":"House","id":592,"reference":"VCL20220019","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Villarmentero de Campos","postcode":"34447","form":1,"jobform":'null',"sector":'null',"lat":"42.2977807","lng":"-4.4999368"},{"type":"House","id":593,"reference":"VCL20220020","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"B\u00e1rcena de Campos","postcode":"34477","form":1,"jobform":'null',"sector":'null',"lat":"42.4843389","lng":"-4.4986488"},{"type":"House","id":594,"reference":"VCL20220021","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Carri\u00f3n de los Condes","postcode":"34120","form":1,"jobform":'null',"sector":'null',"lat":"42.3307302","lng":"-4.6140481"},{"type":"House","id":597,"reference":"VCB20220001","community":6,"ccaa":"Cantabria","province":"Cantabria","town":"San Pedro de Rudag\u00fcera","postcode":"39539","form":1,"jobform":'null',"sector":'null',"lat":"43.3611261","lng":"-4.1495121"},{"type":"House","id":598,"reference":"VCM20220001","community":7,"ccaa":"Castilla La Mancha","province":"Guadalajara","town":"Riba de Saelices","postcode":"19441","form":1,"jobform":'null',"sector":'null',"lat":"40.9119227","lng":"-2.2962865"},{"type":"House","id":600,"reference":"VCT20220001","community":9,"ccaa":"Catalu\u00f1a","province":"Lleida","town":"Rocallaura","postcode":"25269","form":1,"jobform":'null',"sector":'null',"lat":"41.5071295","lng":"1.1481972"},{"type":"House","id":601,"reference":"VVC20220003","community":17,"ccaa":"Comunidad Valenciana","province":"Castellon","town":"El Toro","postcode":"12429","form":2,"jobform":'null',"sector":'null',"lat":"39.9820316","lng":"-0.7491223"},{"type":"House","id":602,"reference":"VVC20220004","community":17,"ccaa":"Comunidad Valenciana","province":"Castellon","town":"Bej\u00eds","postcode":"12430","form":2,"jobform":'null',"sector":'null',"lat":"39.9098511","lng":"-0.7069702"},{"type":"House","id":603,"reference":"VVC20220005","community":17,"ccaa":"Comunidad Valenciana","province":"Castellon","town":"Viver","postcode":"12460","form":2,"jobform":'null',"sector":'null',"lat":"39.9132074","lng":"-0.6095632"},{"type":"House","id":604,"reference":"VVC20220006","community":17,"ccaa":"Comunidad Valenciana","province":"Castellon","town":"Navajas","postcode":"12470","form":2,"jobform":'null',"sector":'null',"lat":"39.8767397","lng":"-0.506292"},{"type":"House","id":605,"reference":"VVC20220007","community":17,"ccaa":"Comunidad Valenciana","province":"Castellon","town":"Navajas","postcode":"12470","form":2,"jobform":'null',"sector":'null',"lat":"39.8767397","lng":"-0.506292"},{"type":"House","id":606,"reference":"VVC20220008","community":17,"ccaa":"Comunidad Valenciana","province":"Castellon","town":"El Toro","postcode":"12429","form":2,"jobform":'null',"sector":'null',"lat":"39.9820316","lng":"-0.7491223"},{"type":"House","id":607,"reference":"VVC20220009","community":17,"ccaa":"Comunidad Valenciana","province":"Castellon","town":"Tor\u00e1s","postcode":"12431","form":2,"jobform":'null',"sector":'null',"lat":"39.9195161","lng":"-0.6850145"},{"type":"House","id":608,"reference":"VCB20220002","community":6,"ccaa":"Cantabria","province":"Cantabria","town":"B\u00ba Candolias","postcode":"39685","form":1,"jobform":'null',"sector":'null',"lat":"43.1423699","lng":"-3.7536663"},{"type":"House","id":609,"reference":"VCB20220003","community":6,"ccaa":"Cantabria","province":"Cantabria","town":"B\u00ba Candolias","postcode":"39685","form":1,"jobform":'null',"sector":'null',"lat":"43.1423699","lng":"-3.7536663"},{"type":"House","id":610,"reference":"VCB20220004","community":6,"ccaa":"Cantabria","province":"Cantabria","town":"B\u00ba Pandillo","postcode":"39685","form":1,"jobform":'null',"sector":'null',"lat":"43.147778","lng":"-3.720556"},{"type":"House","id":611,"reference":"VCB20220005","community":6,"ccaa":"Cantabria","province":"Cantabria","town":"B\u00ba Pandillo","postcode":"39685","form":1,"jobform":'null',"sector":'null',"lat":"43.147778","lng":"-3.720556"},{"type":"House","id":612,"reference":"VCB20220006","community":6,"ccaa":"Cantabria","province":"Cantabria","town":"B\u00ba Merilla","postcode":"39728","form":1,"jobform":'null',"sector":'null',"lat":"43.2521882","lng":"-3.7311325"},{"type":"House","id":613,"reference":"VCB20220007","community":6,"ccaa":"Cantabria","province":"Cantabria","town":"Ore\u00f1a","postcode":"39525","form":1,"jobform":'null',"sector":'null',"lat":"43.399569141879226","lng":"-4.1356507864508805"},{"type":"House","id":614,"reference":"VCB20220008","community":6,"ccaa":"Cantabria","province":"Cantabria","town":"Trece\u00f1o-B\u00ba Hualle),","postcode":"39592","form":1,"jobform":'null',"sector":'null',"lat":"43.317679","lng":"-4.2879888"},{"type":"House","id":615,"reference":"VAS20220004","community":3,"ccaa":"Asturias","province":"Asturias","town":"Lig\u00fceria","postcode":"33537","form":6,"jobform":'null',"sector":'null',"lat":"43.27356330712334","lng":"-5.365393696384307"},{"type":"House","id":616,"reference":"VAS20220005","community":3,"ccaa":"Asturias","province":"Asturias","town":"Cuerries De Espinareu","postcode":"33537","form":3,"jobform":'null',"sector":'null',"lat":"43.2908509","lng":"-5.3625516"},{"type":"House","id":617,"reference":"VAS20220006","community":3,"ccaa":"Asturias","province":"Asturias","town":"Rozapanera","postcode":"33537","form":1,"jobform":'null',"sector":'null',"lat":"43.28670842597821","lng":"-5.37538193078534"},{"type":"House","id":618,"reference":"VAS20220007","community":3,"ccaa":"Asturias","province":"Asturias","town":"Qu\u00e9s","postcode":"33539","form":2,"jobform":'null',"sector":'null',"lat":"43.3432265","lng":"-5.3871696"},{"type":"House","id":619,"reference":"VCL20220023","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"L\u00e9digos","postcode":"34347","form":1,"jobform":'null',"sector":'null',"lat":"42.3544583","lng":"-4.8647046"},{"type":"House","id":620,"reference":"VCL20220024","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Cervatos de la Cueza","postcode":"34309","form":1,"jobform":'null',"sector":'null',"lat":"42.2905776","lng":"-4.7698058"},{"type":"House","id":622,"reference":"VCL20220026","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Carri\u00f3n de los Condes","postcode":"34120","form":1,"jobform":'null',"sector":'null',"lat":"42.3307302","lng":"-4.6140481"},{"type":"House","id":623,"reference":"VCL20220027","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Carri\u00f3n de los Condes","postcode":"34120","form":1,"jobform":'null',"sector":'null',"lat":"42.3307302","lng":"-4.6140481"},{"type":"House","id":624,"reference":"VCL20220028","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Carri\u00f3n de los Condes","postcode":"34120","form":1,"jobform":'null',"sector":'null',"lat":"42.3307302","lng":"-4.6140481"},{"type":"House","id":625,"reference":"VCL20220029","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Carri\u00f3n de los Condes","postcode":"34120","form":6,"jobform":'null',"sector":'null',"lat":"42.3307302","lng":"-4.6140481"},{"type":"House","id":626,"reference":"VCL20220030","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Carri\u00f3n de los Condes","postcode":"34120","form":1,"jobform":'null',"sector":'null',"lat":"42.3307302","lng":"-4.6140481"},{"type":"House","id":627,"reference":"VCL20220031","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Osornillo","postcode":"34468","form":1,"jobform":'null',"sector":'null',"lat":"42.3041693","lng":"-4.2846246"},{"type":"House","id":628,"reference":"VCL20220032","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Carri\u00f3n de los Condes","postcode":"34120","form":2,"jobform":'null',"sector":'null',"lat":"42.3307302","lng":"-4.6140481"},{"type":"House","id":629,"reference":"VCL20220033","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Villoldo","postcode":"34131","form":1,"jobform":'null',"sector":'null',"lat":"42.2460438","lng":"-4.5953355"},{"type":"House","id":630,"reference":"VGA20220002","community":11,"ccaa":"Galicia","province":"Ourense","town":"Ver\u00edn","postcode":"32600","form":1,"jobform":'null',"sector":'null',"lat":"41.9316334","lng":"-7.4327273"},{"type":"House","id":631,"reference":"VGA20220003","community":11,"ccaa":"Galicia","province":"Ourense","town":"Vilardecervos","postcode":"32616","form":1,"jobform":'null',"sector":'null',"lat":"41.9421627","lng":"-6.2867284"},{"type":"House","id":632,"reference":"VGA20220004","community":11,"ccaa":"Galicia","province":"Ourense","town":"Vilardecervos","postcode":"32616","form":1,"jobform":'null',"sector":'null',"lat":"41.9421627","lng":"-6.2867284"},{"type":"House","id":633,"reference":"VGA20220005","community":11,"ccaa":"Galicia","province":"Ourense","town":"Vilardecervos","postcode":"32616","form":1,"jobform":'null',"sector":'null',"lat":"41.9421627","lng":"-6.2867284"},{"type":"House","id":634,"reference":"VGA20220006","community":11,"ccaa":"Galicia","province":"Ourense","town":"Vilardecervos","postcode":"32616","form":1,"jobform":'null',"sector":'null',"lat":"41.9421627","lng":"-6.2867284"},{"type":"House","id":635,"reference":"VGA20220007","community":11,"ccaa":"Galicia","province":"Ourense","town":"Vilardecervos","postcode":"32616","form":1,"jobform":'null',"sector":'null',"lat":"41.9421627","lng":"-6.2867284"},{"type":"House","id":636,"reference":"VGA20220008","community":11,"ccaa":"Galicia","province":"Ourense","town":"Vilarello da Cota","postcode":"32616","form":1,"jobform":'null',"sector":'null',"lat":"41.8761326","lng":"-7.2976617"},{"type":"House","id":637,"reference":"VGA20220009","community":11,"ccaa":"Galicia","province":"Ourense","town":"Cerdedelo,","postcode":"32621","form":1,"jobform":'null',"sector":'null',"lat":"42.5327602","lng":"-8.3903276"},{"type":"House","id":639,"reference":"VGA20220011","community":11,"ccaa":"Galicia","province":"Ourense","town":"Vilardev\u00f3s","postcode":"32616","form":6,"jobform":'null',"sector":'null',"lat":"41.907314","lng":"-7.3125592"},{"type":"House","id":640,"reference":"VGA20220012","community":11,"ccaa":"Galicia","province":"Ourense","town":"Ri\u00f3s","postcode":"32611","form":6,"jobform":'null',"sector":'null',"lat":"42.07490946393204","lng":"-7.314598463512616"},{"type":"House","id":641,"reference":"VCL20220034","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Monz\u00f3n de Campos","postcode":"34410","form":1,"jobform":'null',"sector":'null',"lat":"42.1159093","lng":"-4.4939766"},{"type":"House","id":643,"reference":"VCL20220036","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"San Cebri\u00e1n de Campos","postcode":"34429","form":3,"jobform":'null',"sector":'null',"lat":"42.2018325","lng":"-4.5308585"},{"type":"House","id":644,"reference":"VCL20220037","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Santoyo","postcode":"34490","form":1,"jobform":'null',"sector":'null',"lat":"42.22155833021928","lng":"-4.343426200892219"},{"type":"House","id":651,"reference":"VCL20220041","community":8,"ccaa":"Castilla y Leon","province":"Avila","town":"Nava del Barco","postcode":"05697","form":1,"jobform":'null',"sector":'null',"lat":"40.2928502","lng":"-5.5397458"},{"type":"House","id":652,"reference":"VCL20220042","community":8,"ccaa":"Castilla y Leon","province":"Avila","town":"El Barco de \u00c1vila","postcode":"05600","form":2,"jobform":'null',"sector":'null',"lat":"40.3569063","lng":"-5.5236746"},{"type":"House","id":653,"reference":"VCL20220043","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Carri\u00f3n de los Condes","postcode":"34120","form":1,"jobform":'null',"sector":'null',"lat":"42.3307302","lng":"-4.6140481"},{"type":"House","id":654,"reference":"VCL20220044","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Carri\u00f3n de los Condes","postcode":"34120","form":6,"jobform":'null',"sector":'null',"lat":"42.3307302","lng":"-4.6140481"},{"type":"House","id":655,"reference":"VCL20220045","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Carri\u00f3n de los Condes","postcode":"34120","form":6,"jobform":'null',"sector":'null',"lat":"42.3307302","lng":"-4.6140481"},{"type":"House","id":657,"reference":"VCL20220047","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Monz\u00f3n de Campos","postcode":"34410","form":2,"jobform":'null',"sector":'null',"lat":"42.1159093","lng":"-4.4939766"},{"type":"House","id":658,"reference":"VAS20220008","community":3,"ccaa":"Asturias","province":"Asturias","town":"San Tirso de Abres","postcode":"33774","form":1,"jobform":'null',"sector":'null',"lat":"43.4102135","lng":"-7.1422864"},{"type":"House","id":662,"reference":"VCL20220048","community":8,"ccaa":"Castilla y Leon","province":"Avila","town":"Umbr\u00edas","postcode":"05693","form":2,"jobform":'null',"sector":'null',"lat":"40.3126247","lng":"-5.5784661"},{"type":"House","id":663,"reference":"VCL20220049","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Villaproviano","postcode":"34128","form":1,"jobform":'null',"sector":'null',"lat":"42.4272353","lng":"-4.6553679"},{"type":"House","id":664,"reference":"VCL20220050","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Itero Seco","postcode":"34477","form":1,"jobform":'null',"sector":'null',"lat":"42.4612253","lng":"-4.55664"},{"type":"House","id":665,"reference":"VCL20220051","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Abia de las Torres","postcode":"34479","form":1,"jobform":'null',"sector":'null',"lat":"42.4200011","lng":"-4.4210977"},{"type":"House","id":666,"reference":"VCL20220052","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Calzadilla de la Cueza","postcode":"34309","form":1,"jobform":'null',"sector":'null',"lat":"42.3287562","lng":"-4.8044427"},{"type":"House","id":667,"reference":"VCL20220053","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"Benavides de \u00d3rbigo","postcode":"24280","form":2,"jobform":'null',"sector":'null',"lat":"42.5117776","lng":"-5.9039944"},{"type":"House","id":669,"reference":"VCL20220054","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Terradillos de los Templarios","postcode":"34349","form":1,"jobform":'null',"sector":'null',"lat":"42.362882","lng":"-4.8903195"},{"type":"House","id":670,"reference":"VAS20220009","community":3,"ccaa":"Asturias","province":"Asturias","town":"Vega de Perv\u00eds","postcode":"33558","form":1,"jobform":'null',"sector":'null',"lat":"43.2839386","lng":"-5.1355516"},{"type":"House","id":672,"reference":"VAS20220010","community":3,"ccaa":"Asturias","province":"Asturias","town":"Trasdacorda","postcode":"33774","form":6,"jobform":'null',"sector":'null',"lat":"43.4047947","lng":"-7.1467072"},{"type":"House","id":673,"reference":"VAN20220001","community":1,"ccaa":"Andalucia","province":"Huelva","town":"Arroyomolinos de Le\u00f3n","postcode":"21280","form":1,"jobform":'null',"sector":'null',"lat":"38.0240295","lng":"-6.4216722"},{"type":"House","id":675,"reference":"VVC20220016","community":17,"ccaa":"Comunidad Valenciana","province":"Castellon","town":"Cat\u00ed","postcode":"12513","form":1,"jobform":'null',"sector":'null',"lat":"40.47512877713052","lng":"0.022755615197926963"},{"type":"House","id":676,"reference":"VCL20220056","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"La Puebla de Valdavia","postcode":"34470","form":2,"jobform":'null',"sector":'null',"lat":"42.6755457","lng":"-4.6095307"},{"type":"House","id":677,"reference":"VCT20220002","community":9,"ccaa":"Catalu\u00f1a","province":"Tarragona","town":"Corbera d\u00b4Ebre","postcode":"43784","form":1,"jobform":'null',"sector":'null',"lat":"41.0768503","lng":"0.4755993"},{"type":"House","id":679,"reference":"VCT20220004","community":9,"ccaa":"Catalu\u00f1a","province":"Lleida","town":"Vallbona de les Monges","postcode":"25268","form":2,"jobform":'null',"sector":'null',"lat":"41.5251965","lng":"1.0888526"},{"type":"House","id":681,"reference":"VCL20220057","community":8,"ccaa":"Castilla y Leon","province":"Avila","town":"La Aldehuela","postcode":"05593","form":1,"jobform":'null',"sector":'null',"lat":"40.4109199","lng":"-5.409336"},{"type":"House","id":684,"reference":"VGA20220015","community":11,"ccaa":"Galicia","province":"Ourense","town":"Vences","postcode":"32624","form":1,"jobform":'null',"sector":'null',"lat":"41.993827785785875","lng":"-7.459947999107896"},{"type":"House","id":686,"reference":"VGA20220016","community":11,"ccaa":"Galicia","province":"Ourense","town":"Laro\u00e1","postcode":"32632","form":1,"jobform":'null',"sector":'null',"lat":"42.01187588908001","lng":"-7.700017124885502"},{"type":"House","id":687,"reference":"VGA20220017","community":11,"ccaa":"Galicia","province":"Ourense","town":"Laro\u00e1","postcode":"32632","form":1,"jobform":'null',"sector":'null',"lat":"42.01073981416066","lng":"-7.69899778394066"},{"type":"House","id":688,"reference":"VGA20220018","community":11,"ccaa":"Galicia","province":"Ourense","town":"Morgade","postcode":"32636","form":1,"jobform":'null',"sector":'null',"lat":"42.1083532","lng":"-7.6656625"},{"type":"House","id":689,"reference":"VGA20220019","community":11,"ccaa":"Galicia","province":"Ourense","town":"Vences","postcode":"32624","form":1,"jobform":'null',"sector":'null',"lat":"41.99289942913127","lng":"-7.460329270608883"},{"type":"House","id":690,"reference":"VGA20220020","community":11,"ccaa":"Galicia","province":"Ourense","town":"Arz\u00e1degos","postcode":"32616","form":6,"jobform":'null',"sector":'null',"lat":"41.8761326","lng":"-7.2976617"},{"type":"House","id":691,"reference":"VGA20220021","community":11,"ccaa":"Galicia","province":"Ourense","town":"Moimenta","postcode":"32617","form":1,"jobform":'null',"sector":'null',"lat":"41.9028487","lng":"-7.3384908"},{"type":"House","id":692,"reference":"VGA20220022","community":11,"ccaa":"Galicia","province":"Ourense","town":"A Trave","postcode":"32617","form":6,"jobform":'null',"sector":'null',"lat":"41.9128794","lng":"-7.3408535"},{"type":"House","id":693,"reference":"VAS20220011","community":3,"ccaa":"Asturias","province":"Asturias","town":"La Noval\/La Marea","postcode":"33536","form":1,"jobform":'null',"sector":'null',"lat":"43.27529112553528","lng":"-5.420343447433906"},{"type":"House","id":696,"reference":"VGA20220025","community":11,"ccaa":"Galicia","province":"Ourense","town":"Morgade","postcode":"32636","form":6,"jobform":'null',"sector":'null',"lat":"42.1083532","lng":"-7.6656625"},{"type":"House","id":697,"reference":"VCL20220059","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Carri\u00f3n de los Condes","postcode":"34120","form":1,"jobform":'null',"sector":'null',"lat":"42.3307302","lng":"-4.6140481"},{"type":"House","id":698,"reference":"VCL20220060","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Carri\u00f3n de los Condes","postcode":"34120","form":2,"jobform":'null',"sector":'null',"lat":"42.3307302","lng":"-4.6140481"},{"type":"House","id":699,"reference":"VCL20220061","community":8,"ccaa":"Castilla y Leon","province":"Zamora","town":"Villalpando","postcode":"49630","form":2,"jobform":'null',"sector":'null',"lat":"41.8654475","lng":"-5.413591"},{"type":"House","id":702,"reference":"VGA20220026","community":11,"ccaa":"Galicia","province":"Ourense","town":"Ordes","postcode":"32654","form":2,"jobform":'null',"sector":'null',"lat":"42.0557467","lng":"-7.8612767"},{"type":"House","id":706,"reference":"VGA20220027","community":11,"ccaa":"Galicia","province":"Ourense","town":"Vilar de Barrio","postcode":"32702","form":6,"jobform":'null',"sector":'null',"lat":"42.16634582686376","lng":"-7.6145952682284985"},{"type":"House","id":707,"reference":"VGA20220028","community":11,"ccaa":"Galicia","province":"Ourense","town":"Vilar de Barrio","postcode":"32702","form":6,"jobform":'null',"sector":'null',"lat":"42.16380107084287","lng":"-7.613565300037207"},{"type":"House","id":708,"reference":"VGA20220029","community":11,"ccaa":"Galicia","province":"Ourense","town":"TOXEDI\u00d1O, VILAR DE SANTOS","postcode":"32650","form":1,"jobform":'null',"sector":'null',"lat":"42.12044418107381","lng":"-7.81411430732223"},{"type":"House","id":709,"reference":"VGA20220030","community":11,"ccaa":"Galicia","province":"Ourense","town":"San Pedro","postcode":"32632","form":6,"jobform":'null',"sector":'null',"lat":"42.032203642535954","lng":"-7.697587002509327"},{"type":"House","id":710,"reference":"VCL20220063","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"SESAMO","postcode":"24434","form":2,"jobform":'null',"sector":'null',"lat":"42.74400895150144","lng":"-6.65502597512858"},{"type":"House","id":711,"reference":"VCL20220064","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"VEGA DE ESPINAREDA","postcode":"24430","form":2,"jobform":'null',"sector":'null',"lat":"42.73138621524078","lng":"-6.654002961512407"},{"type":"House","id":712,"reference":"VCB20220010","community":6,"ccaa":"Cantabria","province":"Cantabria","town":"SAN ROQUE DE RIOMIERA","postcode":"39728","form":1,"jobform":'null',"sector":'null',"lat":"43.236756813988805","lng":"-3.702328071121999"},{"type":"House","id":714,"reference":"VGA20220032","community":11,"ccaa":"Galicia","province":"Ourense","town":"PI\u00d1EIRA SECA","postcode":"32631","form":1,"jobform":'null',"sector":'null',"lat":"42.09448698211465","lng":"-7.630882483644151"},{"type":"House","id":715,"reference":"VGA20220033","community":11,"ccaa":"Galicia","province":"Ourense","town":"CALVOS DE RAND\u00cdN","postcode":"32646","form":2,"jobform":'null',"sector":'null',"lat":"41.912298095427495","lng":"-7.875005279109895"},{"type":"House","id":716,"reference":"VCB20220011","community":6,"ccaa":"Cantabria","province":"Cantabria","town":"ORE\u00d1A","postcode":"39525","form":2,"jobform":'null',"sector":'null',"lat":"43.400081857427324","lng":"-4.134109438709537"},{"type":"House","id":717,"reference":"VCB20220012","community":6,"ccaa":"Cantabria","province":"Cantabria","town":"VEGA DE PAS","postcode":"39685","form":1,"jobform":'null',"sector":'null',"lat":"43.164895442278905","lng":"-3.780616270544252"},{"type":"House","id":718,"reference":"VCB20220013","community":6,"ccaa":"Cantabria","province":"Cantabria","town":"ORE\u00d1A","postcode":"39525","form":1,"jobform":'null',"sector":'null',"lat":"43.39908405211089","lng":"-4.134109438709537"},{"type":"House","id":719,"reference":"VVC20220017","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"ALMOINES","postcode":"46723","form":1,"jobform":'null',"sector":'null',"lat":"38.9548679991166","lng":"-0.1819196573664946"},{"type":"House","id":720,"reference":"VVC20220018","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"BELLREGUARD","postcode":"46713","form":1,"jobform":'null',"sector":'null',"lat":"38.95386802153561","lng":"-0.16096357552239277"},{"type":"House","id":721,"reference":"VVC20220019","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"POTRIES","postcode":"46721","form":1,"jobform":'null',"sector":'null',"lat":"38.91815639417632","lng":"-0.19516758461238887"},{"type":"House","id":722,"reference":"VVC20220020","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"BELLREGUARD","postcode":"46713","form":1,"jobform":'null',"sector":'null',"lat":"38.95947033621853","lng":"-0.1641977880766989"},{"type":"House","id":723,"reference":"VVC20220021","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"BELLREGUARD","postcode":"46713","form":1,"jobform":'null',"sector":'null',"lat":"38.96053818679343","lng":"-0.16007791531153034"},{"type":"House","id":724,"reference":"VVC20220022","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"BELLREGUARD","postcode":"46713","form":1,"jobform":'null',"sector":'null',"lat":"38.96053818679343","lng":"-0.16007791531153034"},{"type":"House","id":725,"reference":"VVC20220023","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"R\u00d2TOVA","postcode":"46725","form":1,"jobform":'null',"sector":'null',"lat":"38.93661307637198","lng":"-0.25758433463716063"},{"type":"House","id":726,"reference":"VGA20220034","community":11,"ccaa":"Galicia","province":"Ourense","town":"OS BLANCOS","postcode":"32634","form":6,"jobform":'null',"sector":'null',"lat":"42.03384794140836","lng":"-7.786448352448858"},{"type":"House","id":727,"reference":"VGA20220035","community":11,"ccaa":"Galicia","province":"Ourense","town":"AGU\u00cdS, OS BLANCOS","postcode":"32634","form":1,"jobform":'null',"sector":'null',"lat":"41.982390268096964","lng":"-7.781994240473111"},{"type":"House","id":728,"reference":"VGA20220036","community":11,"ccaa":"Galicia","province":"Ourense","town":"AGU\u00cdS, OS BLANCOS","postcode":"32634","form":1,"jobform":'null',"sector":'null',"lat":"41.982581674647264","lng":"-7.782251732520933"},{"type":"House","id":730,"reference":"VCL20220065","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Paredes de Nava","postcode":"34300","form":1,"jobform":'null',"sector":'null',"lat":"42.15089276660979","lng":"-4.69760974363026"},{"type":"House","id":731,"reference":"VCL20220066","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"Villameca","postcode":"24397","form":1,"jobform":'null',"sector":'null',"lat":"42.643596731155874","lng":"-6.076580801346097"},{"type":"House","id":732,"reference":"VCL20220067","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"Alcoba de la Ribera","postcode":"24393","form":1,"jobform":'null',"sector":'null',"lat":"42.558384176916576","lng":"-5.827459320609562"},{"type":"House","id":733,"reference":"VEX20220001","community":10,"ccaa":"Extremadura","province":"Caceres","town":"Serrej\u00f3n","postcode":"10528","form":2,"jobform":'null',"sector":'null',"lat":"39.82014760172437","lng":"-5.803583120085828"},{"type":"House","id":734,"reference":"VEX20220002","community":10,"ccaa":"Extremadura","province":"Caceres","town":"Villanueva de la Vera","postcode":"10470","form":2,"jobform":'null',"sector":'null',"lat":"40.13567752229507","lng":"-5.462779347347455"},{"type":"House","id":736,"reference":"VEX20220004","community":10,"ccaa":"Extremadura","province":"Caceres","town":"Ahigal","postcode":"10650","form":2,"jobform":'null',"sector":'null',"lat":"40.20635458867068","lng":"-6.184443184475173"},{"type":"House","id":737,"reference":"VCB20220014","community":6,"ccaa":"Cantabria","province":"Cantabria","town":"ESLES","postcode":"39694","form":1,"jobform":'null',"sector":'null',"lat":"43.28280023399016","lng":"-3.80505859071964"},{"type":"House","id":738,"reference":"VCB20220015","community":6,"ccaa":"Cantabria","province":"Cantabria","town":"LA BUSTA","postcode":"39590","form":2,"jobform":'null',"sector":'null',"lat":"43.35527498201805","lng":"-4.180135452738174"},{"type":"House","id":739,"reference":"VCB20220016","community":6,"ccaa":"Cantabria","province":"Cantabria","town":"VEGA DE PAS","postcode":"39685","form":2,"jobform":'null',"sector":'null',"lat":"43.16163995456436","lng":"-3.7802729478137387"},{"type":"House","id":740,"reference":"VCB20220017","community":6,"ccaa":"Cantabria","province":"Cantabria","town":"SAN PEDRO DEL ROMERAL","postcode":"39686","form":1,"jobform":'null',"sector":'null',"lat":"43.11551899923177","lng":"-3.817946587360563"},{"type":"House","id":744,"reference":"VVC20220026","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"POTRIES","postcode":"46721","form":1,"jobform":'null',"sector":'null',"lat":"38.91788927343055","lng":"-0.19379429369061518"},{"type":"House","id":746,"reference":"VEX20220005","community":10,"ccaa":"Extremadura","province":"Caceres","town":"Jerte","postcode":"10612","form":2,"jobform":'null',"sector":'null',"lat":"40.23510801519866","lng":"-5.7499359258078435"},{"type":"House","id":747,"reference":"VCL20220068","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"Quintanilla de Sollamas","postcode":"24271","form":1,"jobform":'null',"sector":'null',"lat":"42.604200878584635","lng":"-5.821862222609262"},{"type":"House","id":748,"reference":"VCL20220069","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Osorno la Mayor","postcode":"34460","form":1,"jobform":'null',"sector":'null',"lat":"42.537971262072254","lng":"-4.686653932988334"},{"type":"House","id":749,"reference":"VCL20220070","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Salda\u00f1a","postcode":"34100","form":1,"jobform":'null',"sector":'null',"lat":"42.530991661047835","lng":"-4.7406162524564"},{"type":"House","id":750,"reference":"VCL20220071","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Carri\u00f3n de los Condes","postcode":"34120","form":1,"jobform":'null',"sector":'null',"lat":"42.42995267080712","lng":"-4.6147577620825935"},{"type":"House","id":751,"reference":"VCL20220072","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Villaturde","postcode":"34129","form":6,"jobform":'null',"sector":'null',"lat":"42.38180014225929","lng":"-4.670470329097279"},{"type":"House","id":752,"reference":"VAS20220013","community":3,"ccaa":"Asturias","province":"Asturias","town":"Salas","postcode":"33860","form":1,"jobform":'null',"sector":'null',"lat":"43.41823871313729,","lng":"6.261010565914434"},{"type":"House","id":753,"reference":"VCL20220073","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"TORENO","postcode":"24450","form":2,"jobform":'null',"sector":'null',"lat":"42.70322386031769","lng":"-6.517059820628607"},{"type":"House","id":754,"reference":"VCL20220074","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"TORENO","postcode":"24450","form":2,"jobform":'null',"sector":'null',"lat":"42.70372845786085","lng":"-6.514999884246023"},{"type":"House","id":756,"reference":"VCL20220076","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"VILLABUENA","postcode":"24540","form":2,"jobform":'null',"sector":'null',"lat":"42.60095783415407","lng":"-6.731403445998384"},{"type":"House","id":757,"reference":"VGA20220038","community":11,"ccaa":"Galicia","province":"Ourense","town":"Berrande","postcode":"32617","form":2,"jobform":'null',"sector":'null',"lat":"41.924742349791","lng":"-7.218097643407903"},{"type":"House","id":758,"reference":"VEX20220006","community":10,"ccaa":"Extremadura","province":"Badajoz","town":"Zalamea de la Serena","postcode":"06430","form":1,"jobform":'null',"sector":'null',"lat":"38.663948457623704","lng":"-5.6596709608234965"},{"type":"House","id":759,"reference":"VVC20220027","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"VILLALONGA","postcode":"46720","form":1,"jobform":'null',"sector":'null',"lat":"38.89485894790992","lng":"-0.20732426961190933"},{"type":"House","id":761,"reference":"VVC20220028","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"EL REAL DE GANDIA","postcode":"46727","form":1,"jobform":'null',"sector":'null',"lat":"38.952540658059256","lng":"-0.19353008042912465"},{"type":"House","id":762,"reference":"VVC20220029","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"CASTELL\u00d3 DE RUGAT","postcode":"46841","form":1,"jobform":'null',"sector":'null',"lat":"38.87952014653315","lng":"-0.3847470937559454"},{"type":"House","id":763,"reference":"VVC20220030","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"BENIG\u00c0NIM","postcode":"46830","form":1,"jobform":'null',"sector":'null',"lat":"38.9941466301189","lng":"-0.43803371183617507"},{"type":"House","id":784,"reference":"VCL20220069","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Villasabariego de Ucieza","postcode":"34127","form":1,"jobform":'null',"sector":'null',"lat":"42.38487231653444","lng":"-4.571547455033507"},{"type":"House","id":786,"reference":"VCL20220071","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Carri\u00f3n de los Condes","postcode":"34120","form":1,"jobform":'null',"sector":'null',"lat":"42.44886222934823","lng":"-4.637215459806361"},{"type":"House","id":787,"reference":"VCL20220072","community":8,"ccaa":"Castilla y Leon","province":"Avila","town":"La Aldehuela","postcode":"05593","form":1,"jobform":'null',"sector":'null',"lat":"40.41256177450988","lng":"-5.409984985212927"},{"type":"House","id":788,"reference":"VCL20220073","community":8,"ccaa":"Castilla y Leon","province":"Avila","town":"La Aldehuela","postcode":"05593","form":1,"jobform":'null',"sector":'null',"lat":"40.41248292800288","lng":"-5.4095707648230675"},{"type":"House","id":790,"reference":"VGA20220039","community":11,"ccaa":"Galicia","province":"Ourense","town":"Montederramos","postcode":"32750","form":1,"jobform":'null',"sector":'null',"lat":"42.278653044993774","lng":"-7.503059139556504"},{"type":"House","id":791,"reference":"VAS20220013","community":3,"ccaa":"Asturias","province":"Asturias","town":"Villar de Huergo","postcode":"33584","form":1,"jobform":'null',"sector":'null',"lat":"43.36244193015269","lng":"-5.242308669008698"},{"type":"House","id":792,"reference":"VAS20220014","community":3,"ccaa":"Asturias","province":"Asturias","town":"Sames","postcode":"33558","form":2,"jobform":'null',"sector":'null',"lat":"43.27090820274457","lng":"-5.129018367978255"},{"type":"House","id":795,"reference":"VCL20220075","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"SANCEDO","postcode":"24439","form":1,"jobform":'null',"sector":'null',"lat":"42.670892527937376","lng":"-6.6378364779107635"},{"type":"House","id":796,"reference":"VAR20220010","community":2,"ccaa":"Aragon","province":"Huesca","town":"ESP\u00c9S ALTO","postcode":"22471","form":1,"jobform":'null',"sector":'null',"lat":"42.47139405295242","lng":"0.5989915081633811"},{"type":"House","id":798,"reference":"VEX20220006","community":10,"ccaa":"Extremadura","province":"Caceres","town":"Villar del Pedroso","postcode":"10330","form":1,"jobform":'null',"sector":'null',"lat":"39.705269526267905","lng":"-5.197802089272617"},{"type":"House","id":799,"reference":"VAR20220012","community":2,"ccaa":"Aragon","province":"Huesca","town":"SEIRA","postcode":"22463","form":2,"jobform":'null',"sector":'null',"lat":"42.48078017263374","lng":"0.429594017107915"},{"type":"House","id":804,"reference":"VCL20220078","community":8,"ccaa":"Castilla y Leon","province":"Valladolid","town":"Moral de la Reina","postcode":"47691","form":1,"jobform":'null',"sector":'null',"lat":"41.996393319146556","lng":"-5.067191588875168"},{"type":"House","id":805,"reference":"VCL20220079","community":8,"ccaa":"Castilla y Leon","province":"Valladolid","town":"Moral de la Reina","postcode":"47691","form":1,"jobform":'null',"sector":'null',"lat":"41.9974525977475","lng":"-5.072843767258469"},{"type":"House","id":807,"reference":"VCL20220081","community":8,"ccaa":"Castilla y Leon","province":"Soria","town":"Reznos","postcode":"42126","form":1,"jobform":'null',"sector":'null',"lat":"41.608098517304654","lng":"-2.0308581873033247"},{"type":"House","id":809,"reference":"VVC20220027","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"VILLALONGA","postcode":"46720","form":1,"jobform":'null',"sector":'null',"lat":"38.89356094277473","lng":"-0.20736467268506478"},{"type":"House","id":810,"reference":"VVC20220028","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"BELLREGUARD","postcode":"46713","form":1,"jobform":'null',"sector":'null',"lat":"38.96137145315324","lng":"-0.1621379652934667"},{"type":"House","id":811,"reference":"VVC20220029","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"VILLALONGA","postcode":"46720","form":1,"jobform":'null',"sector":'null',"lat":"38.89382815507519","lng":"-0.20839464092338877"},{"type":"House","id":812,"reference":"VVC20220030","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"PALMA DE GANDIA","postcode":"46724","form":1,"jobform":'null',"sector":'null',"lat":"38.96480276058828","lng":"-0.22510262497789274"},{"type":"House","id":813,"reference":"VVC20220031","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"BELLREGUARD","postcode":"46713","form":1,"jobform":'null',"sector":'null',"lat":"38.96083753615658","lng":"-0.1641979017702262"},{"type":"House","id":814,"reference":"VCL20220083","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"BEMBIBRE","postcode":"24300","form":1,"jobform":'null',"sector":'null',"lat":"42.61753800802622","lng":"-6.4161798671397365"},{"type":"House","id":815,"reference":"VCL20220084","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"CONGOSTO","postcode":"24398","form":1,"jobform":'null',"sector":'null',"lat":"42.62109749040749","lng":"-6.520195715196368"},{"type":"House","id":816,"reference":"VEX20220007","community":10,"ccaa":"Extremadura","province":"Badajoz","town":"Zalamea de la Serena","postcode":"06430","form":1,"jobform":'null',"sector":'null',"lat":"38.65587607864781","lng":"-5.658713826395143"},{"type":"House","id":817,"reference":"VCL20220085","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"SAN JUAN DE LA MATA","postcode":"24546","form":1,"jobform":'null',"sector":'null',"lat":"42.64698950684971","lng":"-6.6660301253299865"},{"type":"House","id":818,"reference":"VEX20220008","community":10,"ccaa":"Extremadura","province":"Badajoz","town":"Zalamea de la Serena","postcode":"06430","form":1,"jobform":'null',"sector":'null',"lat":"38.65692669584651","lng":"-5.658713826395143"},{"type":"House","id":819,"reference":"VEX20220009","community":10,"ccaa":"Extremadura","province":"Badajoz","town":"Zalamea de la Serena","postcode":'null',"form":1,"jobform":'null',"sector":'null',"lat":"38.65832749480684","lng":"-5.661404591251479"},{"type":"House","id":820,"reference":"VEX20220010","community":10,"ccaa":"Extremadura","province":"Badajoz","town":"Zalamea de la Serena","postcode":"06430","form":1,"jobform":'null',"sector":'null',"lat":"38.656576491825824","lng":"-5.6591622872045315"},{"type":"House","id":821,"reference":"VCB20220021","community":6,"ccaa":"Cantabria","province":"Cantabria","town":"NOVALES","postcode":"39526","form":1,"jobform":'null',"sector":'null',"lat":"43.38171473394109","lng":"-4.179223148553131"},{"type":"House","id":822,"reference":"VVC20220032","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"PALMERA","postcode":"46714","form":1,"jobform":'null',"sector":'null',"lat":"38.94412848528355","lng":"-0.1526436166787589"},{"type":"House","id":823,"reference":"VVC20220033","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"LLOCNOU DE SANT JERONI","postcode":"46726","form":1,"jobform":'null',"sector":'null',"lat":"38.913911883132904","lng":"-0.2848828730812258"},{"type":"House","id":824,"reference":"VEX20220011","community":10,"ccaa":"Extremadura","province":"Badajoz","town":"Zalamea de la Serena","postcode":"06430","form":1,"jobform":'null',"sector":'null',"lat":"38.65762709875118","lng":"-5.6605076696327"},{"type":"House","id":825,"reference":"VEX20220012","community":10,"ccaa":"Extremadura","province":"Badajoz","town":"Zalamea de la Serena","postcode":"06430","form":1,"jobform":'null',"sector":'null',"lat":"38.65692669584651","lng":"-5.6578169047763645"},{"type":"House","id":826,"reference":"VEX20220013","community":10,"ccaa":"Extremadura","province":"Badajoz","town":"Zalamea de la Serena","postcode":"06430","form":1,"jobform":'null',"sector":'null',"lat":"38.659027884013526","lng":"-5.657368443966974"},{"type":"House","id":827,"reference":"VEX20220014","community":10,"ccaa":"Extremadura","province":"Badajoz","town":"Zalamea de la Serena","postcode":"06430","form":1,"jobform":'null',"sector":'null',"lat":"38.656576491825824","lng":"-5.659610748013921"},{"type":"House","id":828,"reference":"VEX20220015","community":10,"ccaa":"Extremadura","province":"Badajoz","town":"Zalamea de la SErena","postcode":"06430","form":1,"jobform":'null',"sector":'null',"lat":"38.658677690266316","lng":"-5.6578169047763645"},{"type":"House","id":829,"reference":"VEX20220016","community":10,"ccaa":"Extremadura","province":"Badajoz","town":"Zalamea de la Serena","postcode":"06430","form":1,"jobform":'null',"sector":'null',"lat":"38.65692669584651","lng":"-5.656919983157586"},{"type":"House","id":830,"reference":"VEX20220017","community":10,"ccaa":"Extremadura","province":"Badajoz","town":"Zalamea de la Serena","postcode":"06430","form":1,"jobform":'null',"sector":'null',"lat":"38.65832749480684","lng":"-5.657368443966974"},{"type":"House","id":831,"reference":"VEX20220018","community":10,"ccaa":"Extremadura","province":"Badajoz","town":"Zalamea de la Serena","postcode":"06430","form":1,"jobform":'null',"sector":'null',"lat":"38.659027884013526","lng":"-5.656471522348195"},{"type":"House","id":832,"reference":"VEX20220019","community":10,"ccaa":"Extremadura","province":"Badajoz","town":"Zalamea de la Serena","postcode":"06430","form":1,"jobform":'null',"sector":'null',"lat":"38.65832749480684","lng":"-5.6578169047763645"},{"type":"House","id":833,"reference":"VEX20220020","community":10,"ccaa":"Extremadura","province":"Badajoz","town":"Zalamea de la Serena","postcode":"06430","form":1,"jobform":'null',"sector":'null',"lat":"38.65832749480684","lng":"-5.656919983157586"},{"type":"House","id":834,"reference":"VEX20220021","community":10,"ccaa":"Extremadura","province":"Badajoz","town":"Zalamea de la Serena","postcode":"06430","form":1,"jobform":'null',"sector":'null',"lat":"38.65937807604848","lng":"-5.661404591251479"},{"type":"House","id":835,"reference":"VEX20220022","community":10,"ccaa":"Extremadura","province":"Badajoz","town":"Zalamea de la Serena","postcode":"06430","form":1,"jobform":'null',"sector":'null',"lat":"38.657276898154954","lng":"-5.660956130442091"},{"type":"House","id":836,"reference":"VCL20220086","community":8,"ccaa":"Castilla y Leon","province":"Valladolid","town":"Villabr\u00e1gima","postcode":"47820","form":1,"jobform":'null',"sector":'null',"lat":"41.82638878051019","lng":"-5.116126059932358"},{"type":"House","id":837,"reference":"VAR20220014","community":2,"ccaa":"Aragon","province":"Zaragoza","town":"Frasno (El)","postcode":"50320","form":2,"jobform":'null',"sector":'null',"lat":"41.418176753740376","lng":"-1.496764442841762"},{"type":"House","id":838,"reference":"VVC20220034","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"BELLREGUARD","postcode":"46713","form":1,"jobform":'null',"sector":'null',"lat":"38.965508640817895","lng":"-0.15845341660049397"},{"type":"House","id":839,"reference":"VEX20220023","community":10,"ccaa":"Extremadura","province":"Badajoz","town":"Zalamea de la Serena","postcode":"06430","form":1,"jobform":'null',"sector":'null',"lat":"38.655175658620905","lng":"-5.661404591251479"},{"type":"House","id":840,"reference":"VEX20220024","community":10,"ccaa":"Extremadura","province":"Badajoz","town":"Zalamea de la Serena","postcode":"06430","form":1,"jobform":'null',"sector":'null',"lat":"38.65937807604848","lng":"-5.6578169047763645"},{"type":"House","id":841,"reference":"VEX20220025","community":10,"ccaa":"Extremadura","province":"Badajoz","town":"Zalamea de la Serena","postcode":"06430","form":1,"jobform":'null',"sector":'null',"lat":"38.65552586949045","lng":"-5.658713826395143"},{"type":"House","id":842,"reference":"VEX20220026","community":10,"ccaa":"Extremadura","province":"Badajoz","town":"Zalamea de la Serena","postcode":"06430","form":1,"jobform":'null',"sector":'null',"lat":"38.65937807604848","lng":"-5.662301512870258"},{"type":"House","id":843,"reference":"VCL20220087","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"PALACIOS DE LA VALDUERNA","postcode":"24764","form":2,"jobform":'null',"sector":'null',"lat":"42.3323805189434","lng":"-5.938320474845254"},{"type":"House","id":844,"reference":"VGA20220042","community":11,"ccaa":"Galicia","province":"Ourense","town":"Morgade, Xinzo de Limia","postcode":"32630","form":1,"jobform":'null',"sector":'null',"lat":"42.08291750236254","lng":"-7.699957964556297"},{"type":"House","id":846,"reference":"VEX20220028","community":10,"ccaa":"Extremadura","province":"Badajoz","town":"Esparragosa de la Serena","postcode":"06439","form":1,"jobform":'null',"sector":'null',"lat":"38.65218834755894","lng":"-5.605246740383717"},{"type":"House","id":850,"reference":"VCB20220022","community":6,"ccaa":"Cantabria","province":"Cantabria","town":"novales","postcode":"39526","form":1,"jobform":'null',"sector":'null',"lat":"43.378416","lng":"-4.1783102"},{"type":"House","id":851,"reference":"VCB20220023","community":6,"ccaa":"Cantabria","province":"Cantabria","town":"novales","postcode":"39526","form":1,"jobform":'null',"sector":'null',"lat":"43.378416","lng":"-4.1783102"},{"type":"House","id":852,"reference":"VCL20220088","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Carbonera","postcode":"34100","form":6,"jobform":'null',"sector":'null',"lat":"42.5931378","lng":"-4.7364842"},{"type":"House","id":853,"reference":"VVC20220037","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"PILES","postcode":"46712","form":1,"jobform":'null',"sector":'null',"lat":"38.941941","lng":"-0.1342888"},{"type":"House","id":854,"reference":"VVC20220038","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"BENIFL\u00c0","postcode":"46722","form":1,"jobform":'null',"sector":'null',"lat":"38.92841","lng":"-0.1885938"},{"type":"House","id":855,"reference":"VVC20220039","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"BENIFAIR\u00d3 DE LA VALLDIGNA","postcode":"46791","form":1,"jobform":'null',"sector":'null',"lat":"39.0547984","lng":"-0.3031019"},{"type":"House","id":856,"reference":"VVC20220040","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"LA FONT D\u0027EN CARR\u00d2S","postcode":"46717","form":1,"jobform":'null',"sector":'null',"lat":"38.9164419","lng":"-0.1711542"},{"type":"House","id":857,"reference":"VVC20220041","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"VILALONGA\/VILLALONGA","postcode":"46720","form":1,"jobform":'null',"sector":'null',"lat":"38.8860393","lng":"-0.2070885"},{"type":"House","id":858,"reference":"VVC20220042","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"PLAYA DE BELLREGUARD","postcode":"46713","form":1,"jobform":'null',"sector":'null',"lat":"38.9618123","lng":"-0.133505"},{"type":"House","id":859,"reference":"VVC20220043","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"BENIARJ\u00d3","postcode":"46722","form":1,"jobform":'null',"sector":'null',"lat":"38.9326258","lng":"-0.1865978"},{"type":"House","id":860,"reference":"VVC20220044","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"BENIFAIR\u00d3 DE LA VALLDIGNA","postcode":"46791","form":1,"jobform":'null',"sector":'null',"lat":"39.0547984","lng":"-0.3031019"},{"type":"House","id":861,"reference":"VVC20220045","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"VILALLONGA\/VILLALONGA","postcode":"46720","form":1,"jobform":'null',"sector":'null',"lat":"38.8860393","lng":"-0.2070885"},{"type":"House","id":862,"reference":"VAR20220015","community":2,"ccaa":"Aragon","province":"Zaragoza","town":"Frasno (El)","postcode":"50320","form":1,"jobform":'null',"sector":'null',"lat":"41.4146655","lng":"-1.4960571"},{"type":"House","id":863,"reference":"VCL20220090","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"TRABADELO","postcode":"24522","form":1,"jobform":'null',"sector":'null',"lat":"42.6496751","lng":"-6.88168"},{"type":"House","id":865,"reference":"VAS20220015","community":3,"ccaa":"Asturias","province":"Asturias","town":"Vega De Sebarga","postcode":"33557","form":1,"jobform":'null',"sector":'null',"lat":"43.2735212","lng":"-5.1661789"},{"type":"House","id":866,"reference":"VAS20220016","community":3,"ccaa":"Asturias","province":"Asturias","town":"Sames","postcode":"33558","form":1,"jobform":'null',"sector":'null',"lat":"43.2696933","lng":"-5.1290389"},{"type":"House","id":867,"reference":"VAS20220017","community":3,"ccaa":"Asturias","province":"Asturias","town":"cerca de Sames","postcode":"33558","form":1,"jobform":'null',"sector":'null',"lat":"43.2696933","lng":"-5.1290389"},{"type":"House","id":868,"reference":"VAS20220018","community":3,"ccaa":"Asturias","province":"Asturias","town":"Bele\u00f1o","postcode":'null',"form":1,"jobform":'null',"sector":'null',"lat":"43.1938484","lng":"-5.1640605"},{"type":"House","id":869,"reference":"VAS20220019","community":3,"ccaa":"Asturias","province":"Asturias","town":"Viboli","postcode":"33558","form":1,"jobform":'null',"sector":'null',"lat":"43.1938484","lng":"-5.1640605"},{"type":"House","id":870,"reference":"VAS20220020","community":3,"ccaa":"Asturias","province":"Asturias","town":"Bele\u00f1o","postcode":"33557","form":1,"jobform":'null',"sector":'null',"lat":"43.1938484","lng":"-5.1640605"},{"type":"House","id":871,"reference":"VAS20220021","community":3,"ccaa":"Asturias","province":"Asturias","town":"Carangas","postcode":"33557","form":1,"jobform":'null',"sector":'null',"lat":"43.2250528","lng":"-5.1912796"},{"type":"House","id":872,"reference":"VAS20220022","community":3,"ccaa":"Asturias","province":"Asturias","town":"Car\u00faa","postcode":"33585","form":1,"jobform":'null',"sector":'null',"lat":"43.3300356","lng":"-5.216636"},{"type":"House","id":873,"reference":"VAS20220023","community":3,"ccaa":"Asturias","province":"Asturias","town":"Seg\u00fcenco","postcode":"33559","form":1,"jobform":'null',"sector":'null',"lat":"43.3208743","lng":"-5.1077081"},{"type":"House","id":874,"reference":"VCL20220091","community":8,"ccaa":"Castilla y Leon","province":"Valladolid","town":"Villabr\u00e1gima","postcode":"47820","form":1,"jobform":'null',"sector":'null',"lat":"41.8220989","lng":"-5.115848"},{"type":"House","id":875,"reference":"VVC20220047","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"PLAYA DE BELLREGUARD","postcode":"46713","form":1,"jobform":'null',"sector":'null',"lat":"38.9618123","lng":"-0.133505"},{"type":"House","id":876,"reference":"VCL20220092","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Salda\u00f1a","postcode":"34100","form":2,"jobform":'null',"sector":'null',"lat":"42.5180071","lng":"-4.7419825"},{"type":"House","id":877,"reference":"VCL20220093","community":8,"ccaa":"Castilla y Leon","province":"Avila","town":"EL BARCO DE \u00c1VILA","postcode":"05600","form":2,"jobform":'null',"sector":'null',"lat":"40.3580563","lng":"-5.5191156"},{"type":"House","id":878,"reference":"VCL20220094","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Castromocho","postcode":"34305","form":3,"jobform":'null',"sector":'null',"lat":"42.0308091","lng":"-4.8241988"},{"type":"House","id":879,"reference":"VCL20220095","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Carri\u00f3n de los Condes","postcode":"34120","form":1,"jobform":'null',"sector":'null',"lat":"42.3307302","lng":"-4.6140481"},{"type":"House","id":880,"reference":"VGA20220043","community":11,"ccaa":"Galicia","province":"Ourense","town":"Laza","postcode":"32621","form":1,"jobform":'null',"sector":'null',"lat":"42.0924463","lng":"-7.4597725"},{"type":"House","id":881,"reference":"VVC20220048","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"BENIARJ\u00d3","postcode":"46722","form":1,"jobform":'null',"sector":'null',"lat":"38.938568994853725","lng":"-0.1868848822018041"},{"type":"House","id":882,"reference":"VVC20220049","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"L\u0027ALQUERIA DE LA COMTESSA","postcode":"46715","form":1,"jobform":'null',"sector":'null',"lat":"38.945312859496205","lng":"-0.14934181001215874"},{"type":"House","id":883,"reference":"VCL20220096","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Osorno La Mayor","postcode":"34460","form":1,"jobform":'null',"sector":'null',"lat":"42.410566","lng":"-4.360873"},{"type":"House","id":884,"reference":"VVC20230050","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"BENIFAIR\u00d3 DE LA VALLDIGNA","postcode":"46791","form":1,"jobform":'null',"sector":'null',"lat":"39.0547984","lng":"-0.3031019"},{"type":"House","id":885,"reference":"VCL20230097","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Villoldo","postcode":"34131","form":1,"jobform":'null',"sector":'null',"lat":"42.2460438","lng":"-4.5953355"},{"type":"House","id":886,"reference":"VCL20230098","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"MAGAZ DE ABAJO","postcode":"24410","form":1,"jobform":'null',"sector":'null',"lat":"42.594355","lng":"-6.686959"},{"type":"House","id":887,"reference":"VVC20230051","community":17,"ccaa":"Comunidad Valenciana","province":"Castellon","town":"Alboc\u00e0sser","postcode":"12140","form":2,"jobform":'null',"sector":'null',"lat":"40.356701","lng":"0.0250705"},{"type":"House","id":888,"reference":"VVC20230052","community":17,"ccaa":"Comunidad Valenciana","province":"Castellon","town":"Alboc\u00e0sser","postcode":"12140","form":2,"jobform":'null',"sector":'null',"lat":"40.356701","lng":"0.0250705"},{"type":"House","id":890,"reference":"VCL20230100","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"SAN PEDRO DE OLLEROS","postcode":"24436","form":1,"jobform":'null',"sector":'null',"lat":"42.6968374","lng":"-6.7179595"},{"type":"House","id":891,"reference":"VCL20230101","community":8,"ccaa":"Castilla y Leon","province":"Avila","town":"Gil Garc\u00eda","postcode":"05693","form":1,"jobform":'null',"sector":'null',"lat":"40.2978672","lng":"-5.594224"},{"type":"House","id":892,"reference":"VCL20230102","community":8,"ccaa":"Castilla y Leon","province":"Avila","town":"Gil Garc\u00eda","postcode":"05693","form":1,"jobform":'null',"sector":'null',"lat":"40.2978672","lng":"-5.594224"},{"type":"House","id":893,"reference":"VAS20230024","community":3,"ccaa":"Asturias","province":"Asturias","town":'null',"postcode":'null',"form":6,"jobform":'null',"sector":'null',"lat":"43.4480757","lng":"-6.7049198"},{"type":"House","id":894,"reference":"VCL20230103","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"LA BRA\u00d1A","postcode":"24520","form":1,"jobform":'null',"sector":'null',"lat":"42.7060579","lng":"-6.9643711"},{"type":"House","id":896,"reference":"VAS20230025","community":3,"ccaa":"Asturias","province":"Asturias","town":"Cedemonio","postcode":"33732","form":6,"jobform":'null',"sector":'null',"lat":"43.372868","lng":"-6.8401249"},{"type":"House","id":897,"reference":"VAS20230026","community":3,"ccaa":"Asturias","province":"Asturias","town":"Cedemonio","postcode":"33732","form":1,"jobform":'null',"sector":'null',"lat":"43.372868","lng":"-6.8401249"},{"type":"House","id":899,"reference":"VAS20230027","community":3,"ccaa":"Asturias","province":"Asturias","town":"Busmente","postcode":"33717","form":2,"jobform":'null',"sector":'null',"lat":"43.4243513","lng":"-6.6902116"},{"type":"House","id":901,"reference":"VCL20230105","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"Carrizo","postcode":"24270","form":2,"jobform":'null',"sector":'null',"lat":"42.5839551","lng":"-5.8292753"},{"type":"House","id":902,"reference":"VCL20230106","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Osorno La Mayor","postcode":"34460","form":1,"jobform":'null',"sector":'null',"lat":"42.369478","lng":"-4.3770704"},{"type":"House","id":903,"reference":"VAR20230017","community":2,"ccaa":"Aragon","province":"Zaragoza","town":"Aguilon","postcode":"50155","form":1,"jobform":'null',"sector":'null',"lat":"41.2943156","lng":"-1.0460453"},{"type":"House","id":904,"reference":"VVC20230053","community":17,"ccaa":"Comunidad Valenciana","province":"Castellon","town":"Xodos","postcode":"12134","form":1,"jobform":'null',"sector":'null',"lat":"40.2449792","lng":"-0.2869518"},{"type":"House","id":905,"reference":"VVC20230054","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"XERESA","postcode":"46760","form":1,"jobform":'null',"sector":'null',"lat":"39.0102892","lng":"-0.2168457"},{"type":"House","id":906,"reference":"VVC20230055","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"BELLREGUARD","postcode":"46713","form":1,"jobform":'null',"sector":'null',"lat":"38.9458572","lng":"-0.1619949"},{"type":"House","id":907,"reference":"VVC20230056","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"DAIM\u00daS","postcode":"46710","form":1,"jobform":'null',"sector":'null',"lat":"38.97236","lng":"-0.1486987"},{"type":"House","id":908,"reference":"VVC20230057","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"PILES","postcode":"46712","form":1,"jobform":'null',"sector":'null',"lat":"38.941941","lng":"-0.1342888"},{"type":"House","id":909,"reference":"VVC20230058","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"BELLREGUARD","postcode":"46713","form":1,"jobform":'null',"sector":'null',"lat":"38.9458572","lng":"-0.1619949"},{"type":"House","id":910,"reference":"VVC20230059","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"RAFELCOFER","postcode":"46716","form":1,"jobform":'null',"sector":'null',"lat":"38.9321104","lng":"-0.1676417"},{"type":"House","id":911,"reference":"VVC20230060","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"RAFELCOFER","postcode":"46716","form":1,"jobform":'null',"sector":'null',"lat":"38.9321104","lng":"-0.1676417"},{"type":"House","id":912,"reference":"VVC20230061","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"RAFELCOFER","postcode":"46716","form":1,"jobform":'null',"sector":'null',"lat":"38.9321104","lng":"-0.1676417"},{"type":"House","id":913,"reference":"VVC20230062","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"LA FONT D\u0027EN CARR\u00d2S","postcode":"46717","form":1,"jobform":'null',"sector":'null',"lat":"38.9164419","lng":"-0.1711542"},{"type":"House","id":914,"reference":"VVC20230063","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"LA FONT D\u0027EN CARR\u00d2S","postcode":"46717","form":1,"jobform":'null',"sector":'null',"lat":"38.9164419","lng":"-0.1711542"},{"type":"House","id":915,"reference":"VVC20230064","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"PALMA DE GANDIA","postcode":"46724","form":1,"jobform":'null',"sector":'null',"lat":"38.9250593","lng":"-0.2214767"},{"type":"House","id":916,"reference":"VVC20230065","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"ADOR","postcode":"46729","form":1,"jobform":'null',"sector":'null',"lat":"38.9183079","lng":"-0.2256387"},{"type":"House","id":918,"reference":"VVC20230067","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"PALMA DE GANDIA","postcode":"46724","form":1,"jobform":'null',"sector":'null',"lat":"38.9250593","lng":"-0.2214767"},{"type":"House","id":920,"reference":"VAR20230018","community":2,"ccaa":"Aragon","province":"Huesca","town":"Castej\u00f3n de Sos","postcode":"22466","form":2,"jobform":'null',"sector":'null',"lat":"42.5351538","lng":"0.5021589"},{"type":"House","id":921,"reference":"VCL20230107","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Monz\u00f3n de Campos","postcode":"34410","form":1,"jobform":'null',"sector":'null',"lat":"42.1159093","lng":"-4.4939766"},{"type":"House","id":928,"reference":"VCL20230109","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Monz\u00f3n de Campos","postcode":"34410","form":1,"jobform":'null',"sector":'null',"lat":"42.1159093","lng":"-4.4939766"},{"type":"House","id":929,"reference":"VCL20230110","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Monz\u00f3n de Campos","postcode":"34410","form":1,"jobform":'null',"sector":'null',"lat":"42.1159093","lng":"-4.4939766"},{"type":"House","id":930,"reference":"VCL20230111","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Fuentes de Valdepero","postcode":"34419","form":1,"jobform":'null',"sector":'null',"lat":"42.0747776","lng":"-4.5012249"},{"type":"House","id":931,"reference":"VVC20230069","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"XERACO","postcode":"46770","form":1,"jobform":'null',"sector":'null',"lat":"39.033098","lng":"-0.2162596"},{"type":"House","id":932,"reference":"VVC20230070","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"RAFELCOFER","postcode":"46716","form":1,"jobform":'null',"sector":'null',"lat":"38.9321104","lng":"-0.1676417"},{"type":"House","id":933,"reference":"VGA20230045","community":11,"ccaa":"Galicia","province":"Ourense","town":"Rexosende","postcode":"32616","form":2,"jobform":'null',"sector":'null',"lat":"41.9008635","lng":"-7.2484295"},{"type":"House","id":934,"reference":"VVC20230071","community":17,"ccaa":"Comunidad Valenciana","province":"Castellon","town":"Alboc\u00e0sser","postcode":"12140","form":3,"jobform":'null',"sector":'null',"lat":"40.356701","lng":"0.0250705"},{"type":"House","id":935,"reference":"VCB20230024","community":6,"ccaa":"Cantabria","province":"Cantabria","town":"Vega de Pas","postcode":"39696","form":3,"jobform":'null',"sector":'null',"lat":"43.1576","lng":"-3.7860815"},{"type":"House","id":936,"reference":"VCB20230025","community":6,"ccaa":"Cantabria","province":"Cantabria","town":"Vega de Pas","postcode":"39685","form":1,"jobform":'null',"sector":'null',"lat":"43.1576","lng":"-3.7860815"},{"type":"House","id":937,"reference":"VVC20230072","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"BELLREGUARD PLAYA","postcode":"46713","form":1,"jobform":'null',"sector":'null',"lat":"38.9618123","lng":"-0.133505"},{"type":"House","id":939,"reference":"VCL20230112","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"cacabelos","postcode":"24540","form":2,"jobform":'null',"sector":'null',"lat":"42.5997655","lng":"-6.7276913"},{"type":"House","id":940,"reference":"VCL20230113","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"CACABELOS","postcode":"24540","form":2,"jobform":'null',"sector":'null',"lat":"42.5997655","lng":"-6.7276913"},{"type":"House","id":941,"reference":"VCL20230114","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"CACABELOS","postcode":"24540","form":2,"jobform":'null',"sector":'null',"lat":"42.5997655","lng":"-6.7276913"},{"type":"Land","id":10,"reference":"TCL2021027","community":8,"ccaa":"Castilla y Leon","province":"Avila","town":"Becedas","postcode":"05610","form":1,"jobform":'null',"sector":'null',"lat":"40,403609","lng":"5,635280"},{"type":"Land","id":54,"reference":"TCL2021026","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"SANTIBA\u00d1EZ DE VALDEIGLESIAS","postcode":"24288","form":6,"jobform":'null',"sector":'null',"lat":"42.46481929820288","lng":"-5.928343125158764"},{"type":"Land","id":55,"reference":"TAR2022015","community":2,"ccaa":"Aragon","province":"Zaragoza","town":"CODOS","postcode":"50326","form":2,"jobform":'null',"sector":'null',"lat":"41.300434658451756","lng":"-1.372007297282539"},{"type":"Land","id":56,"reference":"TAR2022014","community":2,"ccaa":"Aragon","province":"Zaragoza","town":"CODOS","postcode":"50326","form":2,"jobform":'null',"sector":'null',"lat":"41.300434658451756","lng":"-1.372007297282539"},{"type":"Land","id":57,"reference":"TAR2022013","community":2,"ccaa":"Aragon","province":"Zaragoza","town":"CODOS","postcode":"50326","form":2,"jobform":'null',"sector":'null',"lat":"41.300434658451756","lng":"-1.372007297282539"},{"type":"Land","id":58,"reference":"TAR2022012","community":2,"ccaa":"Aragon","province":"Zaragoza","town":"CODOS","postcode":"50326","form":2,"jobform":'null',"sector":'null',"lat":"41.300434658451756","lng":"-1.372007297282539"},{"type":"Land","id":59,"reference":"TAR2022011","community":2,"ccaa":"Aragon","province":"Zaragoza","town":"CODOS","postcode":"50326","form":2,"jobform":'null',"sector":'null',"lat":"41.300434658451756","lng":"-1.372007297282539"},{"type":"Land","id":60,"reference":"TAR2022010","community":2,"ccaa":"Aragon","province":"Zaragoza","town":"CODOS","postcode":"50326","form":2,"jobform":'null',"sector":'null',"lat":"41.300434658451756","lng":"-1.372007297282539"},{"type":"Land","id":61,"reference":"TAR2022009","community":2,"ccaa":"Aragon","province":"Zaragoza","town":"CODOS","postcode":"50326","form":2,"jobform":'null',"sector":'null',"lat":"41.300434658451756","lng":"-1.372007297282539"},{"type":"Land","id":62,"reference":"TAR2022008","community":2,"ccaa":"Aragon","province":"Zaragoza","town":"CODOS","postcode":"50326","form":2,"jobform":'null',"sector":'null',"lat":"41.300434658451756","lng":"-1.372007297282539"},{"type":"Land","id":63,"reference":"TAR2022007","community":2,"ccaa":"Aragon","province":"Zaragoza","town":"CODOS","postcode":"50326","form":2,"jobform":'null',"sector":'null',"lat":"41.300434658451756","lng":"-1.372007297282539"},{"type":"Land","id":64,"reference":"TAR2022006","community":2,"ccaa":"Aragon","province":"Zaragoza","town":"CODOS","postcode":"50326","form":2,"jobform":'null',"sector":'null',"lat":"41.300434658451756","lng":"-1.372007297282539"},{"type":"Land","id":69,"reference":"TVC2021010","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"La Font d\u0027en Carr\u00f2s","postcode":"46717","form":4,"jobform":'null',"sector":'null',"lat":"38.9164419","lng":"-0.1711542"},{"type":"Land","id":70,"reference":"TVC2021009","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"Almoines","postcode":"46723","form":4,"jobform":'null',"sector":'null',"lat":"38.9438345","lng":"-0.1816936"},{"type":"Land","id":71,"reference":"TVC2021008","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"Almoines","postcode":"46723","form":4,"jobform":'null',"sector":'null',"lat":"38.9438345","lng":"-0.1816936"},{"type":"Land","id":73,"reference":"TVC2021007","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"Ador","postcode":"46729","form":4,"jobform":'null',"sector":'null',"lat":"38.9183079","lng":"-0.2256387"},{"type":"Land","id":74,"reference":"TVC2021006","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"Ador","postcode":"46729","form":4,"jobform":'null',"sector":'null',"lat":"38.9183079","lng":"-0.2256387"},{"type":"Land","id":75,"reference":"TVC2021005","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"Ador","postcode":"46729","form":4,"jobform":'null',"sector":'null',"lat":"38.9183079","lng":"-0.2256387"},{"type":"Land","id":76,"reference":"TVC2021004","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"Llocnou de Sant Jeroni","postcode":"46726","form":4,"jobform":'null',"sector":'null',"lat":"38.8985934","lng":"-0.2854839"},{"type":"Land","id":78,"reference":"TVC2021003","community":17,"ccaa":"Comunidad Valenciana","province":"Alicante","town":"Pedreguer","postcode":"03750","form":4,"jobform":'null',"sector":'null',"lat":"38.7930797","lng":"0.0338198"},{"type":"Land","id":79,"reference":"TVC2021002","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"Palma de Gandia","postcode":"46724","form":8,"jobform":'null',"sector":'null',"lat":"38.9250593","lng":"-0.2214767"},{"type":"Land","id":80,"reference":"TVC2021001","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"Palma de Gandia","postcode":"46724","form":8,"jobform":'null',"sector":'null',"lat":"38.9250593","lng":"-0.2214767"},{"type":"Land","id":81,"reference":"TAN2022022","community":1,"ccaa":"Andalucia","province":"Malaga","town":"BENALAUR\u00cdA","postcode":"29491","form":2,"jobform":'null',"sector":'null',"lat":"36.595862865985254","lng":"-5.260624992578665"},{"type":"Land","id":82,"reference":"TAN2022021","community":1,"ccaa":"Andalucia","province":"Malaga","town":"BENALAUR\u00cdA","postcode":"29491","form":2,"jobform":'null',"sector":'null',"lat":"36.595862865985254","lng":"-5.260624992578665"},{"type":"Land","id":83,"reference":"TAN2022020","community":1,"ccaa":"Andalucia","province":"Malaga","town":"BENALAUR\u00cdA","postcode":"29491","form":2,"jobform":'null',"sector":'null',"lat":"36.595862865985254","lng":"-5.260624992578665"},{"type":"Land","id":84,"reference":"TAN2022019","community":1,"ccaa":"Andalucia","province":"Malaga","town":"BENALAUR\u00cdA","postcode":"29491","form":2,"jobform":'null',"sector":'null',"lat":"36.595862865985254","lng":"-5.260624992578665"},{"type":"Land","id":85,"reference":"TAN2022018","community":1,"ccaa":"Andalucia","province":"Malaga","town":"BENALAUR\u00cdA","postcode":"29491","form":2,"jobform":'null',"sector":'null',"lat":"36.595862865985254","lng":"-5.260624992578665"},{"type":"Land","id":86,"reference":"TAN2022017","community":1,"ccaa":"Andalucia","province":"Malaga","town":"BENALAUR\u00cdA","postcode":"29491","form":2,"jobform":'null',"sector":'null',"lat":"36.595862865985254","lng":"-5.260624992578665"},{"type":"Land","id":87,"reference":"TAN2022016","community":1,"ccaa":"Andalucia","province":"Malaga","town":"BENALAUR\u00cdA","postcode":"29491","form":8,"jobform":'null',"sector":'null',"lat":"36.595862865985254","lng":"-5.260624992578665"},{"type":"Land","id":114,"reference":"TCL2021025","community":8,"ccaa":"Castilla y Leon","province":"Valladolid","town":"Mayorga","postcode":"47680","form":1,"jobform":'null',"sector":'null',"lat":"42.1662402","lng":"-5.2645259"},{"type":"Land","id":115,"reference":"TCL2021024","community":8,"ccaa":"Castilla y Leon","province":"Valladolid","town":"Mayorga","postcode":"47680","form":1,"jobform":'null',"sector":'null',"lat":"42.1662402","lng":"-5.2645259"},{"type":"Land","id":116,"reference":"TCL2021023","community":8,"ccaa":"Castilla y Leon","province":"Valladolid","town":"Mayorga","postcode":"47680","form":1,"jobform":'null',"sector":'null',"lat":"42.1662402","lng":"-5.2645259"},{"type":"Land","id":117,"reference":"TCL2021022","community":8,"ccaa":"Castilla y Leon","province":"Valladolid","town":"Mayorga","postcode":"47680","form":1,"jobform":'null',"sector":'null',"lat":"42.1662402","lng":"-5.2645259"},{"type":"Land","id":118,"reference":"TCL2021021","community":8,"ccaa":"Castilla y Leon","province":"Valladolid","town":"Mayorga","postcode":"47680","form":1,"jobform":'null',"sector":'null',"lat":"42.1662402","lng":"-5.2645259"},{"type":"Land","id":119,"reference":"TCL2021020","community":8,"ccaa":"Castilla y Leon","province":"Valladolid","town":"Melgar de Abajo","postcode":"47687","form":2,"jobform":'null',"sector":'null',"lat":"42.2437542","lng":"-5.1425553"},{"type":"Land","id":120,"reference":"TCL2021019","community":8,"ccaa":"Castilla y Leon","province":"Valladolid","town":"Melgar de Abajo","postcode":"47687","form":2,"jobform":'null',"sector":'null',"lat":"42.2437542","lng":"-5.1425553"},{"type":"Land","id":121,"reference":"TCL2021018","community":8,"ccaa":"Castilla y Leon","province":"Valladolid","town":"Melgar de Abajo","postcode":"47687","form":2,"jobform":'null',"sector":'null',"lat":"42.2437542","lng":"-5.1425553"},{"type":"Land","id":122,"reference":"TCL2021017","community":8,"ccaa":"Castilla y Leon","province":"Valladolid","town":"Melgar de Abajo","postcode":"47687","form":2,"jobform":'null',"sector":'null',"lat":"42.2437542","lng":"-5.1425553"},{"type":"Land","id":123,"reference":"TCL2021016","community":8,"ccaa":"Castilla y Leon","province":"Valladolid","town":"Melgar de Abajo","postcode":"47687","form":2,"jobform":'null',"sector":'null',"lat":"42.2437542","lng":"-5.1425553"},{"type":"Land","id":124,"reference":"TCL2021015","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"VILLACETE","postcode":"24227","form":1,"jobform":'null',"sector":'null',"lat":"42.56524664251375","lng":"-5.469702939331428"},{"type":"Land","id":125,"reference":"TCL2021014","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"VILLACETE","postcode":"24227","form":1,"jobform":'null',"sector":'null',"lat":"42.56524664251375","lng":"-5.469702939331428"},{"type":"Land","id":126,"reference":"TCL2021013","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"VILLACETE","postcode":"24227","form":1,"jobform":'null',"sector":'null',"lat":"42.56524664251375","lng":"-5.469702939331428"},{"type":"Land","id":127,"reference":"TCL2021012","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"VILLACETE","postcode":"24227","form":1,"jobform":'null',"sector":'null',"lat":"42.56524664251375","lng":"-5.469702939331428"},{"type":"Land","id":128,"reference":"TCL2021011","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"VILLACETE","postcode":"24227","form":1,"jobform":'null',"sector":'null',"lat":"42.56524664251375","lng":"-5.469702939331428"},{"type":"Land","id":129,"reference":"TCL2021010","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"VILLACETE","postcode":"24227","form":1,"jobform":'null',"sector":'null',"lat":"42.56524664251375","lng":"-5.469702939331428"},{"type":"Land","id":130,"reference":"TCL2021009","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"VILLACETE","postcode":"24227","form":1,"jobform":'null',"sector":'null',"lat":"42.56524664251375","lng":"-5.469702939331428"},{"type":"Land","id":131,"reference":"TAN2022015","community":1,"ccaa":"Andalucia","province":"Granada","town":"Galera","postcode":'null',"form":1,"jobform":'null',"sector":'null',"lat":"37.74928323008714","lng":"-2.550485815563536"},{"type":"Land","id":132,"reference":"TAN2022014","community":1,"ccaa":"Andalucia","province":"Granada","town":"Galera","postcode":"18840","form":1,"jobform":'null',"sector":'null',"lat":"37.74928323008714","lng":"-2.550485815563536"},{"type":"Land","id":133,"reference":"TAN2022013","community":1,"ccaa":"Andalucia","province":"Granada","town":"Galera","postcode":"18840","form":1,"jobform":'null',"sector":'null',"lat":"37.74928323008714","lng":"-2.550485815563536"},{"type":"Land","id":134,"reference":"TAN2022012","community":1,"ccaa":"Andalucia","province":"Granada","town":"Galera","postcode":"18840","form":1,"jobform":'null',"sector":'null',"lat":"37.74928323008714","lng":"-2.550485815563536"},{"type":"Land","id":135,"reference":"TAN2022011","community":1,"ccaa":"Andalucia","province":"Granada","town":"Galera","postcode":"18840","form":1,"jobform":'null',"sector":'null',"lat":"37.74928323008714","lng":"-2.550485815563536"},{"type":"Land","id":136,"reference":"TAN2022010","community":1,"ccaa":"Andalucia","province":"Granada","town":"Galera","postcode":"18840","form":1,"jobform":'null',"sector":'null',"lat":"37.74928323008714","lng":"-2.550485815563536"},{"type":"Land","id":137,"reference":"TAN2022009","community":1,"ccaa":"Andalucia","province":"Granada","town":"Galera","postcode":"18840","form":1,"jobform":'null',"sector":'null',"lat":"37.746131454681446","lng":"-2.55121253385013"},{"type":"Land","id":138,"reference":"TAN2022008","community":1,"ccaa":"Andalucia","province":"Granada","town":"Galera","postcode":"18840","form":1,"jobform":'null',"sector":'null',"lat":"37.74596200867488","lng":"-2.552069694269959"},{"type":"Land","id":139,"reference":"TAN2022007","community":1,"ccaa":"Andalucia","province":"Granada","town":"Galera","postcode":"18840","form":1,"jobform":'null',"sector":'null',"lat":"37.74596200867488","lng":"-2.551855404165002"},{"type":"Land","id":140,"reference":"TAN2022006","community":1,"ccaa":"Andalucia","province":"Granada","town":"Galera","postcode":"18840","form":1,"jobform":'null',"sector":'null',"lat":"37.74579256228036","lng":"-2.551855404165002"},{"type":"Land","id":141,"reference":"TAN2022005","community":1,"ccaa":"Andalucia","province":"Granada","town":"Galera","postcode":"18840","form":1,"jobform":'null',"sector":'null',"lat":"37.74596200867488","lng":"-2.5507839536402157"},{"type":"Land","id":142,"reference":"TAN2022004","community":1,"ccaa":"Andalucia","province":"Granada","town":"Galera","postcode":"18840","form":1,"jobform":'null',"sector":'null',"lat":"37.745284220769065","lng":"-2.5514268239550875"},{"type":"Land","id":143,"reference":"TAN2022003","community":1,"ccaa":"Andalucia","province":"Granada","town":"Galera","postcode":"18840","form":1,"jobform":'null',"sector":'null',"lat":"37.74681869780325","lng":"-2.5505494573143594"},{"type":"Land","id":144,"reference":"TAN2022002","community":1,"ccaa":"Andalucia","province":"Granada","town":"Galera","postcode":"18840","form":1,"jobform":'null',"sector":'null',"lat":"37.90202636931686","lng":"-2.570066116350584"},{"type":"Land","id":146,"reference":"TGA2021002","community":11,"ccaa":"Galicia","province":"Pontevedra","town":"As Motas, A Ca\u00f1iza","postcode":"36880","form":1,"jobform":'null',"sector":'null',"lat":"42.2183941","lng":"-8.2705298"},{"type":"Land","id":147,"reference":"TGA2021001","community":11,"ccaa":"Galicia","province":"Pontevedra","town":"A Veiga, A Ca\u00f1iza","postcode":"36880","form":1,"jobform":'null',"sector":'null',"lat":"42.2183941","lng":"-8.2705298"},{"type":"Land","id":148,"reference":"TCL2021008","community":8,"ccaa":"Castilla y Leon","province":"Salamanca","town":"Valdelacasa","postcode":"37791","form":4,"jobform":'null',"sector":'null',"lat":"40.5066472804582","lng":"-5.763198909180277"},{"type":"Land","id":149,"reference":"TCL2021007","community":8,"ccaa":"Castilla y Leon","province":"Salamanca","town":"Valdelacasa","postcode":"37791","form":4,"jobform":'null',"sector":'null',"lat":"40.50651944698668","lng":"-5.763156877021853"},{"type":"Land","id":150,"reference":"TCL2021006","community":8,"ccaa":"Castilla y Leon","province":"Salamanca","town":"Valdelacasa","postcode":"37791","form":4,"jobform":'null',"sector":'null',"lat":"40.50651710000002","lng":"-5.763051842455985"},{"type":"Land","id":151,"reference":"TCL2021005","community":8,"ccaa":"Castilla y Leon","province":"Salamanca","town":"Valdelacasa","postcode":"37791","form":4,"jobform":'null',"sector":'null',"lat":"40.57874701647815","lng":"-5.759576535458701"},{"type":"Land","id":152,"reference":"TCL2021004","community":8,"ccaa":"Castilla y Leon","province":"Zamora","town":"Torregamones","postcode":'null',"form":10,"jobform":'null',"sector":'null',"lat":"41.49237310854473","lng":"-6.179707697223397"},{"type":"Land","id":153,"reference":"TEX2021003","community":10,"ccaa":"Extremadura","province":"Badajoz","town":"Torremayor","postcode":"06880","form":1,"jobform":'null',"sector":'null',"lat":"38.899667881440834","lng":"-6.538839875369223"},{"type":"Land","id":206,"reference":"TEX2021002","community":10,"ccaa":"Extremadura","province":"Badajoz","town":"Calera de Le\u00f3n","postcode":'null',"form":2,"jobform":'null',"sector":'null',"lat":"38.10663538001527","lng":"-6.336859623650426"},{"type":"Land","id":207,"reference":"TEX2021001","community":10,"ccaa":"Extremadura","province":"Badajoz","town":"Calera de Le\u00f3n","postcode":"06292","form":2,"jobform":'null',"sector":'null',"lat":"38.171919276298546","lng":"-6.3329463307960525"},{"type":"Land","id":208,"reference":"TCL2021003","community":8,"ccaa":"Castilla y Leon","province":"Avila","town":"Poyales del Hoyo","postcode":"05492","form":1,"jobform":'null',"sector":'null',"lat":"40.18309357676101","lng":"-5.166255792256665"},{"type":"Land","id":209,"reference":"TCL2021002","community":8,"ccaa":"Castilla y Leon","province":"Avila","town":"Poyales del Hoyo","postcode":"05492","form":1,"jobform":'null',"sector":'null',"lat":"40.18309357676101","lng":"-5.166255792256665"},{"type":"Land","id":210,"reference":"TCL2021001","community":8,"ccaa":"Castilla y Leon","province":"Avila","town":"Poyales del Hoyo","postcode":"05492","form":1,"jobform":'null',"sector":'null',"lat":"40.18309357676101","lng":"-5.166255792256665"},{"type":"Land","id":243,"reference":"TCB2022001","community":6,"ccaa":"Cantabria","province":"Cantabria","town":"San Pedro de Rudag\u00fcera","postcode":"39539","form":1,"jobform":'null',"sector":'null',"lat":"43.3611261","lng":"-4.1495121"},{"type":"Land","id":244,"reference":"TCL2022001","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Villaumbrales","postcode":"34192","form":1,"jobform":'null',"sector":'null',"lat":"42.0857364","lng":"-4.5830789"},{"type":"Land","id":245,"reference":"TNC2022001","community":14,"ccaa":"Navarra","province":"Navarra","town":"Artaza","postcode":"31272","form":1,"jobform":'null',"sector":'null',"lat":"42.77331865787118","lng":"-2.1096559798964405"},{"type":"Land","id":246,"reference":"TCL2022002","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Poblaci\u00f3n de Campos","postcode":"34449","form":1,"jobform":'null',"sector":'null',"lat":"42.2703272","lng":"-4.4471534"},{"type":"Land","id":247,"reference":"TCL2022003","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Fr\u00f3mista","postcode":"34440","form":1,"jobform":'null',"sector":'null',"lat":"42.2676329","lng":"-4.4053497"},{"type":"Land","id":248,"reference":"TCL2022004","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Grijota","postcode":"34192","form":6,"jobform":'null',"sector":'null',"lat":"42.0530505","lng":"-4.5824339"},{"type":"Land","id":249,"reference":"TCL2022005","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Carri\u00f3n de los Condes","postcode":"34120","form":1,"jobform":'null',"sector":'null',"lat":"42.3307302","lng":"-4.6140481"},{"type":"Land","id":250,"reference":"TCL2022006","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Cervatos de la Cueza","postcode":"34309","form":1,"jobform":'null',"sector":'null',"lat":"42.2905776","lng":"-4.7698058"},{"type":"Land","id":251,"reference":"TCL2022007","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Carri\u00f3n de los Condes","postcode":"34120","form":1,"jobform":'null',"sector":'null',"lat":"42.3307302","lng":"-4.6140481"},{"type":"Land","id":252,"reference":"TCL2022008","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Carri\u00f3n de los Condes","postcode":"34120","form":1,"jobform":'null',"sector":'null',"lat":"42.3307302","lng":"-4.6140481"},{"type":"Land","id":253,"reference":"TCL2022009","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Villoldo","postcode":"34131","form":1,"jobform":'null',"sector":'null',"lat":"42.2460438","lng":"-4.5953355"},{"type":"Land","id":254,"reference":"TGA2022002","community":11,"ccaa":"Galicia","province":"Ourense","town":"Ver\u00edn","postcode":"32600","form":1,"jobform":'null',"sector":'null',"lat":"41.9316334","lng":"-7.4327273"},{"type":"Land","id":255,"reference":"TGA2022003","community":11,"ccaa":"Galicia","province":"Ourense","town":"Ver\u00edn","postcode":"32600","form":1,"jobform":'null',"sector":'null',"lat":"41.9316334","lng":"-7.4327273"},{"type":"Land","id":256,"reference":"TCL2022010","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Monz\u00f3n de Campos","postcode":"34410","form":1,"jobform":'null',"sector":'null',"lat":"42.1159093","lng":"-4.4939766"},{"type":"Land","id":257,"reference":"TCL2022011","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"Santoyo","postcode":"34490","form":1,"jobform":'null',"sector":'null',"lat":"42.220303607645555","lng":"-4.342529360225567"},{"type":"Land","id":258,"reference":"TAR2022001","community":2,"ccaa":"Aragon","province":"Huesca","town":"Ch\u00eda","postcode":"22465","form":1,"jobform":'null',"sector":'null',"lat":"42.5209012","lng":"0.4652754"},{"type":"Land","id":259,"reference":"TCL2022012","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"Puente de \u00d3rbigo","postcode":"24286","form":1,"jobform":'null',"sector":'null',"lat":"42.4636342","lng":"-5.8769366"},{"type":"Land","id":260,"reference":"TVC2022001","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"R\u00f3tova","postcode":"46725","form":1,"jobform":'null',"sector":'null',"lat":"38.9319466","lng":"-0.2578967"},{"type":"Land","id":261,"reference":"TAR2022002","community":2,"ccaa":"Aragon","province":"Zaragoza","town":"Luceni","postcode":"50640","form":1,"jobform":'null',"sector":'null',"lat":"41.8273021","lng":"-1.2421823"},{"type":"Land","id":262,"reference":"TAN2022001","community":1,"ccaa":"Andalucia","province":"Huelva","town":"Arroyomolinos de Le\u00f3n","postcode":"21280","form":1,"jobform":'null',"sector":'null',"lat":"38.0240295","lng":"-6.4216722"},{"type":"Land","id":263,"reference":"TAS2022001","community":3,"ccaa":"Asturias","province":"Asturias","town":"Cia\u00f1o","postcode":"33900","form":1,"jobform":'null',"sector":'null',"lat":"43.28978528104795","lng":"-5.669418923597622"},{"type":"Land","id":264,"reference":"TVC2022002","community":17,"ccaa":"Comunidad Valenciana","province":"Valencia","town":"BENIARJ\u00d3","postcode":"46722","form":1,"jobform":'null',"sector":'null',"lat":"38.936824907567114","lng":"-0.1865415519168182"},{"type":"Land","id":265,"reference":"TAS2022002","community":3,"ccaa":"Asturias","province":"Asturias","town":"Varios emplazamientos","postcode":"33757","form":8,"jobform":'null',"sector":'null',"lat":"43.54206723564714","lng":"-6.865754914406009"},{"type":"Land","id":266,"reference":"TAS2022003","community":3,"ccaa":"Asturias","province":"Asturias","town":"Varios","postcode":"33730","form":8,"jobform":'null',"sector":'null',"lat":"43.255028653002725","lng":"-6.866422732259388"},{"type":"Land","id":267,"reference":"TVC2022003","community":17,"ccaa":"Comunidad Valenciana","province":"Castellon","town":"CASTELLON","postcode":"12132","form":1,"jobform":'null',"sector":'null',"lat":"41.1274451353833","lng":"-0.10768210371760241"},{"type":"Land","id":268,"reference":"TVC2022004","community":17,"ccaa":"Comunidad Valenciana","province":"Castellon","town":"CASTELLON","postcode":"12163","form":1,"jobform":'null',"sector":'null',"lat":"40.35068033327605","lng":"-0.16367734461008804"},{"type":"Land","id":269,"reference":"TVC2022005","community":17,"ccaa":"Comunidad Valenciana","province":"Castellon","town":"CASTELLON","postcode":"12163","form":1,"jobform":'null',"sector":'null',"lat":"40.34031377217158","lng":"-0.16596953822908045"},{"type":"Land","id":270,"reference":"TVC2022006","community":17,"ccaa":"Comunidad Valenciana","province":"Castellon","town":"CASTELLON","postcode":"12163","form":1,"jobform":'null',"sector":'null',"lat":"40.34109882284341","lng":"-0.1657978768638651"},{"type":"Land","id":271,"reference":"TVC2022007","community":17,"ccaa":"Comunidad Valenciana","province":"Castellon","town":"CULLA","postcode":"12163","form":1,"jobform":'null',"sector":'null',"lat":"40.34044461458463","lng":"-0.16476790867257296"},{"type":"Land","id":272,"reference":"TVC2022008","community":17,"ccaa":"Comunidad Valenciana","province":"Castellon","town":"CULLA","postcode":"12163","form":1,"jobform":'null',"sector":'null',"lat":"40.34031377217158","lng":"-0.16562621549864973"},{"type":"Land","id":277,"reference":"TGA2022004","community":11,"ccaa":"Galicia","province":"Pontevedra","town":"Parroquia de Angudes y Saboi","postcode":"36420","form":1,"jobform":'null',"sector":'null',"lat":"42.19811137362692","lng":"-8.236163593749366"},{"type":"Land","id":278,"reference":"TGA2022005","community":11,"ccaa":"Galicia","province":"Ourense","town":"Ribadavia","postcode":"32400","form":1,"jobform":'null',"sector":'null',"lat":"42.35608420641772","lng":"-8.161038169786492"},{"type":"Land","id":279,"reference":"TCL2022013","community":8,"ccaa":"Castilla y Leon","province":"Burgos","town":"Castrojeriz","postcode":"09110","form":1,"jobform":'null',"sector":'null',"lat":"42.30198115816918","lng":"-4.141805282541699"},{"type":"Land","id":280,"reference":"TCL2022014","community":8,"ccaa":"Castilla y Leon","province":"Segovia","town":"Sep\u00falveda","postcode":"40300","form":1,"jobform":'null',"sector":'null',"lat":"41.37645446193141","lng":"-3.7453934684244494"},{"type":"Land","id":281,"reference":"TGA2022006","community":11,"ccaa":"Galicia","province":"Ourense","town":"Francelos","postcode":"32418","form":1,"jobform":'null',"sector":'null',"lat":"42.274490464269796","lng":"-8.16145172944904"},{"type":"Land","id":282,"reference":"TNC2022002","community":14,"ccaa":"Navarra","province":"Navarra","town":"Cabredo","postcode":"31227","form":1,"jobform":'null',"sector":'null',"lat":"42.64343436790612","lng":"-2.4149895875828338"},{"type":"Land","id":283,"reference":"TAS2022004","community":3,"ccaa":"Asturias","province":"Asturias","town":"Ballota","postcode":"33158","form":1,"jobform":'null',"sector":'null',"lat":"43.551930447575785","lng":"-6.329898716725107"},{"type":"Land","id":284,"reference":"TCL2022015","community":8,"ccaa":"Castilla y Leon","province":"Avila","town":"Bohoyo","postcode":"05690","form":1,"jobform":'null',"sector":'null',"lat":"40.3156321","lng":"-5.4417473"},{"type":"Land","id":285,"reference":"TCB2022002","community":6,"ccaa":"Cantabria","province":"Cantabria","town":"Ore\u00f1a","postcode":"39525","form":1,"jobform":'null',"sector":'null',"lat":"43.3941216","lng":"-4.1353837"},{"type":"Land","id":286,"reference":"TCL2023016","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"CAMPONARAYA","postcode":"24410","form":2,"jobform":'null',"sector":'null',"lat":"42.5784571","lng":"-6.6673978"},{"type":"Land","id":287,"reference":"TCL2023017","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"CARRACEDELO","postcode":"24549","form":2,"jobform":'null',"sector":'null',"lat":"42.5559174","lng":"-6.7326972"},{"type":"Land","id":288,"reference":"TCL2023018","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"SORRIBAS","postcode":"24560","form":2,"jobform":'null',"sector":'null',"lat":"42.5850697","lng":"-6.7412034"},{"type":"Land","id":289,"reference":"TCL2023019","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":'null',"postcode":'null',"form":2,"jobform":'null',"sector":'null',"lat":"42.6203117","lng":"-6.6374517"},{"type":"Land","id":290,"reference":"TCL2023020","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":'null',"postcode":'null',"form":2,"jobform":'null',"sector":'null',"lat":"42.6236268","lng":"-6.5640882"},{"type":"Land","id":292,"reference":"TCL2023021","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"Villavante","postcode":"24393","form":1,"jobform":'null',"sector":'null',"lat":"42.4626305","lng":"-5.8367024"},{"type":"Land","id":293,"reference":"TAR2023003","community":2,"ccaa":"Aragon","province":"Huesca","town":"VILLANOVA","postcode":"22467","form":1,"jobform":'null',"sector":'null',"lat":"42.5472755","lng":"0.4617364"},{"type":"Land","id":294,"reference":"TAR2023004","community":2,"ccaa":"Aragon","province":"Huesca","town":"GAB\u00c1S","postcode":'null',"form":1,"jobform":'null',"sector":'null',"lat":"42.4829189","lng":"0.4905"},{"type":"Land","id":295,"reference":"TCL2023022","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":"T\u00e1mara de Campos","postcode":"34439","form":1,"jobform":'null',"sector":'null',"lat":"42.2034861","lng":"-4.3931363"},{"type":"Job","id":15,"reference":"ECL2022009","community":8,"ccaa":"Castilla y Leon","province":"Avila","town":"Barco de \u00c1vila (El)","postcode":"05600","form":'null',"jobform":6,"sector":8,"lat":"40.3569063","lng":"-5.5236746"},{"type":"Job","id":37,"reference":"ECL2022022","community":8,"ccaa":"Castilla y Leon","province":"Valladolid","town":"Melgar de Arriba","postcode":"7686","form":'null',"jobform":6,"sector":'null',"lat":"42.267274","lng":"-5.0976161"},{"type":"Job","id":57,"reference":"ECL2022036","community":8,"ccaa":"Castilla y Leon","province":"Avila","town":"Piedrah\u00edta","postcode":"05500","form":'null',"jobform":6,"sector":8,"lat":"40.4652069","lng":"-5.3272431"},{"type":"Job","id":58,"reference":"ECL2022037","community":8,"ccaa":"Castilla y Leon","province":"Avila","town":"Piedrah\u00edta","postcode":"05500","form":'null',"jobform":6,"sector":8,"lat":"40.4652069","lng":"-5.3272431"},{"type":"Job","id":64,"reference":"EAR2022011","community":2,"ccaa":"Aragon","province":"Zaragoza","town":"Codos","postcode":"50326","form":'null',"jobform":1,"sector":8,"lat":"41.2940092","lng":"-1.3746587"},{"type":"Job","id":65,"reference":"EGA2022002","community":11,"ccaa":"Galicia","province":"Ourense","town":"Lodoselo","postcode":"32696","form":'null',"jobform":6,"sector":8,"lat":"42.0701091","lng":"-7.5938025"},{"type":"Job","id":69,"reference":"ECL2022040","community":8,"ccaa":"Castilla y Leon","province":"Zamora","town":"Villar del Buey","postcode":"49240","form":'null',"jobform":3,"sector":2,"lat":"41.3294996","lng":"-6.189163"},{"type":"Job","id":71,"reference":"ECL2022041","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"Vega de Valcarce","postcode":"24520","form":'null',"jobform":6,"sector":5,"lat":"42.6640879","lng":"-6.9454043"},{"type":"Job","id":77,"reference":"ECL2022043","community":8,"ccaa":"Castilla y Leon","province":"Avila","town":"NAVARREDONDA DE GREDOS","postcode":"05635","form":'null',"jobform":6,"sector":8,"lat":"40.36163","lng":"-5.1308286"},{"type":"Job","id":78,"reference":"ECL2022044","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"FABERO","postcode":"24420","form":'null',"jobform":1,"sector":10,"lat":"42.7673954","lng":"-6.6237322"},{"type":"Job","id":83,"reference":"EEX2022003","community":10,"ccaa":"Extremadura","province":"Caceres","town":"Torrecillas de la TIesa","postcode":"10252","form":'null',"jobform":1,"sector":5,"lat":"39.57292863179694","lng":"-5.742896330034069"},{"type":"Job","id":90,"reference":"EEX2022010","community":10,"ccaa":"Extremadura","province":"Badajoz","town":"Zalamea de la Serena","postcode":"06430","form":'null',"jobform":6,"sector":2,"lat":"38.71362525250026","lng":"-5.648315228399656"},{"type":"Job","id":97,"reference":"EGA2022003","community":11,"ccaa":"Galicia","province":"Ourense","town":"Castro Caldela","postcode":"32760","form":'null',"jobform":6,"sector":2,"lat":"42.406317926646004","lng":"-7.413458468330445"},{"type":"Job","id":99,"reference":"EEX2022011","community":10,"ccaa":"Extremadura","province":"Badajoz","town":"Zalamea de la Serena","postcode":"06430","form":'null',"jobform":6,"sector":2,"lat":"38.66208866651269","lng":"-5.658297676699731"},{"type":"Job","id":101,"reference":"ECL2022050","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":"cubillos del sil","postcode":"24492","form":'null',"jobform":1,"sector":9,"lat":"42.694360426368384","lng":"-6.589462236033173"},{"type":"Job","id":103,"reference":"ECT2022001","community":9,"ccaa":"Catalu\u00f1a","province":"Lleida","town":'null',"postcode":'null',"form":'null',"jobform":2,"sector":6,"lat":"41.66089593067382","lng":"1.1360864529138979"},{"type":"Job","id":104,"reference":"ECT2022002","community":9,"ccaa":"Catalu\u00f1a","province":"Lleida","town":'null',"postcode":'null',"form":'null',"jobform":6,"sector":7,"lat":"41.66089593067382","lng":"1.1360864529138979"},{"type":"Job","id":105,"reference":"ECT2022003","community":9,"ccaa":"Catalu\u00f1a","province":"Lleida","town":'null',"postcode":'null',"form":'null',"jobform":3,"sector":1,"lat":"41.637173519950025","lng":"1.0113802154100582"},{"type":"Job","id":106,"reference":"ECT2022004","community":9,"ccaa":"Catalu\u00f1a","province":"Lleida","town":'null',"postcode":'null',"form":'null',"jobform":8,"sector":8,"lat":"41.66089593067382","lng":"1.1360864529138979"},{"type":"Job","id":107,"reference":"ECT2022005","community":9,"ccaa":"Catalu\u00f1a","province":"Lleida","town":'null',"postcode":'null',"form":'null',"jobform":1,"sector":10,"lat":"41.66089593067382","lng":"1.1360864529138979"},{"type":"Job","id":108,"reference":"ECT2022006","community":9,"ccaa":"Catalu\u00f1a","province":"Lleida","town":'null',"postcode":'null',"form":'null',"jobform":1,"sector":10,"lat":"41.66089593067382","lng":"1.1360864529138979"},{"type":"Job","id":109,"reference":"ECT2022007","community":9,"ccaa":"Catalu\u00f1a","province":"Lleida","town":'null',"postcode":'null',"form":'null',"jobform":8,"sector":9,"lat":"41.66089593067382","lng":"1.1360864529138979"},{"type":"Job","id":110,"reference":"ECT2022008","community":9,"ccaa":"Catalu\u00f1a","province":"Lleida","town":'null',"postcode":'null',"form":'null',"jobform":8,"sector":9,"lat":"41.65791098101579","lng":"1.1344883202950218"},{"type":"Job","id":111,"reference":"ECB2022004","community":6,"ccaa":"Cantabria","province":"Cantabria","town":'null',"postcode":'null',"form":'null',"jobform":2,"sector":8,"lat":"43.31217355862712","lng":"-3.854086591763754"},{"type":"Job","id":112,"reference":"ECB2022005","community":6,"ccaa":"Cantabria","province":"Cantabria","town":'null',"postcode":'null',"form":'null',"jobform":5,"sector":7,"lat":"43.34365514633862","lng":"-3.744636795631434"},{"type":"Job","id":113,"reference":"ECB2022006","community":6,"ccaa":"Cantabria","province":"Cantabria","town":'null',"postcode":'null',"form":'null',"jobform":2,"sector":10,"lat":"43.34054413065656","lng":"-3.8056045349420837"},{"type":"Job","id":114,"reference":"EAS2022003","community":3,"ccaa":"Asturias","province":"Asturias","town":"Boal","postcode":"33720","form":'null',"jobform":2,"sector":8,"lat":"43.44119230908011","lng":"-6.8205092818648225"},{"type":"Job","id":115,"reference":"EEX2022012","community":10,"ccaa":"Extremadura","province":"Badajoz","town":"Zalamea de la Serena","postcode":"06430","form":'null',"jobform":6,"sector":9,"lat":"38.6564759502694","lng":"-5.65989131513679"},{"type":"Job","id":117,"reference":"ECT2022009","community":9,"ccaa":"Catalu\u00f1a","province":"Lleida","town":"Bellpuig","postcode":'null',"form":'null',"jobform":2,"sector":1,"lat":"41.6254321","lng":"1.0119509"},{"type":"Job","id":118,"reference":"ECT2022010","community":9,"ccaa":"Catalu\u00f1a","province":"Lleida","town":'null',"postcode":'null',"form":'null',"jobform":1,"sector":9,"lat":"41.6480797","lng":"1.1409432"},{"type":"Job","id":122,"reference":"ECL2023055","community":8,"ccaa":"Castilla y Leon","province":"Avila","town":"La Carrera","postcode":"05620","form":'null',"jobform":2,"sector":9,"lat":"40.3458217","lng":"-5.55146"},{"type":"Job","id":123,"reference":"ECL2023056","community":8,"ccaa":"Castilla y Leon","province":"Avila","town":"EL BARCO DE \u00c1VILA","postcode":"05600","form":'null',"jobform":2,"sector":2,"lat":"40.3569063","lng":"-5.5236746"},{"type":"Job","id":124,"reference":"ECL2023057","community":8,"ccaa":"Castilla y Leon","province":"Valladolid","town":"Melgar de Abajo","postcode":"47687","form":'null',"jobform":6,"sector":10,"lat":"42.2437542","lng":"-5.1425553"},{"type":"Job","id":125,"reference":"ECL2023058","community":8,"ccaa":"Castilla y Leon","province":"Salamanca","town":"Puente Del Congosto","postcode":"37748","form":'null',"jobform":4,"sector":9,"lat":"40.4881346","lng":"-5.5282659"},{"type":"Job","id":126,"reference":"ECL2023059","community":8,"ccaa":"Castilla y Leon","province":"Salamanca","town":"Puente del Congosto","postcode":"37748","form":'null',"jobform":2,"sector":2,"lat":"40.4881346","lng":"-5.5282659"},{"type":"Job","id":127,"reference":"ECL2023060","community":8,"ccaa":"Castilla y Leon","province":"Salamanca","town":"Puente Del Congosto","postcode":"37748","form":'null',"jobform":2,"sector":2,"lat":"40.4881346","lng":"-5.5282659"},{"type":"Job","id":128,"reference":"ECL2023061","community":8,"ccaa":"Castilla y Leon","province":"Salamanca","town":"Puente Del Congosto","postcode":"37748","form":'null',"jobform":2,"sector":2,"lat":"40.4881346","lng":"-5.5282659"},{"type":"Job","id":129,"reference":"ECL2023062","community":8,"ccaa":"Castilla y Leon","province":"Salamanca","town":"Puente Del Congosto","postcode":"37748","form":'null',"jobform":2,"sector":2,"lat":"40.4881346","lng":"-5.5282659"},{"type":"Job","id":130,"reference":"ECL2023063","community":8,"ccaa":"Castilla y Leon","province":"Salamanca","town":"Puente Del Congosto","postcode":"37748","form":'null',"jobform":2,"sector":2,"lat":"40.4881346","lng":"-5.5282659"},{"type":"Job","id":132,"reference":"ECL2023065","community":8,"ccaa":"Castilla y Leon","province":"Palencia","town":'null',"postcode":"34120","form":'null',"jobform":6,"sector":5,"lat":"42.3307302","lng":"-4.6140481"},{"type":"Job","id":133,"reference":"ECB2023007","community":6,"ccaa":"Cantabria","province":"Cantabria","town":"CANTABRIA","postcode":"39696","form":'null',"jobform":5,"sector":2,"lat":"43.2281523","lng":"-3.8102032"},{"type":"Job","id":134,"reference":"ECB2023008","community":6,"ccaa":"Cantabria","province":"Cantabria","town":"CANTABRIA","postcode":"39696","form":'null',"jobform":6,"sector":9,"lat":"43.3099809","lng":"-3.8534422"},{"type":"Job","id":135,"reference":"ECB2023009","community":6,"ccaa":"Cantabria","province":"Cantabria","town":'null',"postcode":'null',"form":'null',"jobform":2,"sector":9,"lat":"43.2979404","lng":"-3.9677559"},{"type":"Job","id":136,"reference":"ECB2023010","community":6,"ccaa":"Cantabria","province":"Cantabria","town":'null',"postcode":'null',"form":'null',"jobform":1,"sector":5,"lat":"43.378583","lng":"-4.1820961"},{"type":"Job","id":137,"reference":"ECB2023011","community":6,"ccaa":"Cantabria","province":"Cantabria","town":'null',"postcode":'null',"form":'null',"jobform":1,"sector":8,"lat":"43.2281523","lng":"-3.8102032"},{"type":"Job","id":138,"reference":"ECB2023012","community":6,"ccaa":"Cantabria","province":"Cantabria","town":'null',"postcode":'null',"form":'null',"jobform":3,"sector":2,"lat":"43.2370484","lng":"-3.9400298"},{"type":"Job","id":139,"reference":"ECB2023013","community":6,"ccaa":"Cantabria","province":"Cantabria","town":'null',"postcode":'null',"form":'null',"jobform":3,"sector":4,"lat":"43.378583","lng":"-4.1820961"},{"type":"Job","id":140,"reference":"EAR2023015","community":2,"ccaa":"Aragon","province":"Huesca","town":"Sah\u00fan","postcode":"22468","form":'null',"jobform":3,"sector":2,"lat":"42.5760582","lng":"0.4652754"},{"type":"Job","id":141,"reference":"EAR2023016","community":2,"ccaa":"Aragon","province":"Huesca","town":"Sah\u00fan","postcode":"22468","form":'null',"jobform":3,"sector":2,"lat":"42.5760582","lng":"0.4652754"},{"type":"Job","id":142,"reference":"EAR2023017","community":2,"ccaa":"Aragon","province":"Huesca","town":"Refugio de Cap de Llauset (Parque Natural Posets-Maladeta)","postcode":"22487","form":'null',"jobform":5,"sector":2,"lat":"42.5962504","lng":"0.6831175"},{"type":"Job","id":143,"reference":"ECL2023066","community":8,"ccaa":"Castilla y Leon","province":"Leon","town":'null',"postcode":"24420","form":'null',"jobform":6,"sector":8,"lat":"42.7673954","lng":"-6.6237322"},{"type":"Job","id":145,"reference":"EAR2023018","community":2,"ccaa":"Aragon","province":"Huesca","town":"Benasque","postcode":"22440","form":'null',"jobform":2,"sector":8,"lat":"42.6042858","lng":"0.5228286"},{"type":"Job","id":146,"reference":"ECB2023014","community":6,"ccaa":"Cantabria","province":"Cantabria","town":"C\u00f3breces","postcode":'null',"form":'null',"jobform":1,"sector":2,"lat":"43.3859306","lng":"-4.2115526"},{"type":"Job","id":147,"reference":"ECB2023015","community":6,"ccaa":"Cantabria","province":"Cantabria","town":'null',"postcode":'null',"form":'null',"jobform":3,"sector":2,"lat":"43.378583","lng":"-4.1820961"},{"type":"Job","id":148,"reference":"ECB2023016","community":6,"ccaa":"Cantabria","province":"Cantabria","town":"Sar\u00f3n","postcode":'null',"form":'null',"jobform":2,"sector":8,"lat":"43.3244492","lng":"-3.8534422"},{"type":"Job","id":149,"reference":"ECB2023017","community":6,"ccaa":"Cantabria","province":"Cantabria","town":'null',"postcode":'null',"form":'null',"jobform":2,"sector":2,"lat":"43.3396134","lng":"-3.8054364"},{"type":"Job","id":150,"reference":"ECB2023018","community":6,"ccaa":"Cantabria","province":"Cantabria","town":'null',"postcode":'null',"form":'null',"jobform":2,"sector":7,"lat":"43.3396134","lng":"-3.8054364"},{"type":"Job","id":151,"reference":"EAS2023004","community":3,"ccaa":"Asturias","province":"Asturias","town":"Llames de Parres","postcode":"33585","form":'null',"jobform":2,"sector":9,"lat":"43.3553279","lng":"-5.2113355"},{"type":"Job","id":153,"reference":"ECL2023068","community":8,"ccaa":"Castilla y Leon","province":"Salamanca","town":"Monleras","postcode":"37171","form":'null',"jobform":3,"sector":10,"lat":"41.1861677","lng":"-6.2263127"},{"type":"Job","id":154,"reference":"ECL2023069","community":8,"ccaa":"Castilla y Leon","province":"Avila","town":"El Barco de \u00c1vila","postcode":"05600","form":'null',"jobform":8,"sector":2,"lat":"40.3580563","lng":"-5.5191156"},{"type":"Job","id":155,"reference":"ECL2023070","community":8,"ccaa":"Castilla y Leon","province":"Avila","town":"Bohoyo","postcode":"05690","form":'null',"jobform":1,"sector":2,"lat":"40.3156321","lng":"-5.4417473"},{"type":"Job","id":156,"reference":"ECB2023020","community":6,"ccaa":"Cantabria","province":"Cantabria","town":'null',"postcode":'null',"form":'null',"jobform":3,"sector":7,"lat":"43.3429642","lng":"-3.744658"},{"type":"Job","id":159,"reference":"ECL2023071","community":8,"ccaa":"Castilla y Leon","province":"Avila","town":"Navacepeda de Tormes","postcode":"05633","form":'null',"jobform":6,"sector":2,"lat":"40.3604289","lng":"-5.2498365"},{"type":"Job","id":160,"reference":"EAR2023021","community":2,"ccaa":"Aragon","province":"Teruel","town":"Gea de Albarrac\u00edn","postcode":"44110","form":'null',"jobform":6,"sector":10,"lat":"40.4113351","lng":"-1.3475274"}];

In [44]:
df_volver = pd.json_normalize(markers)
df_volver

,type,id,reference,community,ccaa,province,town,postcode,form,jobform,sector,lat,lng
0,Business,14,NCM2022001,7,Castilla La Mancha,Guadalajara,Peralejos de las Truchas,19313,1,null,2,40.5929386,-1.9093132
1,Business,16,NCL2022003,8,Castilla y Leon,Burgos,Villadiego,09120,1,null,2,42.5146862,-4.0101705
2,Business,17,NCL2022004,8,Castilla y Leon,Burgos,Villahoz,09343,2,null,2,42.0771757,-3.9126371
3,Business,19,NCL2022005,8,Castilla y Leon,Palencia,Carrión de los condes,34120,1,null,2,42.3307302,-4.6140481
4,Business,20,NCL2022006,8,Castilla y Leon,Palencia,Frómista,34440,1,null,2,42.2676329,-4.4053497
...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,Job,154,ECL2023069,8,Castilla y Leon,Avila,El Barco de Ávila,05600,null,8,2,40.3580563,-5.5191156
836,Job,155,ECL2023070,8,Castilla y Leon,Avila,Bohoyo,05690,null,1,2,40.3156321,-5.4417473
837,Job,156,ECB2023020,6,Cantabria,Cantabria,null,null,null,3,7,43.3429642,-3.744658
838,Job,159,ECL2023071,8,Castilla y Leon,Avila,Navacepeda de Tormes,05633,null,6,2,40.3604289,-5.2498365


In [45]:
# Se filtra dejando las columnas útiles
df_volver=df_volver[['town','lat','lng']]

In [46]:
# Se eliminan filas duplicadas y se renombran las columnas
df_volver = df_volver.drop_duplicates()
df_volver.columns=['Municipio','lat','lng']

In [47]:
# Se eliminan las filas donde el valor de la columna Municipio es 'null'
df_volver=df_volver[~(df_volver['Municipio']=='null')]

In [48]:
# Se añade la columna 'Link_web' para poder dar opción de ir a la web a buscar más información sobre los pueblos
df_volver['Link_web'] = 'http://volveralpueblo.coceder.org/mapa-de-recursos'
df_volver

,Municipio,lat,lng,Link_web
0,Peralejos de las Truchas,40.5929386,-1.9093132,http://volveralpueblo.coceder.org/mapa-de-recu...
1,Villadiego,42.5146862,-4.0101705,http://volveralpueblo.coceder.org/mapa-de-recu...
2,Villahoz,42.0771757,-3.9126371,http://volveralpueblo.coceder.org/mapa-de-recu...
3,Carrión de los condes,42.3307302,-4.6140481,http://volveralpueblo.coceder.org/mapa-de-recu...
4,Frómista,42.2676329,-4.4053497,http://volveralpueblo.coceder.org/mapa-de-recu...
...,...,...,...,...
833,Llames de Parres,43.3553279,-5.2113355,http://volveralpueblo.coceder.org/mapa-de-recu...
834,Monleras,41.1861677,-6.2263127,http://volveralpueblo.coceder.org/mapa-de-recu...
835,El Barco de Ávila,40.3580563,-5.5191156,http://volveralpueblo.coceder.org/mapa-de-recu...
838,Navacepeda de Tormes,40.3604289,-5.2498365,http://volveralpueblo.coceder.org/mapa-de-recu...


In [49]:
# Se corrigen datos de las columnas lat y lng para poder crear un GeoDataFrame

In [50]:
df_volver=df_volver[~(df_volver['lat']=='-')]
df_volver=df_volver[~(df_volver['lng']=='-')]

In [51]:
df_volver['lat'] = df_volver['lat'].apply(lambda x: x.replace('- ','-'))
df_volver['lng'] = df_volver['lng'].apply(lambda x: x.replace('- ','-'))

In [52]:
df_volver['lat'] = df_volver['lat'].apply(lambda x: x.replace(',','.'))
df_volver['lng'] = df_volver['lng'].apply(lambda x: x.replace(',','.'))

In [53]:
df_volver['lat'] = df_volver['lat'].apply(lambda x: x.split(' Latitud')[0])
df_volver['lng'] = df_volver['lng'].apply(lambda x: x.split(' Latitud')[0])

In [54]:
df_volver['lat'] = df_volver['lat'].apply(lambda x: x.rstrip('.'))
df_volver['lng'] = df_volver['lng'].apply(lambda x: x.rstrip('.'))

In [55]:
df_volver

,Municipio,lat,lng,Link_web
0,Peralejos de las Truchas,40.5929386,-1.9093132,http://volveralpueblo.coceder.org/mapa-de-recu...
1,Villadiego,42.5146862,-4.0101705,http://volveralpueblo.coceder.org/mapa-de-recu...
2,Villahoz,42.0771757,-3.9126371,http://volveralpueblo.coceder.org/mapa-de-recu...
3,Carrión de los condes,42.3307302,-4.6140481,http://volveralpueblo.coceder.org/mapa-de-recu...
4,Frómista,42.2676329,-4.4053497,http://volveralpueblo.coceder.org/mapa-de-recu...
...,...,...,...,...
833,Llames de Parres,43.3553279,-5.2113355,http://volveralpueblo.coceder.org/mapa-de-recu...
834,Monleras,41.1861677,-6.2263127,http://volveralpueblo.coceder.org/mapa-de-recu...
835,El Barco de Ávila,40.3580563,-5.5191156,http://volveralpueblo.coceder.org/mapa-de-recu...
838,Navacepeda de Tormes,40.3604289,-5.2498365,http://volveralpueblo.coceder.org/mapa-de-recu...


In [56]:
df_pueblos_web = pd.concat([df_vente,df_volver], ignore_index=True)
df_pueblos_web

,Municipio,Link_web,lat,lng
0,A Estrada,https://venteaviviraunpueblo.com/pueblo/a-estr...,42.689068,-8.482822
1,A Mariña Lucense,https://venteaviviraunpueblo.com/pueblo/a-mari...,43.535297,-7.045202
2,Aceituna,https://venteaviviraunpueblo.com/pueblo/aceituna/,40.149099,-6.330698
3,Aguilar de Campoo,https://venteaviviraunpueblo.com/pueblo/aguila...,42.791388,-4.260055
4,Albares,https://venteaviviraunpueblo.com/pueblo/albares/,40.308621,-3.011926
...,...,...,...,...
713,Llames de Parres,http://volveralpueblo.coceder.org/mapa-de-recu...,43.3553279,-5.2113355
714,Monleras,http://volveralpueblo.coceder.org/mapa-de-recu...,41.1861677,-6.2263127
715,El Barco de Ávila,http://volveralpueblo.coceder.org/mapa-de-recu...,40.3580563,-5.5191156
716,Navacepeda de Tormes,http://volveralpueblo.coceder.org/mapa-de-recu...,40.3604289,-5.2498365


In [57]:
# Se crea un GeoDataFrame para poder utilizar los datos en un mapa
df_map_pueblos = gpd.GeoDataFrame(df_pueblos_web,geometry=gpd.points_from_xy(df_pueblos_web.lng, df_pueblos_web.lat))
df_map_pueblos

,Municipio,Link_web,lat,lng,geometry
0,A Estrada,https://venteaviviraunpueblo.com/pueblo/a-estr...,42.689068,-8.482822,POINT (-8.48282 42.68907)
1,A Mariña Lucense,https://venteaviviraunpueblo.com/pueblo/a-mari...,43.535297,-7.045202,POINT (-7.04520 43.53530)
2,Aceituna,https://venteaviviraunpueblo.com/pueblo/aceituna/,40.149099,-6.330698,POINT (-6.33070 40.14910)
3,Aguilar de Campoo,https://venteaviviraunpueblo.com/pueblo/aguila...,42.791388,-4.260055,POINT (-4.26006 42.79139)
4,Albares,https://venteaviviraunpueblo.com/pueblo/albares/,40.308621,-3.011926,POINT (-3.01193 40.30862)
...,...,...,...,...,...
713,Llames de Parres,http://volveralpueblo.coceder.org/mapa-de-recu...,43.3553279,-5.2113355,POINT (-5.21134 43.35533)
714,Monleras,http://volveralpueblo.coceder.org/mapa-de-recu...,41.1861677,-6.2263127,POINT (-6.22631 41.18617)
715,El Barco de Ávila,http://volveralpueblo.coceder.org/mapa-de-recu...,40.3580563,-5.5191156,POINT (-5.51912 40.35806)
716,Navacepeda de Tormes,http://volveralpueblo.coceder.org/mapa-de-recu...,40.3604289,-5.2498365,POINT (-5.24984 40.36043)


In [58]:
# Se descarga los datos geojon de las provinvias
df_prov = gpd.read_file('./geojson_files/provinces.geojson')

In [59]:
# Se unen las dos tablas para tener en una sola los datos necesarios para el mapa
df_pueblos = gpd.sjoin(df_map_pueblos, df_prov, how="inner", op='intersects')
df_pueblos

,Municipio,Link_web,lat,lng,geometry,index_right,id,name
0,A Estrada,https://venteaviviraunpueblo.com/pueblo/a-estr...,42.689068,-8.482822,POINT (-8.48282 42.68907),40,36,Pontevedra
20,As Neves,https://venteaviviraunpueblo.com/pueblo/as-neves/,42.088129,-8.414637,POINT (-8.41464 42.08813),40,36,Pontevedra
140,Vilanova de Arousa,https://venteaviviraunpueblo.com/pueblo/vilano...,42.563404,-8.827257,POINT (-8.82726 42.56340),40,36,Pontevedra
379,Gondomar,http://volveralpueblo.coceder.org/mapa-de-recu...,42.1252599,-8.7315981,POINT (-8.73160 42.12526),40,36,Pontevedra
443,"Cerdedelo,",http://volveralpueblo.coceder.org/mapa-de-recu...,42.5327602,-8.3903276,POINT (-8.39033 42.53276),40,36,Pontevedra
...,...,...,...,...,...,...,...,...
441,Vilardecervos,http://volveralpueblo.coceder.org/mapa-de-recu...,41.9421627,-6.2867284,POINT (-6.28673 41.94216),28,49,Zamora
473,Villalpando,http://volveralpueblo.coceder.org/mapa-de-recu...,41.8654475,-5.413591,POINT (-5.41359 41.86545),28,49,Zamora
649,Torregamones,http://volveralpueblo.coceder.org/mapa-de-recu...,41.49237310854473,-6.179707697223397,POINT (-6.17971 41.49237),28,49,Zamora
691,Villar del Buey,http://volveralpueblo.coceder.org/mapa-de-recu...,41.3294996,-6.189163,POINT (-6.18916 41.32950),28,49,Zamora


In [60]:
# Se eliminan columnas innecesarias y se renombran algunas columnas.
df_pueblos.drop(columns='index_right', inplace=True)
df_pueblos.rename(columns={ 'id':'Cod_prov','name':'Provincia'}, inplace=True)
df_pueblos

,Municipio,Link_web,lat,lng,geometry,Cod_prov,Provincia
0,A Estrada,https://venteaviviraunpueblo.com/pueblo/a-estr...,42.689068,-8.482822,POINT (-8.48282 42.68907),36,Pontevedra
20,As Neves,https://venteaviviraunpueblo.com/pueblo/as-neves/,42.088129,-8.414637,POINT (-8.41464 42.08813),36,Pontevedra
140,Vilanova de Arousa,https://venteaviviraunpueblo.com/pueblo/vilano...,42.563404,-8.827257,POINT (-8.82726 42.56340),36,Pontevedra
379,Gondomar,http://volveralpueblo.coceder.org/mapa-de-recu...,42.1252599,-8.7315981,POINT (-8.73160 42.12526),36,Pontevedra
443,"Cerdedelo,",http://volveralpueblo.coceder.org/mapa-de-recu...,42.5327602,-8.3903276,POINT (-8.39033 42.53276),36,Pontevedra
...,...,...,...,...,...,...,...
441,Vilardecervos,http://volveralpueblo.coceder.org/mapa-de-recu...,41.9421627,-6.2867284,POINT (-6.28673 41.94216),49,Zamora
473,Villalpando,http://volveralpueblo.coceder.org/mapa-de-recu...,41.8654475,-5.413591,POINT (-5.41359 41.86545),49,Zamora
649,Torregamones,http://volveralpueblo.coceder.org/mapa-de-recu...,41.49237310854473,-6.179707697223397,POINT (-6.17971 41.49237),49,Zamora
691,Villar del Buey,http://volveralpueblo.coceder.org/mapa-de-recu...,41.3294996,-6.189163,POINT (-6.18916 41.32950),49,Zamora


In [61]:
df_pueblos.dtypes

Municipio      object
Link_web       object
lat            object
lng            object
geometry     geometry
Cod_prov       object
Provincia      object
dtype: object

In [62]:
# Se transfomra a int los valores de la columna 'Cod_prov'
df_pueblos['Cod_prov']=df_pueblos['Cod_prov'].apply(int)

### Mapa de todos los pueblos promocionados en las dos webs

In [63]:
# Se llama a una función que muestra en el mapa todos los pueblos recopilados en la tabla

mapa= f.plot_map_pueblos(df_pueblos)

In [64]:
mapa

In [65]:
pd.to_pickle(df_pueblos,'df_prediccion.pkl') #  # Se guarda la tabla para utilizar con streamlit

### Se filtra la tabla con las 3 provincias mas probables de destino y se muestran los pueblos

In [66]:
df_pueblos_dest = df_pueblos[df_pueblos['Cod_prov'].isin(lista_destino)]
df_pueblos_dest

,Municipio,Link_web,lat,lng,geometry,Cod_prov,Provincia
7,Alcaucín,https://venteaviviraunpueblo.com/pueblo/alcaucin/,36.90298352412578,-4.114043326034021,POINT (-4.11404 36.90298),29,Málaga
10,Alfarnate,https://venteaviviraunpueblo.com/pueblo/alfarn...,36.993524,-4.255433,POINT (-4.25543 36.99352),29,Málaga
11,Alfarnatejo,https://venteaviviraunpueblo.com/pueblo/alfarn...,36.979203,-4.271991,POINT (-4.27199 36.97920),29,Málaga
12,Algatocín,https://venteaviviraunpueblo.com/pueblo/algato...,36.573557,-5.275038,POINT (-5.27504 36.57356),29,Málaga
13,Almáchar,https://venteaviviraunpueblo.com/pueblo/almachar/,36.809020,-4.217358,POINT (-4.21736 36.80902),29,Málaga
15,Almargen,https://venteaviviraunpueblo.com/pueblo/almargen/,37.001850,-5.021827,POINT (-5.02183 37.00185),29,Málaga
16,Alozaina,https://venteaviviraunpueblo.com/pueblo/alozaina/,36.727903,-4.858497,POINT (-4.85850 36.72790),29,Málaga
17,Alpandeire,https://venteaviviraunpueblo.com/pueblo/alpand...,36.634042,-5.203519,POINT (-5.20352 36.63404),29,Málaga
19,Arenas,https://venteaviviraunpueblo.com/pueblo/arenas/,36.816212,-4.044055,POINT (-4.04406 36.81621),29,Málaga
23,Benadalid,https://venteaviviraunpueblo.com/pueblo/benada...,36.606089,-5.269379,POINT (-5.26938 36.60609),29,Málaga


In [67]:
mapadest= f.plot_map_pueblos(df_pueblos_dest)

In [68]:
mapadest